In [1]:

import warnings
warnings.filterwarnings('ignore')
from numpy import set_printoptions
from pandas.plotting import scatter_matrix
import matplotlib.pyplot as plt
import numpy as np 
import pandas as pd 
pd.options.display.max_columns=None
import seaborn as sns 
from pandas import read_csv
import io
import base64
import json
from openpyxl import Workbook
from openpyxl.drawing.image import Image as XLImage

import os
import os.path

from urllib.error import HTTPError

from google.auth.transport.requests import Request
from google.oauth2.credentials import Credentials
from google_auth_oauthlib.flow import InstalledAppFlow
from googleapiclient.discovery import build
from googleapiclient.errors import HttpError
from googleapiclient.http import MediaIoBaseDownload


from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import Normalizer
from sklearn.preprocessing import RobustScaler
from sklearn.preprocessing import PowerTransformer

from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import KFold
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score

from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import LogisticRegression 
from catboost import CatBoostClassifier
from xgboost import XGBClassifier
from sklearn.svm import SVC
from lightgbm import LGBMClassifier
from mlens.ensemble import SuperLearner


from sklearn.ensemble import AdaBoostClassifier 
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.ensemble import BaggingClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import VotingClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import StackingClassifier

from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, classification_report
from sklearn.metrics import normalized_mutual_info_score
from sklearn.metrics import cohen_kappa_score


[MLENS] backend: threading


### CARGUE DE DATOS Y CONEXIÓN A DRIVE

In [2]:
SCOPES = ['https://www.googleapis.com/auth/drive.readonly']

creds = None
if os.path.exists('token.json'):
    creds = Credentials.from_authorized_user_file('token.json', SCOPES)
if not creds or not creds.valid:
    if creds and creds.expired and creds.refresh_token:
        creds.refresh(Request())
    else:
        flow = InstalledAppFlow.from_client_secrets_file('C:/Users/Intevo/Desktop/UNIVERSIDAD DISTRITAL PROYECTO FOLDER/credentials.json', SCOPES) # Reemplazar con la ruta correcta
        creds = flow.run_local_server(port=0)
    with open('C:/Users/Intevo/Desktop/UNIVERSIDAD DISTRITAL PROYECTO FOLDER/token.json', 'w') as token:
        token.write(creds.to_json())
        
# Crear una instancia de la API de Drive
drive_service = build('drive', 'v3', credentials=creds)

# ID de la carpeta de Google Drive
folder_id = '1hQeetmO4XIObUefS_nzePqKqq3VksUEC'

# Ruta de destino para guardar los archivos descargados
save_path = 'C:/Users/Intevo/Desktop/UNIVERSIDAD DISTRITAL PROYECTO FOLDER/UNIVERSIDAD-DISTRITAL-PROYECTO/MÓDULO ANALÍTICA PREDICTIVA/DATOS'  # Reemplazar con la ruta deseada

# Función para descargar archivos de la carpeta de Drive
def download_folder(folder_id, save_path):
    results = drive_service.files().list(
        q=f"'{folder_id}' in parents and trashed=false",
        fields='files(id, name)').execute()
    items = results.get('files', [])
    for item in items:
        file_id = item['id']
        file_name = item['name']
        request = drive_service.files().get_media(fileId=file_id)
        fh = io.FileIO(os.path.join(save_path, file_name), 'wb')
        downloader = MediaIoBaseDownload(fh, request)
        done = False
        while done is False:
            status, done = downloader.next_chunk()
            print(f"Descargando {file_name}: {int(status.progress() * 100)}%")
    print("Descarga completa")

# Descargar archivos de la carpeta
download_folder(folder_id, save_path)

# Listar archivos descargados
files = os.listdir(save_path)
print("Archivos descargados:")
for file in files:
    print(file)


Please visit this URL to authorize this application: https://accounts.google.com/o/oauth2/auth?response_type=code&client_id=899973764197-biu188dkvsgi2al0fh29udm7keak0lh0.apps.googleusercontent.com&redirect_uri=http%3A%2F%2Flocalhost%3A58900%2F&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.readonly&state=Jkd8ALkEBTkNU88JR7NXoOB0XVEgij&access_type=offline
Descargando Limpieza de datos 1 UD.ipynb: 100%
Descargando industrial10.csv: 100%
Descargando industrial9.csv: 100%
Descargando industrial8.csv: 100%
Descargando industrial7.csv: 100%
Descargando industrial6.csv: 100%
Descargando industrial5.csv: 100%
Descargando industrial3.csv: 100%
Descargando industrial4.csv: 100%
Descargando industrial2.csv: 100%
Descargando industrial1.csv: 100%
Descarga completa
Archivos descargados:
industrial1.csv
industrial10.csv
industrial2.csv
industrial3.csv
industrial4.csv
industrial5.csv
industrial6.csv
industrial7.csv
industrial8.csv
industrial9.csv
Limpieza de datos 1 UD.ipynb


In [3]:
"""
variables_por_carrera = {
    'industrial': {
        '1': ['PG_ICFES', 'CON_MAT_ICFES', 'FISICA_ICFES','QUIMICA_ICFES','IDIOMA_ICFES','LOCALIDAD','RENDIMIENTO_UNO', 'PROMEDIO_UNO'],
        '2': ['LOCALIDAD_COLEGIO', 'PG_ICFES', 'CON_MAT_ICFES', 'FISICA_ICFES', 'BIOLOGIA_ICFES', 'IDIOMA_ICFES', 'LOCALIDAD', 'PROMEDIO_UNO', 'CAR_UNO', 'NCC_UNO', 'NAA_UNO', 'NOTA_DIFERENCIAL', 'NOTA_DIBUJO', 'NOTA_QUIMICA', 'NOTA_CFJC', 'NOTA_TEXTOS', 'NOTA_SEMINARIO', 'NOTA_EE_UNO','RENDIMIENTO_DOS', 'PROMEDIO_DOS'],
        '3': ['PROMEDIO_UNO', 'NAA_UNO', 'NOTA_DIFERENCIAL', 'NOTA_DIBUJO', 'NOTA_TEXTOS', 'NOTA_EE_UNO', 'PROMEDIO_DOS', 'NCC_DOS', 'NCA_DOS', 'NAA_DOS', 'NOTA_ALGEBRA', 'NOTA_INTEGRAL', 'NOTA_MATERIALES', 'NOTA_PBASICA', 'NOTA_EE_DOS', 'RENDIMIENTO_TRES', 'PROMEDIO_TRES'],
        '4': ['PROMEDIO_UNO', 'NOTA_EE_UNO', 'PROMEDIO_DOS', 'NOTA_ALGEBRA', 'NOTA_INTEGRAL', 'NOTA_MATERIALES', 'NOTA_EE_DOS', 'NAA_TRES', 'NOTA_MULTIVARIADO', 'NOTA_ESTADISTICA_UNO', 'NOTA_TERMODINAMICA', 'NOTA_TGS', 'NOTA_EE_TRES','RENDIMIENTO_CUATRO', 'PROMEDIO_CUATRO'],
        '5': ['PROMEDIO_UNO', 'PROMEDIO_DOS', 'NOTA_ALGEBRA', 'NOTA_INTEGRAL', 'NOTA_MATERIALES', 'NOTA_EE_DOS','PROMEDIO_TRES', 'NAA_TRES', 'NOTA_MULTIVARIADO', 'NOTA_TERMODINAMICA', 'NOTA_ECUACIONES', 'NOTA_ESTADISTICA_DOS', 'NOTA_FISICA_DOS', 'NOTA_MECANICA', 'NOTA_PROCESOSQ', 'RENDIMIENTO_CINCO', 'RENDIMIENTO_CINCO'],
        '6': ['PROMEDIO_UNO', 'PROMEDIO_DOS', 'NOTA_MATERIALES', 'NOTA_EE_DOS', 'PROMEDIO_TRES', 'NOTA_MULTIVARIADO', 'NOTA_FISICA_DOS', 'NOTA_EE_CUATRO', 'PROMEDIO_CINCO', 'NOTA_PROCESOSM', 'NOTA_ADMINISTRACION', 'NOTA_LENGUA_UNO', 'NOTA_EI_UNO', 'NOTA_EI_DOS', 'RENDIMIENTO_SEIS', 'PROMEDIO_SEIS'],
        '7': ['PROMEDIO_UNO', 'PROMEDIO_DOS', 'NOTA_EE_DOS', 'PROMEDIO_TRES', 'NOTA_MULTIVARIADO', 'NOTA_FISICA_DOS', 'PROMEDIO_CINCO', 'NOTA_PROCESOSM', 'NOTA_LENGUA_UNO', 'NOTA_EI_DOS', 'PROMEDIO_SEIS', 'NCA_SEIS', 'NOTA_PLINEAL', 'NOTA_DISE O', 'NOTA_EI_TRES','RENDIMIENTO_SIETE', 'PROMEDIO_SIETE'],
        '8': ['PROMEDIO_DOS', 'NOTA_EE_CUATRO', 'PROMEDIO_CINCO', 'NOTA_LENGUA_UNO','PROMEDIO_SEIS', 'NOTA_IECONOMICA', 'PROMEDIO_SIETE', 'NAA_SIETE', 'NOTA_GRAFOS', 'NOTA_CALIDAD_UNO', 'NOTA_ERGONOMIA', 'NOTA_EI_CINCO', 'RENDIMIENTO_OCHO', 'PROMEDIO_OCHO'],
        '9': ['PROMEDIO_DOS', 'NOTA_EE_CUATRO', 'PROMEDIO_CINCO', 'PROMEDIO_SEIS', 'NOTA_IECONOMICA', 'PROMEDIO_SIETE', 'NAA_SIETE', 'NOTA_GRAFOS', 'NOTA_CALIDAD_UNO', 'NOTA_ERGONOMIA', 'NOTA_EI_CINCO', 'PROMEDIO_OCHO', 'NCC_OCHO', 'NOTA_LOG_UNO', 'NOTA_GOPERACIONES','NOTA_CALIDAD_DOS', 'NOTA_LENGUA_DOS', 'NOTA_CONTEXTO', 'RENDIMIENTO_NUEVE', 'PROMEDIO_NUEVE'],
        '10': ['PROMEDIO_SEIS', 'PROMEDIO_SIETE', 'PROMEDIO_OCHO', 'NOTA_CALIDAD_DOS', 'PROMEDIO_NUEVE', 'NAA_NUEVE', 'NOTA_GRADO_UNO', 'NOTA_LOG_DOS', 'NOTA_FINANZAS', 'NOTA_HISTORIA', 'RENDIMIENTO_DIEZ', 'PROMEDIO_DIEZ']
    },
    'sistemas': {
        '1': ['LOCALIDAD_COLEGIO', 'PG_ICFES', 'CON_MAT_ICFES', 'FISICA_ICFES', 'QUIMICA_ICFES', 'IDIOMA_ICFES', 'LOCALIDAD', 'RENDIMIENTO_UNO', 'PROMEDIO_UNO'],
        '2': ['LOCALIDAD_COLEGIO', 'PG_ICFES', 'CON_MAT_ICFES', 'IDIOMA_ICFES', 'PROMEDIO_UNO', 'NOTA_DIFERENCIAL', 'NOTA_PROG_BASICA', 'NOTA_CATEDRA_FJC', 'NOTA_TEXTOS', 'NOTA_SEMINARIO', 'NOTA_CATEDRA_DEM', 'NOTA_CATEDRA_CON', 'NOTA_LOGICA', 'RENDIMIENTO_DOS', 'PROMEDIO_DOS'],
        '3': ['LOCALIDAD_COLEGIO', 'PG_ICFES', 'CON_MAT_ICFES', 'IDIOMA_ICFES', 'PROMEDIO_UNO', 'PROMEDIO_DOS', 'NOTA_PROG_BASICA', 'NOTA_TEXTOS', 'NOTA_CATEDRA_DEM', 'NOTA_INTEGRAL', 'NOTA_PROG_ORIENTADA', 'NOTA_ETICA', 'RENDIMIENTO_TRES', 'PROMEDIO_TRES'],
        '4': ['LOCALIDAD_COLEGIO', 'PG_ICFES', 'CON_MAT_ICFES', 'IDIOMA_ICFES', 'PROMEDIO_UNO', 'PROMEDIO_DOS', 'PROMEDIO_TRES', 'NOTA_PROG_BASICA', 'NOTA_TEXTOS', 'NOTA_CATEDRA_DEM', 'NOTA_INTEGRAL', 'NOTA_PROG_ORIENTADA', 'NOTA_ETICA', 'NOTA_FISICA_DOS', 'NOTA_TGS', 'NOTA_PROG_AVANZADA', 'RENDIMIENTO_CUATRO', 'PROMEDIO_CUATRO'],
        '5': ['PROMEDIO_UNO', 'PROMEDIO_DOS', 'NOTA_ALGEBRA', 'NOTA_INTEGRAL', 'NOTA_MATERIALES', 'NOTA_EE_DOS', 'PROMEDIO_TRES', 'NAA_TRES', 'NOTA_MULTIVARIADO', 'NOTA_TERMODINAMICA', 'NOTA_ECUACIONES', 'NOTA_ESTADISTICA_DOS', 'NOTA_FISICA_DOS', 'NOTA_MECANICA', 'NOTA_PROCESOSQ', 'RENDIMIENTO_CINCO', 'PROMEDIO_CINCO'],
        '6': ['PROMEDIO_UNO', 'NOTA_EE_UNO', 'PROMEDIO_DOS', 'NOTA_MATERIALES', 'NOTA_EE_DOS', 'PROMEDIO_TRES', 'NOTA_MULTIVARIADO', 'NOTA_ECONOMIA_UNO', 'NOTA_FISICA_DOS', 'NOTA_EE_CUATRO', 'PROMEDIO_CINCO', 'NOTA_PROCESOSM', 'NOTA_ADMINISTRACION', 'NOTA_LENGUA_UNO', 'NOTA_EI_UNO', 'NOTA_EI_DOS', 'RENDIMIENTO_SEIS', 'PROMEDIO_SEIS'],
        '7': ['PROMEDIO_UNO', 'PROMEDIO_DOS', 'NOTA_EE_DOS', 'PROMEDIO_TRES', 'NOTA_MULTIVARIADO', 'NOTA_FISICA_DOS', 'PROMEDIO_CINCO', 'NOTA_PROCESOSM', 'NOTA_LENGUA_UNO', 'NOTA_EI_DOS', 'PROMEDIO_SEIS', 'NCA_SEIS', 'NOTA_PLINEAL', 'NOTA_DISE O', 'NOTA_EI_TRES', 'RENDIMIENTO_SIETE', 'PROMEDIO_SIETE'],
        '8': ['PROMEDIO_UNO', 'PROMEDIO_DOS', 'NOTA_EE_DOS', 'PROMEDIO_TRES', 'NOTA_MULTIVARIADO', 'NOTA_FISICA_DOS', 'NOTA_EE_CUATRO', 'PROMEDIO_CINCO', 'NOTA_LENGUA_UNO', 'PROMEDIO_SEIS', 'NOTA_IECONOMICA', 'PROMEDIO_SIETE', 'NAA_SIETE', 'NOTA_GRAFOS', 'NOTA_CALIDAD_UNO', 'NOTA_ERGONOMIA', 'NOTA_EI_CINCO', 'RENDIMIENTO_OCHO', 'PROMEDIO_OCHO'],
        '9': ['PROMEDIO_DOS', 'NOTA_EE_CUATRO', 'PROMEDIO_CINCO', 'PROMEDIO_SEIS', 'NOTA_IECONOMICA', 'PROMEDIO_SIETE', 'NAA_SIETE', 'NOTA_GRAFOS', 'NOTA_CALIDAD_UNO', 'NOTA_ERGONOMIA', 'NOTA_EI_CINCO', 'PROMEDIO_OCHO', 'NCC_OCHO', 'NOTA_LOG_UNO', 'NOTA_GOPERACIONES', 'NOTA_CALIDAD_DOS', 'NOTA_LENGUA_DOS', 'NOTA_CONTEXTO', 'RENDIMIENTO_NUEVE', 'PROMEDIO_NUEVE'],
        '10': ['PROMEDIO_SEIS', 'PROMEDIO_SIETE', 'PROMEDIO_OCHO', 'NOTA_CALIDAD_DOS', 'PROMEDIO_NUEVE', 'NAA_NUEVE', 'NOTA_GRADO_UNO', 'NOTA_LOG_DOS', 'NOTA_FINANZAS', 'NOTA_HISTORIA', 'RENDIMIENTO_DIEZ', 'PROMEDIO_DIEZ']
    },
    'catastral': {
        '1': ['variable1_catastral', 'variable2_catastral', 'variable3_catastral'],
        '2': ['variable4_catastral', 'variable5_catastral', 'variable6_catastral']
    }
}
"""

"\nvariables_por_carrera = {\n    'industrial': {\n        '1': ['PG_ICFES', 'CON_MAT_ICFES', 'FISICA_ICFES','QUIMICA_ICFES','IDIOMA_ICFES','LOCALIDAD','RENDIMIENTO_UNO', 'PROMEDIO_UNO'],\n        '2': ['LOCALIDAD_COLEGIO', 'PG_ICFES', 'CON_MAT_ICFES', 'FISICA_ICFES', 'BIOLOGIA_ICFES', 'IDIOMA_ICFES', 'LOCALIDAD', 'PROMEDIO_UNO', 'CAR_UNO', 'NCC_UNO', 'NAA_UNO', 'NOTA_DIFERENCIAL', 'NOTA_DIBUJO', 'NOTA_QUIMICA', 'NOTA_CFJC', 'NOTA_TEXTOS', 'NOTA_SEMINARIO', 'NOTA_EE_UNO','RENDIMIENTO_DOS', 'PROMEDIO_DOS'],\n        '3': ['PROMEDIO_UNO', 'NAA_UNO', 'NOTA_DIFERENCIAL', 'NOTA_DIBUJO', 'NOTA_TEXTOS', 'NOTA_EE_UNO', 'PROMEDIO_DOS', 'NCC_DOS', 'NCA_DOS', 'NAA_DOS', 'NOTA_ALGEBRA', 'NOTA_INTEGRAL', 'NOTA_MATERIALES', 'NOTA_PBASICA', 'NOTA_EE_DOS', 'RENDIMIENTO_TRES', 'PROMEDIO_TRES'],\n        '4': ['PROMEDIO_UNO', 'NOTA_EE_UNO', 'PROMEDIO_DOS', 'NOTA_ALGEBRA', 'NOTA_INTEGRAL', 'NOTA_MATERIALES', 'NOTA_EE_DOS', 'NAA_TRES', 'NOTA_MULTIVARIADO', 'NOTA_ESTADISTICA_UNO', 'NOTA_TERMODINAMICA', 'NO

In [4]:
variables_por_carrera = {
    'industrial': {
        '1': ['PG_ICFES', 'CON_MAT_ICFES', 'FISICA_ICFES','QUIMICA_ICFES','IDIOMA_ICFES','LOCALIDAD','RENDIMIENTO_UNO'],
        '2': ['LOCALIDAD_COLEGIO', 'PG_ICFES', 'CON_MAT_ICFES', 'FISICA_ICFES', 'BIOLOGIA_ICFES', 'IDIOMA_ICFES', 'LOCALIDAD', 'PROMEDIO_UNO', 'CAR_UNO', 'NCC_UNO', 'NAA_UNO', 'NOTA_DIFERENCIAL', 'NOTA_DIBUJO', 'NOTA_QUIMICA', 'NOTA_CFJC', 'NOTA_TEXTOS', 'NOTA_SEMINARIO', 'NOTA_EE_UNO','RENDIMIENTO_DOS'],
        '3': ['PROMEDIO_UNO', 'NAA_UNO', 'NOTA_DIFERENCIAL', 'NOTA_DIBUJO', 'NOTA_TEXTOS', 'NOTA_EE_UNO', 'PROMEDIO_DOS', 'NCC_DOS', 'NCA_DOS', 'NAA_DOS', 'NOTA_ALGEBRA', 'NOTA_INTEGRAL', 'NOTA_MATERIALES', 'NOTA_PBASICA', 'NOTA_EE_DOS', 'RENDIMIENTO_TRES'],
        '4': ['PROMEDIO_UNO', 'NOTA_EE_UNO', 'PROMEDIO_DOS', 'NOTA_ALGEBRA', 'NOTA_INTEGRAL', 'NOTA_MATERIALES', 'NOTA_EE_DOS', 'NAA_TRES', 'NOTA_MULTIVARIADO', 'NOTA_ESTADISTICA_UNO', 'NOTA_TERMODINAMICA', 'NOTA_TGS', 'NOTA_EE_TRES','RENDIMIENTO_CUATRO'],
        '5': ['PROMEDIO_UNO', 'PROMEDIO_DOS', 'NOTA_ALGEBRA', 'NOTA_INTEGRAL', 'NOTA_MATERIALES', 'NOTA_EE_DOS','PROMEDIO_TRES', 'NAA_TRES', 'NOTA_MULTIVARIADO', 'NOTA_TERMODINAMICA', 'NOTA_ECUACIONES', 'NOTA_ESTADISTICA_DOS', 'NOTA_FISICA_DOS', 'NOTA_MECANICA', 'NOTA_PROCESOSQ', 'RENDIMIENTO_CINCO', 'RENDIMIENTO_CINCO'],
        '6': ['PROMEDIO_UNO', 'PROMEDIO_DOS', 'NOTA_MATERIALES', 'NOTA_EE_DOS', 'PROMEDIO_TRES', 'NOTA_MULTIVARIADO', 'NOTA_FISICA_DOS', 'NOTA_EE_CUATRO', 'PROMEDIO_CINCO', 'NOTA_PROCESOSM', 'NOTA_ADMINISTRACION', 'NOTA_LENGUA_UNO', 'NOTA_EI_UNO', 'NOTA_EI_DOS', 'RENDIMIENTO_SEIS'],
        '7': ['PROMEDIO_UNO', 'PROMEDIO_DOS', 'NOTA_EE_DOS', 'PROMEDIO_TRES', 'NOTA_MULTIVARIADO', 'NOTA_FISICA_DOS', 'PROMEDIO_CINCO', 'NOTA_PROCESOSM', 'NOTA_LENGUA_UNO', 'NOTA_EI_DOS', 'PROMEDIO_SEIS', 'NCA_SEIS', 'NOTA_PLINEAL', 'NOTA_DISE O', 'NOTA_EI_TRES','RENDIMIENTO_SIETE'],
        '8': ['PROMEDIO_DOS', 'NOTA_EE_CUATRO', 'PROMEDIO_CINCO', 'NOTA_LENGUA_UNO','PROMEDIO_SEIS', 'NOTA_IECONOMICA', 'PROMEDIO_SIETE', 'NAA_SIETE', 'NOTA_GRAFOS', 'NOTA_CALIDAD_UNO', 'NOTA_ERGONOMIA', 'NOTA_EI_CINCO', 'RENDIMIENTO_OCHO'],
        '9': ['PROMEDIO_DOS', 'NOTA_EE_CUATRO', 'PROMEDIO_CINCO', 'PROMEDIO_SEIS', 'NOTA_IECONOMICA', 'PROMEDIO_SIETE', 'NAA_SIETE', 'NOTA_GRAFOS', 'NOTA_CALIDAD_UNO', 'NOTA_ERGONOMIA', 'NOTA_EI_CINCO', 'PROMEDIO_OCHO', 'NCC_OCHO', 'NOTA_LOG_UNO', 'NOTA_GOPERACIONES','NOTA_CALIDAD_DOS', 'NOTA_LENGUA_DOS', 'NOTA_CONTEXTO', 'RENDIMIENTO_NUEVE'],
        '10': ['PROMEDIO_SEIS', 'PROMEDIO_SIETE', 'PROMEDIO_OCHO', 'NOTA_CALIDAD_DOS', 'PROMEDIO_NUEVE', 'NAA_NUEVE', 'NOTA_GRADO_UNO', 'NOTA_LOG_DOS', 'NOTA_FINANZAS', 'NOTA_HISTORIA', 'RENDIMIENTO_DIEZ']
    },
    'sistemas': {
        '1': ['LOCALIDAD_COLEGIO', 'PG_ICFES', 'CON_MAT_ICFES', 'FISICA_ICFES', 'QUIMICA_ICFES', 'IDIOMA_ICFES', 'LOCALIDAD', 'RENDIMIENTO_UNO'],
        '2': ['LOCALIDAD_COLEGIO', 'PG_ICFES', 'CON_MAT_ICFES', 'IDIOMA_ICFES', 'PROMEDIO_UNO', 'NOTA_DIFERENCIAL', 'NOTA_PROG_BASICA', 'NOTA_CATEDRA_FJC', 'NOTA_TEXTOS', 'NOTA_SEMINARIO', 'NOTA_CATEDRA_DEM', 'NOTA_CATEDRA_CON', 'NOTA_LOGICA', 'RENDIMIENTO_DOS'],
        '3': ['LOCALIDAD_COLEGIO', 'PG_ICFES', 'CON_MAT_ICFES', 'IDIOMA_ICFES', 'PROMEDIO_UNO', 'PROMEDIO_DOS', 'NOTA_PROG_BASICA', 'NOTA_TEXTOS', 'NOTA_CATEDRA_DEM', 'NOTA_INTEGRAL', 'NOTA_PROG_ORIENTADA', 'NOTA_ETICA', 'RENDIMIENTO_TRES'],
        '4': ['LOCALIDAD_COLEGIO', 'PG_ICFES', 'CON_MAT_ICFES', 'IDIOMA_ICFES', 'PROMEDIO_UNO', 'PROMEDIO_DOS', 'PROMEDIO_TRES', 'NOTA_PROG_BASICA', 'NOTA_TEXTOS', 'NOTA_CATEDRA_DEM', 'NOTA_INTEGRAL', 'NOTA_PROG_ORIENTADA', 'NOTA_ETICA', 'NOTA_FISICA_DOS', 'NOTA_TGS', 'NOTA_PROG_AVANZADA', 'RENDIMIENTO_CUATRO'],
        '5': ['PROMEDIO_UNO', 'PROMEDIO_DOS', 'NOTA_ALGEBRA', 'NOTA_INTEGRAL', 'NOTA_MATERIALES', 'NOTA_EE_DOS', 'PROMEDIO_TRES', 'NAA_TRES', 'NOTA_MULTIVARIADO', 'NOTA_TERMODINAMICA', 'NOTA_ECUACIONES', 'NOTA_ESTADISTICA_DOS', 'NOTA_FISICA_DOS', 'NOTA_MECANICA', 'NOTA_PROCESOSQ', 'RENDIMIENTO_CINCO'],
        '6': ['PROMEDIO_UNO', 'NOTA_EE_UNO', 'PROMEDIO_DOS', 'NOTA_MATERIALES', 'NOTA_EE_DOS', 'PROMEDIO_TRES', 'NOTA_MULTIVARIADO', 'NOTA_ECONOMIA_UNO', 'NOTA_FISICA_DOS', 'NOTA_EE_CUATRO', 'PROMEDIO_CINCO', 'NOTA_PROCESOSM', 'NOTA_ADMINISTRACION', 'NOTA_LENGUA_UNO', 'NOTA_EI_UNO', 'NOTA_EI_DOS', 'RENDIMIENTO_SEIS'],
        '7': ['PROMEDIO_UNO', 'PROMEDIO_DOS', 'NOTA_EE_DOS', 'PROMEDIO_TRES', 'NOTA_MULTIVARIADO', 'NOTA_FISICA_DOS', 'PROMEDIO_CINCO', 'NOTA_PROCESOSM', 'NOTA_LENGUA_UNO', 'NOTA_EI_DOS', 'PROMEDIO_SEIS', 'NCA_SEIS', 'NOTA_PLINEAL', 'NOTA_DISE O', 'NOTA_EI_TRES', 'RENDIMIENTO_SIETE'],
        '8': ['PROMEDIO_UNO', 'PROMEDIO_DOS', 'NOTA_EE_DOS', 'PROMEDIO_TRES', 'NOTA_MULTIVARIADO', 'NOTA_FISICA_DOS', 'NOTA_EE_CUATRO', 'PROMEDIO_CINCO', 'NOTA_LENGUA_UNO', 'PROMEDIO_SEIS', 'NOTA_IECONOMICA', 'PROMEDIO_SIETE', 'NAA_SIETE', 'NOTA_GRAFOS', 'NOTA_CALIDAD_UNO', 'NOTA_ERGONOMIA', 'NOTA_EI_CINCO', 'RENDIMIENTO_OCHO'],
        '9': ['PROMEDIO_DOS', 'NOTA_EE_CUATRO', 'PROMEDIO_CINCO', 'PROMEDIO_SEIS', 'NOTA_IECONOMICA', 'PROMEDIO_SIETE', 'NAA_SIETE', 'NOTA_GRAFOS', 'NOTA_CALIDAD_UNO', 'NOTA_ERGONOMIA', 'NOTA_EI_CINCO', 'PROMEDIO_OCHO', 'NCC_OCHO', 'NOTA_LOG_UNO', 'NOTA_GOPERACIONES', 'NOTA_CALIDAD_DOS', 'NOTA_LENGUA_DOS', 'NOTA_CONTEXTO', 'RENDIMIENTO_NUEVE'],
        '10': ['PROMEDIO_SEIS', 'PROMEDIO_SIETE', 'PROMEDIO_OCHO', 'NOTA_CALIDAD_DOS', 'PROMEDIO_NUEVE', 'NAA_NUEVE', 'NOTA_GRADO_UNO', 'NOTA_LOG_DOS', 'NOTA_FINANZAS', 'NOTA_HISTORIA', 'RENDIMIENTO_DIEZ']
    },
    'catastral': {
        '1': ['variable1_catastral', 'variable2_catastral', 'variable3_catastral'],
        '2': ['variable4_catastral', 'variable5_catastral', 'variable6_catastral']
    }
}

In [5]:
def cargar_datos(carrera, semestre):
    
    ruta_archivo = f'C:/Users/Intevo/Desktop/UNIVERSIDAD DISTRITAL PROYECTO FOLDER/UNIVERSIDAD-DISTRITAL-PROYECTO/MÓDULO ANALÍTICA PREDICTIVA/DATOS/{carrera}{semestre}.csv'
    datos = pd.read_csv(ruta_archivo,sep=";")
    
    return datos
carrera="industrial"
semestre="2"

In [6]:
datos = cargar_datos(carrera, semestre)
columnas_filtradas = variables_por_carrera[carrera][semestre]
df = datos[columnas_filtradas]
print("DataFrame con columnas filtradas:")
df=df.astype(int)
df

DataFrame con columnas filtradas:


,LOCALIDAD_COLEGIO,PG_ICFES,CON_MAT_ICFES,FISICA_ICFES,BIOLOGIA_ICFES,IDIOMA_ICFES,LOCALIDAD,PROMEDIO_UNO,CAR_UNO,NCC_UNO,NAA_UNO,NOTA_DIFERENCIAL,NOTA_DIBUJO,NOTA_QUIMICA,NOTA_CFJC,NOTA_TEXTOS,NOTA_SEMINARIO,NOTA_EE_UNO,RENDIMIENTO_DOS
0,20,472,65,63,59,62,8,39,0,9,4,35,0,45,40,0,40,0,1
1,0,478,62,58,59,53,8,31,1,19,6,22,30,40,35,30,40,36,2
2,20,465,78,66,49,88,19,30,1,21,5,30,23,35,40,15,40,40,1
3,10,491,72,56,65,53,10,31,1,21,5,20,40,35,40,30,40,0,2
4,20,464,67,64,62,53,20,36,0,15,7,30,35,40,40,35,35,45,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2634,0,342,68,0,68,58,11,31,0,10,4,24,33,0,35,37,43,0,1
2635,0,347,69,0,71,71,19,36,0,15,6,41,28,30,38,39,48,38,1
2636,0,348,67,0,67,68,11,29,0,13,4,25,36,30,30,27,37,0,1
2637,0,340,70,0,67,63,11,31,0,10,4,28,33,0,34,30,39,0,1


In [7]:
def numero_a_letras(numero):
    numeros_letras = ['cero', 'uno', 'dos', 'tres', 'cuatro', 'cinco', 'seis', 'siete', 'ocho', 'nueve', 'diez']
    return numeros_letras[int(numero)]

semestre_en_letras = numero_a_letras(semestre)
print(semestre_en_letras)


dos


In [8]:
X = df.loc[:, ~df.columns.str.contains(f'RENDIMIENTO_{semestre_en_letras.upper()}')]
Y = df.loc[:, df.columns.str.contains(f'RENDIMIENTO_{semestre_en_letras.upper()}')]                                                     
print("Separación de datos usando Pandas") 
print(X.shape, Y.shape)

Separación de datos usando Pandas
(2639, 18) (2639, 1)


In [9]:
#X = X.astype('float32')                         
Y = LabelEncoder().fit_transform(Y.astype('str'))                
print(X.shape, Y.shape)

(2639, 18) (2639,)


In [10]:
from collections import Counter
frecuencias = Counter(Y)
for valor, frecuencia in frecuencias.items():
    print(f"El valor {valor} aparece {frecuencia} veces")

El valor 0 aparece 1121 veces
El valor 1 aparece 1412 veces
El valor 2 aparece 103 veces
El valor 3 aparece 3 veces


### TRANSFORMACIÓN DE DATOS

In [11]:
X_T_JOHNSON1 = X.copy(deep=True)
def transformacion_johnson(X):
    transformador_johnson = PowerTransformer(method='yeo-johnson', standardize=True).fit(X)
    datos_transformados = transformador_johnson.transform(X)
    set_printoptions(precision=3)
    print(datos_transformados[:5, :])
    datos_transformados_df = pd.DataFrame(data=datos_transformados, columns=X.columns)
    return datos_transformados_df
Xpandas_T_JOHNSON1 = transformacion_johnson(X_T_JOHNSON1)
Xpandas_T_JOHNSON1.head(2)

[[ 1.220e+00  1.211e+00 -6.616e-01  7.632e-01 -3.454e-01 -1.764e-03
  -6.866e-01  1.419e+00 -1.149e+00 -2.320e+00 -5.930e-01  9.884e-01
  -2.309e+00  2.011e+00  4.505e-01 -2.261e+00  4.796e-01 -1.137e+00]
 [-1.675e+00  1.339e+00 -9.879e-01  6.658e-01 -3.454e-01 -7.067e-01
  -6.866e-01 -1.514e-01  1.494e-01  2.141e-01  5.492e-01 -5.259e-01
  -2.823e-01  1.378e+00 -3.777e-01 -4.599e-01  4.796e-01  7.941e-01]
 [ 1.220e+00  1.062e+00  7.691e-01  8.197e-01 -1.557e+00  2.110e+00
   1.199e+00 -3.103e-01  1.494e-01  5.505e-01 -5.574e-02  3.774e-01
  -9.903e-01  7.732e-01  4.505e-01 -1.622e+00  4.796e-01  8.704e-01]
 [-2.297e-02  1.622e+00  1.055e-01  6.257e-01  3.520e-01 -7.067e-01
  -3.062e-01 -1.514e-01  1.494e-01  5.505e-01 -5.574e-02 -7.357e-01
   9.297e-01  7.732e-01  4.505e-01 -4.599e-01  4.796e-01 -1.137e+00]
 [ 1.220e+00  1.041e+00 -4.432e-01  7.822e-01  5.774e-03 -7.067e-01
   1.352e+00  7.658e-01 -1.149e+00 -5.846e-01  1.219e+00  3.774e-01
   2.954e-01  1.378e+00  4.505e-01  1.394e-0

,LOCALIDAD_COLEGIO,PG_ICFES,CON_MAT_ICFES,FISICA_ICFES,BIOLOGIA_ICFES,IDIOMA_ICFES,LOCALIDAD,PROMEDIO_UNO,CAR_UNO,NCC_UNO,NAA_UNO,NOTA_DIFERENCIAL,NOTA_DIBUJO,NOTA_QUIMICA,NOTA_CFJC,NOTA_TEXTOS,NOTA_SEMINARIO,NOTA_EE_UNO
0,1.219622,1.210515,-0.661598,0.763182,-0.345439,-0.001764,-0.686598,1.419203,-1.148708,-2.319518,-0.592955,0.988445,-2.309482,2.011394,0.450509,-2.260658,0.479615,-1.136816
1,-1.674845,1.339257,-0.987923,0.665829,-0.345439,-0.706712,-0.686598,-0.151358,0.149437,0.214105,0.549199,-0.525931,-0.282291,1.377582,-0.377682,-0.459859,0.479615,0.794095


In [12]:
"""
from imblearn.over_sampling import RandomOverSampler
from sklearn.model_selection import train_test_split

# Crear una instancia de RandomOverSampler
ros = RandomOverSampler(sampling_strategy='auto', random_state=42)

# Aplicar RandomOverSampler para generar muestras sintéticas de las clases minoritarias
Xpandas_T_JOHNSON1, Y = ros.fit_resample(Xpandas_T_JOHNSON1, Y)

# Imprimir la distribución de clases después del oversampling
print("Distribución de clases después del oversampling:", Counter(Y))
"""


'\nfrom imblearn.over_sampling import RandomOverSampler\nfrom sklearn.model_selection import train_test_split\n\n# Crear una instancia de RandomOverSampler\nros = RandomOverSampler(sampling_strategy=\'auto\', random_state=42)\n\n# Aplicar RandomOverSampler para generar muestras sintéticas de las clases minoritarias\nXpandas_T_JOHNSON1, Y = ros.fit_resample(Xpandas_T_JOHNSON1, Y)\n\n# Imprimir la distribución de clases después del oversampling\nprint("Distribución de clases después del oversampling:", Counter(Y))\n'

In [13]:
X_trn, X_tst, Y_trn, Y_tst = train_test_split(Xpandas_T_JOHNSON1, Y, test_size=0.3, random_state=2)
print('DATOS: Son {} datos para entrenamiento y {} datos para prueba'.format(X_trn.shape[0], X_tst.shape[0]))

DATOS: Son 1847 datos para entrenamiento y 792 datos para prueba


In [14]:
"""
from imblearn.over_sampling import RandomOverSampler
from sklearn.model_selection import train_test_split

# Crear una instancia de RandomOverSampler
ros = RandomOverSampler(sampling_strategy='auto', random_state=42)

# Aplicar RandomOverSampler para generar muestras sintéticas de las clases minoritarias
X_trn,Y_trn = ros.fit_resample(X_trn, Y_trn)

# Imprimir la distribución de clases después del oversampling
print("Distribución de clases después del oversampling:", Counter(Y_trn))

"""

'\nfrom imblearn.over_sampling import RandomOverSampler\nfrom sklearn.model_selection import train_test_split\n\n# Crear una instancia de RandomOverSampler\nros = RandomOverSampler(sampling_strategy=\'auto\', random_state=42)\n\n# Aplicar RandomOverSampler para generar muestras sintéticas de las clases minoritarias\nX_trn,Y_trn = ros.fit_resample(X_trn, Y_trn)\n\n# Imprimir la distribución de clases después del oversampling\nprint("Distribución de clases después del oversampling:", Counter(Y_trn))\n\n'

In [15]:
from collections import Counter
frecuencias = Counter(Y_trn)   
for valor, frecuencia in frecuencias.items():
    print(f"El valor {valor} aparece {frecuencia} veces")   

El valor 1 aparece 988 veces
El valor 0 aparece 783 veces
El valor 2 aparece 73 veces
El valor 3 aparece 3 veces


### KNEIGHBORSCLASSIFIER

In [16]:
def entrenar_modelo_knn_con_transformacion(X_trn, Y_trn):
    X_trn_transformado = X_trn
    parameters = {
        'n_neighbors': [i for i in range(14, 18, 1)],
        'metric': ['euclidean', 'manhattan', 'minkowski'],
        'algorithm': ['auto'],
        'p': [i for i in range(1, 6)],
        'weights': ['uniform']
    }
    modelo = KNeighborsClassifier()
    semilla = 5
    num_folds = 10
    kfold = KFold(n_splits=num_folds, random_state=semilla, shuffle=True)
    metrica = 'accuracy'
    grid = GridSearchCV(estimator=modelo, param_grid=parameters, scoring=metrica, cv=kfold, n_jobs=-1)
    grid_resultado = grid.fit(X_trn_transformado, Y_trn)
    print("Resultados de GridSearchCV para el modelo")
    print("Mejor valor EXACTITUD usando k-fold:", grid_resultado.best_score_*100)
    print("Mejor valor PARAMETRO usando k-fold:", grid_resultado.best_params_)

    # Entrenar el modelo con los mejores hiperparámetros
    mejor_modelo = KNeighborsClassifier(**grid_resultado.best_params_)
    mejor_modelo.fit(X_trn_transformado, Y_trn)
    return mejor_modelo

X_trn = X_trn
Y_trn = Y_trn 

modelo_knn = entrenar_modelo_knn_con_transformacion(X_trn, Y_trn)

Resultados de GridSearchCV para el modelo
Mejor valor EXACTITUD usando k-fold: 82.88807285546416
Mejor valor PARAMETRO usando k-fold: {'algorithm': 'auto', 'metric': 'minkowski', 'n_neighbors': 14, 'p': 3, 'weights': 'uniform'}


In [17]:
# Predecir las etiquetas para los datos de prueba
resultados_df_knn = pd.DataFrame(columns=['MÉTRICA', 'VALOR'])

Y_pred_entrenamiento= modelo_knn.predict(X_trn)

precision = precision_score(Y_trn, Y_pred_entrenamiento, average='weighted')
recall = recall_score(Y_trn, Y_pred_entrenamiento, average='weighted')
f1 = f1_score(Y_trn, Y_pred_entrenamiento, average='weighted')
accuracy = accuracy_score(Y_trn, Y_pred_entrenamiento)
nmi = normalized_mutual_info_score(Y_trn, Y_pred_entrenamiento)
kappa = cohen_kappa_score(Y_trn, Y_pred_entrenamiento)
print("Precisión: ", round(precision*100,2))
print("Exhaustividad: ", round(recall*100,2))
print("Puntuación F1: ", round(f1*100,2))
print("Exactitud: ", round(accuracy*100,2))
print("Información Mutua Normalizada (NMI):", round(nmi*100,2))
print("Índice Kappa de Cohen:", round(kappa*100,2))

# Crear un DataFrame para cada métrica
df_precision = pd.DataFrame({'MÉTRICA': ['Precisión'], 'VALOR': [round(precision*100, 2)]})
df_recall = pd.DataFrame({'MÉTRICA': ['Exhaustividad'], 'VALOR': [round(recall*100, 2)]})
df_f1 = pd.DataFrame({'MÉTRICA': ['Puntuación F1'], 'VALOR': [round(f1*100, 2)]})
df_accuracy = pd.DataFrame({'MÉTRICA': ['Exactitud'], 'VALOR': [round(accuracy*100, 2)]})
df_nmi=pd.DataFrame({'MÉTRICA': ['Información Mutua Normalizada (NMI)'], 'VALOR': [round(nmi*100, 2)]})
df_kappa=pd.DataFrame({'MÉTRICA': ['Índice Kappa de Cohen'], 'VALOR': [round(kappa*100, 2)]})

# Concatenar los DataFrames
resultados_df_knn_entrenamiento = pd.concat([resultados_df_knn, df_precision, df_recall, df_f1, df_accuracy,df_nmi,df_kappa], ignore_index=True)
resultados_df_knn_entrenamiento["MODELO"]='KNeighborsClassifier'
resultados_df_knn_entrenamiento["TIPO_DE_DATOS"]='Entrenamiento'
# Imprimir el DataFrame con los resultados
resultados_df_knn_entrenamiento


Precisión:  81.07
Exhaustividad:  84.14
Puntuación F1:  82.35
Exactitud:  84.14
Información Mutua Normalizada (NMI): 42.57
Índice Kappa de Cohen: 68.8


,MÉTRICA,VALOR,MODELO,TIPO_DE_DATOS
0,Precisión,81.07,KNeighborsClassifier,Entrenamiento
1,Exhaustividad,84.14,KNeighborsClassifier,Entrenamiento
2,Puntuación F1,82.35,KNeighborsClassifier,Entrenamiento
3,Exactitud,84.14,KNeighborsClassifier,Entrenamiento
4,Información Mutua Normalizada (NMI),42.57,KNeighborsClassifier,Entrenamiento
5,Índice Kappa de Cohen,68.80,KNeighborsClassifier,Entrenamiento


In [18]:
# Predecir las etiquetas para los datos de prueba
resultados_df_knn = pd.DataFrame(columns=['MÉTRICA', 'VALOR'])

Y_pred_prueba = modelo_knn.predict(X_tst)

precision = precision_score(Y_tst, Y_pred_prueba, average='weighted')
recall = recall_score(Y_tst, Y_pred_prueba, average='weighted')
f1 = f1_score(Y_tst, Y_pred_prueba, average='weighted')
accuracy = accuracy_score(Y_tst, Y_pred_prueba)
nmi = normalized_mutual_info_score(Y_tst, Y_pred_prueba)
kappa = cohen_kappa_score(Y_tst, Y_pred_prueba)
print("Precisión: ", round(precision*100,2))
print("Exhaustividad: ", round(recall*100,2))
print("Puntuación F1: ", round(f1*100,2))
print("Exactitud: ", round(accuracy*100,2))
print("Información Mutua Normalizada (NMI):", round(nmi*100,2))
print("Índice Kappa de Cohen:", round(kappa*100,2))

# Crear un DataFrame para cada métrica
df_precision = pd.DataFrame({'MÉTRICA': ['Precisión'], 'VALOR': [round(precision*100, 2)]})
df_recall = pd.DataFrame({'MÉTRICA': ['Exhaustividad'], 'VALOR': [round(recall*100, 2)]})
df_f1 = pd.DataFrame({'MÉTRICA': ['Puntuación F1'], 'VALOR': [round(f1*100, 2)]})
df_accuracy = pd.DataFrame({'MÉTRICA': ['Exactitud'], 'VALOR': [round(accuracy*100, 2)]})
df_nmi=pd.DataFrame({'MÉTRICA': ['Información Mutua Normalizada (NMI)'], 'VALOR': [round(nmi*100, 2)]})
df_kappa=pd.DataFrame({'MÉTRICA': ['Índice Kappa de Cohen'], 'VALOR': [round(kappa*100, 2)]})

# Concatenar los DataFrames
resultados_df_knn_prueba = pd.concat([resultados_df_knn, df_precision, df_recall, df_f1, df_accuracy,df_nmi,df_kappa], ignore_index=True)
resultados_df_knn_prueba["MODELO"]='KNeighborsClassifier'
resultados_df_knn_prueba["TIPO_DE_DATOS"]='Prueba'
# Imprimir el DataFrame con los resultados
resultados_df_knn_prueba

Precisión:  78.17
Exhaustividad:  80.68
Puntuación F1:  79.01
Exactitud:  80.68
Información Mutua Normalizada (NMI): 33.32
Índice Kappa de Cohen: 61.77


,MÉTRICA,VALOR,MODELO,TIPO_DE_DATOS
0,Precisión,78.17,KNeighborsClassifier,Prueba
1,Exhaustividad,80.68,KNeighborsClassifier,Prueba
2,Puntuación F1,79.01,KNeighborsClassifier,Prueba
3,Exactitud,80.68,KNeighborsClassifier,Prueba
4,Información Mutua Normalizada (NMI),33.32,KNeighborsClassifier,Prueba
5,Índice Kappa de Cohen,61.77,KNeighborsClassifier,Prueba


In [19]:
resultados_df_knn = pd.concat([resultados_df_knn_prueba,resultados_df_knn_entrenamiento], ignore_index=True)
resultados_df_knn

,MÉTRICA,VALOR,MODELO,TIPO_DE_DATOS
0,Precisión,78.17,KNeighborsClassifier,Prueba
1,Exhaustividad,80.68,KNeighborsClassifier,Prueba
2,Puntuación F1,79.01,KNeighborsClassifier,Prueba
3,Exactitud,80.68,KNeighborsClassifier,Prueba
4,Información Mutua Normalizada (NMI),33.32,KNeighborsClassifier,Prueba
5,Índice Kappa de Cohen,61.77,KNeighborsClassifier,Prueba
6,Precisión,81.07,KNeighborsClassifier,Entrenamiento
7,Exhaustividad,84.14,KNeighborsClassifier,Entrenamiento
8,Puntuación F1,82.35,KNeighborsClassifier,Entrenamiento
9,Exactitud,84.14,KNeighborsClassifier,Entrenamiento


### SVC

In [20]:
def entrenar_modelo_svc_con_transformacion(X_trn, Y_trn):
    X_trn_transformado = X_trn
    parameters = { 'kernel':  ['rbf'], 
            'C': [i/10000 for i in range(8,12,1)],
            'max_iter':[i for i in range(1,3,1)],
            'gamma' : [i/100 for i in range(90,110,5)],
            'random_state':[i for i in range(1,5,1)]}
    modelo = SVC()
    semilla=5
    num_folds=10
    kfold = KFold(n_splits=num_folds, random_state=semilla, shuffle=True)
    metrica = 'accuracy'
    grid = GridSearchCV(estimator=modelo, param_grid=parameters, scoring=metrica, cv=kfold, n_jobs=-1)
    grid_resultado = grid.fit(X_trn, Y_trn)
    print("Resultados de GridSearchCV para el modelo")
    print("Mejor valor EXACTITUD usando k-fold:", grid_resultado.best_score_*100)
    print("Mejor valor PARAMETRO usando k-fold:", grid_resultado.best_params_)
    mejor_modelo = SVC(**grid_resultado.best_params_)
    mejor_modelo.fit(X_trn_transformado, Y_trn)
    return mejor_modelo

X_trn = X_trn
Y_trn = Y_trn 

modelo_svc = entrenar_modelo_svc_con_transformacion(X_trn, Y_trn)

Resultados de GridSearchCV para el modelo
Mejor valor EXACTITUD usando k-fold: 42.662162162162176
Mejor valor PARAMETRO usando k-fold: {'C': 0.0008, 'gamma': 0.9, 'kernel': 'rbf', 'max_iter': 2, 'random_state': 1}


In [21]:
# Predecir las etiquetas para los datos de prueba
resultados_df_SVC = pd.DataFrame(columns=['MÉTRICA', 'VALOR'])

Y_pred_entrenamiento= modelo_svc.predict(X_trn)

precision = precision_score(Y_trn, Y_pred_entrenamiento, average='weighted')
recall = recall_score(Y_trn, Y_pred_entrenamiento, average='weighted')
f1 = f1_score(Y_trn, Y_pred_entrenamiento, average='weighted')
accuracy = accuracy_score(Y_trn, Y_pred_entrenamiento)
nmi = normalized_mutual_info_score(Y_trn, Y_pred_entrenamiento)
kappa = cohen_kappa_score(Y_trn, Y_pred_entrenamiento)
print("Precisión: ", round(precision*100,2))
print("Exhaustividad: ", round(recall*100,2))
print("Puntuación F1: ", round(f1*100,2))
print("Exactitud: ", round(accuracy*100,2))
print("Información Mutua Normalizada (NMI):", round(nmi*100,2))
print("Índice Kappa de Cohen:", round(kappa*100,2))

# Crear un DataFrame para cada métrica
df_precision = pd.DataFrame({'MÉTRICA': ['Precisión'], 'VALOR': [round(precision*100, 2)]})
df_recall = pd.DataFrame({'MÉTRICA': ['Exhaustividad'], 'VALOR': [round(recall*100, 2)]})
df_f1 = pd.DataFrame({'MÉTRICA': ['Puntuación F1'], 'VALOR': [round(f1*100, 2)]})
df_accuracy = pd.DataFrame({'MÉTRICA': ['Exactitud'], 'VALOR': [round(accuracy*100, 2)]})
df_nmi=pd.DataFrame({'MÉTRICA': ['Información Mutua Normalizada (NMI)'], 'VALOR': [round(nmi*100, 2)]})
df_kappa=pd.DataFrame({'MÉTRICA': ['Índice Kappa de Cohen'], 'VALOR': [round(kappa*100, 2)]})

# Concatenar los DataFrames
resultados_df_svc_entrenamiento = pd.concat([resultados_df_SVC, df_precision, df_recall, df_f1, df_accuracy,df_nmi,df_kappa], ignore_index=True)
resultados_df_svc_entrenamiento["MODELO"]='SVC'
resultados_df_svc_entrenamiento["TIPO_DE_DATOS"]='Entrenamiento'
# Imprimir el DataFrame con los resultados
resultados_df_svc_entrenamiento

Precisión:  56.36
Exhaustividad:  42.72
Puntuación F1:  25.95
Exactitud:  42.72
Información Mutua Normalizada (NMI): 2.07
Índice Kappa de Cohen: 0.61


,MÉTRICA,VALOR,MODELO,TIPO_DE_DATOS
0,Precisión,56.36,SVC,Entrenamiento
1,Exhaustividad,42.72,SVC,Entrenamiento
2,Puntuación F1,25.95,SVC,Entrenamiento
3,Exactitud,42.72,SVC,Entrenamiento
4,Información Mutua Normalizada (NMI),2.07,SVC,Entrenamiento
5,Índice Kappa de Cohen,0.61,SVC,Entrenamiento


In [22]:
# Predecir las etiquetas para los datos de prueba
resultados_df_SVC = pd.DataFrame(columns=['MÉTRICA', 'VALOR'])

Y_pred_prueba = modelo_svc.predict(X_tst)

precision = precision_score(Y_tst, Y_pred_prueba, average='weighted')
recall = recall_score(Y_tst, Y_pred_prueba, average='weighted')
f1 = f1_score(Y_tst, Y_pred_prueba, average='weighted')
accuracy = accuracy_score(Y_tst, Y_pred_prueba)
nmi = normalized_mutual_info_score(Y_tst, Y_pred_prueba)
kappa = cohen_kappa_score(Y_tst, Y_pred_prueba)
print("Precisión: ", round(precision*100,2))
print("Exhaustividad: ", round(recall*100,2))
print("Puntuación F1: ", round(f1*100,2))
print("Exactitud: ", round(accuracy*100,2))
print("Información Mutua Normalizada (NMI):", round(nmi*100,2))
print("Índice Kappa de Cohen:", round(kappa*100,2))

# Crear un DataFrame para cada métrica
df_precision = pd.DataFrame({'MÉTRICA': ['Precisión'], 'VALOR': [round(precision*100, 2)]})
df_recall = pd.DataFrame({'MÉTRICA': ['Exhaustividad'], 'VALOR': [round(recall*100, 2)]})
df_f1 = pd.DataFrame({'MÉTRICA': ['Puntuación F1'], 'VALOR': [round(f1*100, 2)]})
df_accuracy = pd.DataFrame({'MÉTRICA': ['Exactitud'], 'VALOR': [round(accuracy*100, 2)]})
df_nmi=pd.DataFrame({'MÉTRICA': ['Información Mutua Normalizada (NMI)'], 'VALOR': [round(nmi*100, 2)]})
df_kappa=pd.DataFrame({'MÉTRICA': ['Índice Kappa de Cohen'], 'VALOR': [round(kappa*100, 2)]})

# Concatenar los DataFrames
resultados_df_svc_prueba = pd.concat([resultados_df_SVC, df_precision, df_recall, df_f1, df_accuracy,df_nmi,df_kappa], ignore_index=True)
resultados_df_svc_prueba["MODELO"]='SVC'
resultados_df_svc_prueba["TIPO_DE_DATOS"]='Prueba'
# Imprimir el DataFrame con los resultados
resultados_df_svc_prueba

Precisión:  31.6
Exhaustividad:  42.55
Puntuación F1:  25.74
Exactitud:  42.55
Información Mutua Normalizada (NMI): 0.61
Índice Kappa de Cohen: -0.22


,MÉTRICA,VALOR,MODELO,TIPO_DE_DATOS
0,Precisión,31.60,SVC,Prueba
1,Exhaustividad,42.55,SVC,Prueba
2,Puntuación F1,25.74,SVC,Prueba
3,Exactitud,42.55,SVC,Prueba
4,Información Mutua Normalizada (NMI),0.61,SVC,Prueba
5,Índice Kappa de Cohen,-0.22,SVC,Prueba


In [23]:
resultados_df_svc = pd.concat([resultados_df_svc_prueba,resultados_df_svc_entrenamiento], ignore_index=True)
resultados_df_svc

,MÉTRICA,VALOR,MODELO,TIPO_DE_DATOS
0,Precisión,31.60,SVC,Prueba
1,Exhaustividad,42.55,SVC,Prueba
2,Puntuación F1,25.74,SVC,Prueba
3,Exactitud,42.55,SVC,Prueba
4,Información Mutua Normalizada (NMI),0.61,SVC,Prueba
5,Índice Kappa de Cohen,-0.22,SVC,Prueba
6,Precisión,56.36,SVC,Entrenamiento
7,Exhaustividad,42.72,SVC,Entrenamiento
8,Puntuación F1,25.95,SVC,Entrenamiento
9,Exactitud,42.72,SVC,Entrenamiento


### DECISION TREE

In [24]:
def entrenar_modelo_tree_con_transformacion(X_trn, Y_trn):
    X_trn_transformado = X_trn
    parameters = {          
            'max_depth':[i for i in range(3,6,1)],
            'min_samples_split' :  [i for i in range(1,3,1)],  
            'min_samples_leaf' : [i for i in range(1,3,1)], 
            'max_features' : [i for i in range(5,7,1)], 
            'splitter': ["best", "random"],
            'random_state': [i for i in range(1,4,1)],
            'criterion': ['entropy']}
    modelo = DecisionTreeClassifier()
    semilla=7
    num_folds=10
    kfold = KFold(n_splits=num_folds, random_state=semilla, shuffle=True)
    metrica = 'accuracy'
    grid = GridSearchCV(estimator=modelo, param_grid=parameters, scoring=metrica, cv=kfold, n_jobs=-1)
    grid_resultado = grid.fit(X_trn, Y_trn)
    print("Resultados de GridSearchCV para el modelo")
    print("Mejor valor EXACTITUD usando k-fold:", grid_resultado.best_score_*100)
    print("Mejor valor PARAMETRO usando k-fold:", grid_resultado.best_params_)
    mejor_modelo = DecisionTreeClassifier(**grid_resultado.best_params_)
    mejor_modelo.fit(X_trn_transformado, Y_trn)
    return mejor_modelo

X_trn = X_trn
Y_trn = Y_trn 

modelo_tree = entrenar_modelo_tree_con_transformacion(X_trn, Y_trn)

Resultados de GridSearchCV para el modelo
Mejor valor EXACTITUD usando k-fold: 81.15775558166864
Mejor valor PARAMETRO usando k-fold: {'criterion': 'entropy', 'max_depth': 5, 'max_features': 5, 'min_samples_leaf': 1, 'min_samples_split': 2, 'random_state': 3, 'splitter': 'best'}


In [25]:
# Predecir las etiquetas para los datos de prueba
resultados_df_tree = pd.DataFrame(columns=['MÉTRICA', 'VALOR'])

Y_pred_entrenamiento= modelo_tree.predict(X_trn)

precision = precision_score(Y_trn, Y_pred_entrenamiento, average='weighted')
recall = recall_score(Y_trn, Y_pred_entrenamiento, average='weighted')
f1 = f1_score(Y_trn, Y_pred_entrenamiento, average='weighted')
accuracy = accuracy_score(Y_trn, Y_pred_entrenamiento)
nmi = normalized_mutual_info_score(Y_trn, Y_pred_entrenamiento)
kappa = cohen_kappa_score(Y_trn, Y_pred_entrenamiento)
print("Precisión: ", round(precision*100,2))
print("Exhaustividad: ", round(recall*100,2))
print("Puntuación F1: ", round(f1*100,2))
print("Exactitud: ", round(accuracy*100,2))
print("Información Mutua Normalizada (NMI):", round(nmi*100,2))
print("Índice Kappa de Cohen:", round(kappa*100,2))

# Crear un DataFrame para cada métrica
df_precision = pd.DataFrame({'MÉTRICA': ['Precisión'], 'VALOR': [round(precision*100, 2)]})
df_recall = pd.DataFrame({'MÉTRICA': ['Exhaustividad'], 'VALOR': [round(recall*100, 2)]})
df_f1 = pd.DataFrame({'MÉTRICA': ['Puntuación F1'], 'VALOR': [round(f1*100, 2)]})
df_accuracy = pd.DataFrame({'MÉTRICA': ['Exactitud'], 'VALOR': [round(accuracy*100, 2)]})
df_nmi=pd.DataFrame({'MÉTRICA': ['Información Mutua Normalizada (NMI)'], 'VALOR': [round(nmi*100, 2)]})
df_kappa=pd.DataFrame({'MÉTRICA': ['Índice Kappa de Cohen'], 'VALOR': [round(kappa*100, 2)]})

# Concatenar los DataFrames
resultados_df_tree_entrenamiento = pd.concat([resultados_df_tree, df_precision, df_recall, df_f1, df_accuracy,df_nmi,df_kappa], ignore_index=True)
resultados_df_tree_entrenamiento["MODELO"]='Decision Tree'
resultados_df_tree_entrenamiento["TIPO_DE_DATOS"]='Entrenamiento'
# Imprimir el DataFrame con los resultados
resultados_df_tree_entrenamiento

Precisión:  80.35
Exhaustividad:  81.81
Puntuación F1:  80.64
Exactitud:  81.81
Información Mutua Normalizada (NMI): 36.99
Índice Kappa de Cohen: 64.85


,MÉTRICA,VALOR,MODELO,TIPO_DE_DATOS
0,Precisión,80.35,Decision Tree,Entrenamiento
1,Exhaustividad,81.81,Decision Tree,Entrenamiento
2,Puntuación F1,80.64,Decision Tree,Entrenamiento
3,Exactitud,81.81,Decision Tree,Entrenamiento
4,Información Mutua Normalizada (NMI),36.99,Decision Tree,Entrenamiento
5,Índice Kappa de Cohen,64.85,Decision Tree,Entrenamiento


In [26]:
# Predecir las etiquetas para los datos de prueba
resultados_df_tree = pd.DataFrame(columns=['MÉTRICA', 'VALOR'])

Y_pred_prueba = modelo_tree.predict(X_tst)

precision = precision_score(Y_tst, Y_pred_prueba, average='weighted')
recall = recall_score(Y_tst, Y_pred_prueba, average='weighted')
f1 = f1_score(Y_tst, Y_pred_prueba, average='weighted')
accuracy = accuracy_score(Y_tst, Y_pred_prueba)
nmi = normalized_mutual_info_score(Y_tst, Y_pred_prueba)
kappa = cohen_kappa_score(Y_tst, Y_pred_prueba)
print("Precisión: ", round(precision*100,2))
print("Exhaustividad: ", round(recall*100,2))
print("Puntuación F1: ", round(f1*100,2))
print("Exactitud: ", round(accuracy*100,2))
print("Información Mutua Normalizada (NMI):", round(nmi*100,2))
print("Índice Kappa de Cohen:", round(kappa*100,2))

# Crear un DataFrame para cada métrica
df_precision = pd.DataFrame({'MÉTRICA': ['Precisión'], 'VALOR': [round(precision*100, 2)]})
df_recall = pd.DataFrame({'MÉTRICA': ['Exhaustividad'], 'VALOR': [round(recall*100, 2)]})
df_f1 = pd.DataFrame({'MÉTRICA': ['Puntuación F1'], 'VALOR': [round(f1*100, 2)]})
df_accuracy = pd.DataFrame({'MÉTRICA': ['Exactitud'], 'VALOR': [round(accuracy*100, 2)]})
df_nmi=pd.DataFrame({'MÉTRICA': ['Información Mutua Normalizada (NMI)'], 'VALOR': [round(nmi*100, 2)]})
df_kappa=pd.DataFrame({'MÉTRICA': ['Índice Kappa de Cohen'], 'VALOR': [round(kappa*100, 2)]})

# Concatenar los DataFrames
resultados_df_tree_prueba = pd.concat([resultados_df_tree, df_precision, df_recall, df_f1, df_accuracy,df_nmi,df_kappa], ignore_index=True)
resultados_df_tree_prueba["MODELO"]='Decision Tree'
resultados_df_tree_prueba["TIPO_DE_DATOS"]='Prueba'
# Imprimir el DataFrame con los resultados
resultados_df_tree_prueba

Precisión:  78.24
Exhaustividad:  80.81
Puntuación F1:  79.5
Exactitud:  80.81
Información Mutua Normalizada (NMI): 33.99
Índice Kappa de Cohen: 62.75


,MÉTRICA,VALOR,MODELO,TIPO_DE_DATOS
0,Precisión,78.24,Decision Tree,Prueba
1,Exhaustividad,80.81,Decision Tree,Prueba
2,Puntuación F1,79.50,Decision Tree,Prueba
3,Exactitud,80.81,Decision Tree,Prueba
4,Información Mutua Normalizada (NMI),33.99,Decision Tree,Prueba
5,Índice Kappa de Cohen,62.75,Decision Tree,Prueba


In [27]:
resultados_df_tree = pd.concat([resultados_df_tree_prueba,resultados_df_tree_entrenamiento], ignore_index=True)
resultados_df_tree

,MÉTRICA,VALOR,MODELO,TIPO_DE_DATOS
0,Precisión,78.24,Decision Tree,Prueba
1,Exhaustividad,80.81,Decision Tree,Prueba
2,Puntuación F1,79.50,Decision Tree,Prueba
3,Exactitud,80.81,Decision Tree,Prueba
4,Información Mutua Normalizada (NMI),33.99,Decision Tree,Prueba
5,Índice Kappa de Cohen,62.75,Decision Tree,Prueba
6,Precisión,80.35,Decision Tree,Entrenamiento
7,Exhaustividad,81.81,Decision Tree,Entrenamiento
8,Puntuación F1,80.64,Decision Tree,Entrenamiento
9,Exactitud,81.81,Decision Tree,Entrenamiento


### NAIVE BAYES

In [28]:
def entrenar_modelo_gaussian_con_transformacion(X_trn, Y_trn):
    X_trn_transformado = X_trn
    parameters = {}
    modelo = GaussianNB()
    semilla=7
    num_folds=10
    kfold = KFold(n_splits=num_folds, random_state=semilla, shuffle=True)
    metrica = 'accuracy'
    grid = GridSearchCV(estimator=modelo, param_grid=parameters, scoring=metrica, cv=kfold, n_jobs=-1)
    grid_resultado = grid.fit(X_trn, Y_trn)
    print("Resultados de GridSearchCV para el modelo")
    print("Mejor valor EXACTITUD usando k-fold:", grid_resultado.best_score_*100)
    print("Mejor valor PARAMETRO usando k-fold:", grid_resultado.best_params_)
    mejor_modelo = GaussianNB(**grid_resultado.best_params_)
    mejor_modelo.fit(X_trn_transformado, Y_trn)
    return mejor_modelo

X_trn = X_trn
Y_trn = Y_trn 

modelo_gaussian = entrenar_modelo_gaussian_con_transformacion(X_trn, Y_trn)

Resultados de GridSearchCV para el modelo
Mejor valor EXACTITUD usando k-fold: 73.19917743830787
Mejor valor PARAMETRO usando k-fold: {}


In [29]:
# Predecir las etiquetas para los datos de prueba
resultados_df_gaussian = pd.DataFrame(columns=['MÉTRICA', 'VALOR'])

Y_pred_entrenamiento= modelo_gaussian.predict(X_trn)

precision = precision_score(Y_trn, Y_pred_entrenamiento, average='weighted')
recall = recall_score(Y_trn, Y_pred_entrenamiento, average='weighted')
f1 = f1_score(Y_trn, Y_pred_entrenamiento, average='weighted')
accuracy = accuracy_score(Y_trn, Y_pred_entrenamiento)
nmi = normalized_mutual_info_score(Y_trn, Y_pred_entrenamiento)
kappa = cohen_kappa_score(Y_trn, Y_pred_entrenamiento)
print("Precisión: ", round(precision*100,2))
print("Exhaustividad: ", round(recall*100,2))
print("Puntuación F1: ", round(f1*100,2))
print("Exactitud: ", round(accuracy*100,2))
print("Información Mutua Normalizada (NMI):", round(nmi*100,2))
print("Índice Kappa de Cohen:", round(kappa*100,2))

# Crear un DataFrame para cada métrica
df_precision = pd.DataFrame({'MÉTRICA': ['Precisión'], 'VALOR': [round(precision*100, 2)]})
df_recall = pd.DataFrame({'MÉTRICA': ['Exhaustividad'], 'VALOR': [round(recall*100, 2)]})
df_f1 = pd.DataFrame({'MÉTRICA': ['Puntuación F1'], 'VALOR': [round(f1*100, 2)]})
df_accuracy = pd.DataFrame({'MÉTRICA': ['Exactitud'], 'VALOR': [round(accuracy*100, 2)]})
df_nmi=pd.DataFrame({'MÉTRICA': ['Información Mutua Normalizada (NMI)'], 'VALOR': [round(nmi*100, 2)]})
df_kappa=pd.DataFrame({'MÉTRICA': ['Índice Kappa de Cohen'], 'VALOR': [round(kappa*100, 2)]})

# Concatenar los DataFrames
resultados_df_gaussian_entrenamiento = pd.concat([resultados_df_gaussian, df_precision, df_recall, df_f1, df_accuracy,df_nmi,df_kappa], ignore_index=True)
resultados_df_gaussian_entrenamiento["MODELO"]='Naive Bayes'
resultados_df_gaussian_entrenamiento["TIPO_DE_DATOS"]='Entrenamiento'
# Imprimir el DataFrame con los resultados
resultados_df_gaussian_entrenamiento

Precisión:  79.28
Exhaustividad:  73.2
Puntuación F1:  75.51
Exactitud:  73.2
Información Mutua Normalizada (NMI): 32.62
Índice Kappa de Cohen: 53.62


,MÉTRICA,VALOR,MODELO,TIPO_DE_DATOS
0,Precisión,79.28,Naive Bayes,Entrenamiento
1,Exhaustividad,73.20,Naive Bayes,Entrenamiento
2,Puntuación F1,75.51,Naive Bayes,Entrenamiento
3,Exactitud,73.20,Naive Bayes,Entrenamiento
4,Información Mutua Normalizada (NMI),32.62,Naive Bayes,Entrenamiento
5,Índice Kappa de Cohen,53.62,Naive Bayes,Entrenamiento


In [30]:
# Predecir las etiquetas para los datos de prueba
resultados_df_gaussian = pd.DataFrame(columns=['MÉTRICA', 'VALOR'])

Y_pred_prueba = modelo_gaussian.predict(X_tst)

precision = precision_score(Y_tst, Y_pred_prueba, average='weighted')
recall = recall_score(Y_tst, Y_pred_prueba, average='weighted')
f1 = f1_score(Y_tst, Y_pred_prueba, average='weighted')
accuracy = accuracy_score(Y_tst, Y_pred_prueba)
nmi = normalized_mutual_info_score(Y_tst, Y_pred_prueba)
kappa = cohen_kappa_score(Y_tst, Y_pred_prueba)
print("Precisión: ", round(precision*100,2))
print("Exhaustividad: ", round(recall*100,2))
print("Puntuación F1: ", round(f1*100,2))
print("Exactitud: ", round(accuracy*100,2))
print("Información Mutua Normalizada (NMI):", round(nmi*100,2))
print("Índice Kappa de Cohen:", round(kappa*100,2))

# Crear un DataFrame para cada métrica
df_precision = pd.DataFrame({'MÉTRICA': ['Precisión'], 'VALOR': [round(precision*100, 2)]})
df_recall = pd.DataFrame({'MÉTRICA': ['Exhaustividad'], 'VALOR': [round(recall*100, 2)]})
df_f1 = pd.DataFrame({'MÉTRICA': ['Puntuación F1'], 'VALOR': [round(f1*100, 2)]})
df_accuracy = pd.DataFrame({'MÉTRICA': ['Exactitud'], 'VALOR': [round(accuracy*100, 2)]})
df_nmi=pd.DataFrame({'MÉTRICA': ['Información Mutua Normalizada (NMI)'], 'VALOR': [round(nmi*100, 2)]})
df_kappa=pd.DataFrame({'MÉTRICA': ['Índice Kappa de Cohen'], 'VALOR': [round(kappa*100, 2)]})

# Concatenar los DataFrames
resultados_df_gaussian_prueba = pd.concat([resultados_df_gaussian, df_precision, df_recall, df_f1, df_accuracy,df_nmi,df_kappa], ignore_index=True)
resultados_df_gaussian_prueba["MODELO"]='Naive Bayes'
resultados_df_gaussian_prueba["TIPO_DE_DATOS"]='Prueba'
# Imprimir el DataFrame con los resultados
resultados_df_gaussian_prueba

Precisión:  79.91
Exhaustividad:  73.74
Puntuación F1:  76.22
Exactitud:  73.74
Información Mutua Normalizada (NMI): 31.25
Índice Kappa de Cohen: 54.1


,MÉTRICA,VALOR,MODELO,TIPO_DE_DATOS
0,Precisión,79.91,Naive Bayes,Prueba
1,Exhaustividad,73.74,Naive Bayes,Prueba
2,Puntuación F1,76.22,Naive Bayes,Prueba
3,Exactitud,73.74,Naive Bayes,Prueba
4,Información Mutua Normalizada (NMI),31.25,Naive Bayes,Prueba
5,Índice Kappa de Cohen,54.10,Naive Bayes,Prueba


In [31]:
resultados_df_gaussian = pd.concat([resultados_df_gaussian_prueba,resultados_df_gaussian_entrenamiento], ignore_index=True)
resultados_df_gaussian

,MÉTRICA,VALOR,MODELO,TIPO_DE_DATOS
0,Precisión,79.91,Naive Bayes,Prueba
1,Exhaustividad,73.74,Naive Bayes,Prueba
2,Puntuación F1,76.22,Naive Bayes,Prueba
3,Exactitud,73.74,Naive Bayes,Prueba
4,Información Mutua Normalizada (NMI),31.25,Naive Bayes,Prueba
5,Índice Kappa de Cohen,54.10,Naive Bayes,Prueba
6,Precisión,79.28,Naive Bayes,Entrenamiento
7,Exhaustividad,73.20,Naive Bayes,Entrenamiento
8,Puntuación F1,75.51,Naive Bayes,Entrenamiento
9,Exactitud,73.20,Naive Bayes,Entrenamiento


### LDA

In [32]:
def entrenar_modelo_LDA_con_transformacion(X_trn, Y_trn):
    X_trn_transformado = X_trn
    parameters = { 'solver':  ['svd','lsqr','eigen'],
            'n_components':[1,2,3,4,5,None],
            'shrinkage': ['auto',None, 0, 0.001, 0.01, 0.1, 0.5,1]}
    modelo = LinearDiscriminantAnalysis()
    semilla=7
    num_folds=10
    kfold = KFold(n_splits=num_folds, random_state=semilla, shuffle=True)
    metrica = 'accuracy'
    grid = GridSearchCV(estimator=modelo, param_grid=parameters, scoring=metrica, cv=kfold, n_jobs=-1)
    grid_resultado = grid.fit(X_trn, Y_trn)
    print("Resultados de GridSearchCV para el modelo")
    print("Mejor valor EXACTITUD usando k-fold:", grid_resultado.best_score_*100)
    print("Mejor valor PARAMETRO usando k-fold:", grid_resultado.best_params_)
    mejor_modelo = LinearDiscriminantAnalysis(**grid_resultado.best_params_)
    mejor_modelo.fit(X_trn_transformado, Y_trn)
    return mejor_modelo

X_trn = X_trn
Y_trn = Y_trn 

modelo_LDA = entrenar_modelo_LDA_con_transformacion(X_trn, Y_trn)


Resultados de GridSearchCV para el modelo
Mejor valor EXACTITUD usando k-fold: 80.07667450058754
Mejor valor PARAMETRO usando k-fold: {'n_components': 1, 'shrinkage': 0.1, 'solver': 'lsqr'}


In [33]:
# Predecir las etiquetas para los datos de prueba
resultados_df_LDA = pd.DataFrame(columns=['MÉTRICA', 'VALOR'])

Y_pred_entrenamiento= modelo_LDA.predict(X_trn)

precision = precision_score(Y_trn, Y_pred_entrenamiento, average='weighted')
recall = recall_score(Y_trn, Y_pred_entrenamiento, average='weighted')
f1 = f1_score(Y_trn, Y_pred_entrenamiento, average='weighted')
accuracy = accuracy_score(Y_trn, Y_pred_entrenamiento)
nmi = normalized_mutual_info_score(Y_trn, Y_pred_entrenamiento)
kappa = cohen_kappa_score(Y_trn, Y_pred_entrenamiento)
print("Precisión: ", round(precision*100,2))
print("Exhaustividad: ", round(recall*100,2))
print("Puntuación F1: ", round(f1*100,2))
print("Exactitud: ", round(accuracy*100,2))
print("Información Mutua Normalizada (NMI):", round(nmi*100,2))
print("Índice Kappa de Cohen:", round(kappa*100,2))

# Crear un DataFrame para cada métrica
df_precision = pd.DataFrame({'MÉTRICA': ['Precisión'], 'VALOR': [round(precision*100, 2)]})
df_recall = pd.DataFrame({'MÉTRICA': ['Exhaustividad'], 'VALOR': [round(recall*100, 2)]})
df_f1 = pd.DataFrame({'MÉTRICA': ['Puntuación F1'], 'VALOR': [round(f1*100, 2)]})
df_accuracy = pd.DataFrame({'MÉTRICA': ['Exactitud'], 'VALOR': [round(accuracy*100, 2)]})
df_nmi=pd.DataFrame({'MÉTRICA': ['Información Mutua Normalizada (NMI)'], 'VALOR': [round(nmi*100, 2)]})
df_kappa=pd.DataFrame({'MÉTRICA': ['Índice Kappa de Cohen'], 'VALOR': [round(kappa*100, 2)]})

# Concatenar los DataFrames
resultados_df_LDA_entrenamiento = pd.concat([resultados_df_LDA, df_precision, df_recall, df_f1, df_accuracy,df_nmi,df_kappa], ignore_index=True)
resultados_df_LDA_entrenamiento["MODELO"]='LDA'
resultados_df_LDA_entrenamiento["TIPO_DE_DATOS"]='Entrenamiento'
# Imprimir el DataFrame con los resultados
resultados_df_LDA_entrenamiento

Precisión:  79.45
Exhaustividad:  80.67
Puntuación F1:  79.31
Exactitud:  80.67
Información Mutua Normalizada (NMI): 34.4
Índice Kappa de Cohen: 62.31


,MÉTRICA,VALOR,MODELO,TIPO_DE_DATOS
0,Precisión,79.45,LDA,Entrenamiento
1,Exhaustividad,80.67,LDA,Entrenamiento
2,Puntuación F1,79.31,LDA,Entrenamiento
3,Exactitud,80.67,LDA,Entrenamiento
4,Información Mutua Normalizada (NMI),34.40,LDA,Entrenamiento
5,Índice Kappa de Cohen,62.31,LDA,Entrenamiento


In [34]:
# Predecir las etiquetas para los datos de prueba
resultados_df_LDA = pd.DataFrame(columns=['MÉTRICA', 'VALOR'])

Y_pred_prueba = modelo_LDA.predict(X_tst)

precision = precision_score(Y_tst, Y_pred_prueba, average='weighted')
recall = recall_score(Y_tst, Y_pred_prueba, average='weighted')
f1 = f1_score(Y_tst, Y_pred_prueba, average='weighted')
accuracy = accuracy_score(Y_tst, Y_pred_prueba)
nmi = normalized_mutual_info_score(Y_tst, Y_pred_prueba)
kappa = cohen_kappa_score(Y_tst, Y_pred_prueba)
print("Precisión: ", round(precision*100,2))
print("Exhaustividad: ", round(recall*100,2))
print("Puntuación F1: ", round(f1*100,2))
print("Exactitud: ", round(accuracy*100,2))
print("Información Mutua Normalizada (NMI):", round(nmi*100,2))
print("Índice Kappa de Cohen:", round(kappa*100,2))

# Crear un DataFrame para cada métrica
df_precision = pd.DataFrame({'MÉTRICA': ['Precisión'], 'VALOR': [round(precision*100, 2)]})
df_recall = pd.DataFrame({'MÉTRICA': ['Exhaustividad'], 'VALOR': [round(recall*100, 2)]})
df_f1 = pd.DataFrame({'MÉTRICA': ['Puntuación F1'], 'VALOR': [round(f1*100, 2)]})
df_accuracy = pd.DataFrame({'MÉTRICA': ['Exactitud'], 'VALOR': [round(accuracy*100, 2)]})
df_nmi=pd.DataFrame({'MÉTRICA': ['Información Mutua Normalizada (NMI)'], 'VALOR': [round(nmi*100, 2)]})
df_kappa=pd.DataFrame({'MÉTRICA': ['Índice Kappa de Cohen'], 'VALOR': [round(kappa*100, 2)]})

# Concatenar los DataFrames
resultados_df_LDA_prueba = pd.concat([resultados_df_LDA, df_precision, df_recall, df_f1, df_accuracy,df_nmi,df_kappa], ignore_index=True)
resultados_df_LDA_prueba["MODELO"]='LDA'
resultados_df_LDA_prueba["TIPO_DE_DATOS"]='Prueba'
# Imprimir el DataFrame con los resultados
resultados_df_LDA_prueba

Precisión:  80.85
Exhaustividad:  79.8
Puntuación F1:  78.35
Exactitud:  79.8
Información Mutua Normalizada (NMI): 31.63
Índice Kappa de Cohen: 60.19


,MÉTRICA,VALOR,MODELO,TIPO_DE_DATOS
0,Precisión,80.85,LDA,Prueba
1,Exhaustividad,79.80,LDA,Prueba
2,Puntuación F1,78.35,LDA,Prueba
3,Exactitud,79.80,LDA,Prueba
4,Información Mutua Normalizada (NMI),31.63,LDA,Prueba
5,Índice Kappa de Cohen,60.19,LDA,Prueba


In [35]:
resultados_df_LDA = pd.concat([resultados_df_LDA_prueba,resultados_df_LDA_entrenamiento], ignore_index=True)
resultados_df_LDA

,MÉTRICA,VALOR,MODELO,TIPO_DE_DATOS
0,Precisión,80.85,LDA,Prueba
1,Exhaustividad,79.80,LDA,Prueba
2,Puntuación F1,78.35,LDA,Prueba
3,Exactitud,79.80,LDA,Prueba
4,Información Mutua Normalizada (NMI),31.63,LDA,Prueba
5,Índice Kappa de Cohen,60.19,LDA,Prueba
6,Precisión,79.45,LDA,Entrenamiento
7,Exhaustividad,80.67,LDA,Entrenamiento
8,Puntuación F1,79.31,LDA,Entrenamiento
9,Exactitud,80.67,LDA,Entrenamiento


### BAGGINGCLASSIFIER

In [36]:
from sklearn.ensemble import BaggingClassifier
estimator = BaggingClassifier()
estimator.get_params()

{'bootstrap': True,
 'bootstrap_features': False,
 'estimator': None,
 'max_features': 1.0,
 'max_samples': 1.0,
 'n_estimators': 10,
 'n_jobs': None,
 'oob_score': False,
 'random_state': None,
 'verbose': 0,
 'warm_start': False}

In [37]:
def entrenar_modelo_BG_con_transformacion(X_trn, Y_trn):
    # Aplicar la transformación Yeo-Johnson
    X_trn_transformado = X_trn
    parameters = {'n_estimators': [i for i in range(750,760,5)],
            'max_samples' : [i/100.0 for i in range(40,50,5)],
            'max_features': [i/100.0 for i in range(60,65,5)],
            'bootstrap': [True], 
            'bootstrap_features': [True]}
    
    base_estimator= DecisionTreeClassifier(criterion= 'gini', 
                                max_depth=5, max_features= 3,min_samples_leaf= 4, 
                                min_samples_split = 8,random_state= 10, splitter= 'random')

    modelo = BaggingClassifier(estimator=base_estimator, oob_score=True, random_state=1)
    semilla=7
    num_folds=10
    kfold = KFold(n_splits=num_folds, random_state=semilla, shuffle=True)
    metrica = 'accuracy'
    grid = GridSearchCV(estimator=modelo, param_grid=parameters, scoring=metrica, cv=kfold, n_jobs=-1)
    grid_resultado = grid.fit(X_trn, Y_trn)
    print("Resultados de GridSearchCV para el modelo")
    print("Mejor valor EXACTITUD usando k-fold:", grid_resultado.best_score_*100)
    print("Mejor valor PARAMETRO usando k-fold:", grid_resultado.best_params_)
    mejor_modelo = BaggingClassifier(**grid_resultado.best_params_)
    mejor_modelo.fit(X_trn_transformado, Y_trn)
    return mejor_modelo
X_trn = X_trn
Y_trn = Y_trn 
modelo_BG = entrenar_modelo_BG_con_transformacion(X_trn, Y_trn)


Resultados de GridSearchCV para el modelo
Mejor valor EXACTITUD usando k-fold: 82.62103407755582
Mejor valor PARAMETRO usando k-fold: {'bootstrap': True, 'bootstrap_features': True, 'max_features': 0.6, 'max_samples': 0.45, 'n_estimators': 750}


In [38]:
# Predecir las etiquetas para los datos de prueba
resultados_df_BG = pd.DataFrame(columns=['MÉTRICA', 'VALOR'])

Y_pred_entrenamiento= modelo_BG.predict(X_trn)

precision = precision_score(Y_trn, Y_pred_entrenamiento, average='weighted')
recall = recall_score(Y_trn, Y_pred_entrenamiento, average='weighted')
f1 = f1_score(Y_trn, Y_pred_entrenamiento, average='weighted')
accuracy = accuracy_score(Y_trn, Y_pred_entrenamiento)
nmi = normalized_mutual_info_score(Y_trn, Y_pred_entrenamiento)
kappa = cohen_kappa_score(Y_trn, Y_pred_entrenamiento)
print("Precisión: ", round(precision*100,2))
print("Exhaustividad: ", round(recall*100,2))
print("Puntuación F1: ", round(f1*100,2))
print("Exactitud: ", round(accuracy*100,2))
print("Información Mutua Normalizada (NMI):", round(nmi*100,2))
print("Índice Kappa de Cohen:", round(kappa*100,2))

# Crear un DataFrame para cada métrica
df_precision = pd.DataFrame({'MÉTRICA': ['Precisión'], 'VALOR': [round(precision*100, 2)]})
df_recall = pd.DataFrame({'MÉTRICA': ['Exhaustividad'], 'VALOR': [round(recall*100, 2)]})
df_f1 = pd.DataFrame({'MÉTRICA': ['Puntuación F1'], 'VALOR': [round(f1*100, 2)]})
df_accuracy = pd.DataFrame({'MÉTRICA': ['Exactitud'], 'VALOR': [round(accuracy*100, 2)]})
df_nmi=pd.DataFrame({'MÉTRICA': ['Información Mutua Normalizada (NMI)'], 'VALOR': [round(nmi*100, 2)]})
df_kappa=pd.DataFrame({'MÉTRICA': ['Índice Kappa de Cohen'], 'VALOR': [round(kappa*100, 2)]})

# Concatenar los DataFrames
resultados_df_BG_entrenamiento = pd.concat([resultados_df_BG, df_precision, df_recall, df_f1, df_accuracy,df_nmi,df_kappa], ignore_index=True)
resultados_df_BG_entrenamiento["MODELO"]='BaggingClassifier'
resultados_df_BG_entrenamiento["TIPO_DE_DATOS"]='Entrenamiento'
# Imprimir el DataFrame con los resultados
resultados_df_BG_entrenamiento

Precisión:  96.26
Exhaustividad:  96.1
Puntuación F1:  95.76
Exactitud:  96.1
Información Mutua Normalizada (NMI): 81.6
Índice Kappa de Cohen: 92.52


,MÉTRICA,VALOR,MODELO,TIPO_DE_DATOS
0,Precisión,96.26,BaggingClassifier,Entrenamiento
1,Exhaustividad,96.10,BaggingClassifier,Entrenamiento
2,Puntuación F1,95.76,BaggingClassifier,Entrenamiento
3,Exactitud,96.10,BaggingClassifier,Entrenamiento
4,Información Mutua Normalizada (NMI),81.60,BaggingClassifier,Entrenamiento
5,Índice Kappa de Cohen,92.52,BaggingClassifier,Entrenamiento


In [39]:
# Predecir las etiquetas para los datos de prueba
resultados_df_BG = pd.DataFrame(columns=['MÉTRICA', 'VALOR'])

Y_pred_prueba = modelo_BG.predict(X_tst)

precision = precision_score(Y_tst, Y_pred_prueba, average='weighted')
recall = recall_score(Y_tst, Y_pred_prueba, average='weighted')
f1 = f1_score(Y_tst, Y_pred_prueba, average='weighted')
accuracy = accuracy_score(Y_tst, Y_pred_prueba)
nmi = normalized_mutual_info_score(Y_tst, Y_pred_prueba)
kappa = cohen_kappa_score(Y_tst, Y_pred_prueba)
print("Precisión: ", round(precision*100,2))
print("Exhaustividad: ", round(recall*100,2))
print("Puntuación F1: ", round(f1*100,2))
print("Exactitud: ", round(accuracy*100,2))
print("Información Mutua Normalizada (NMI):", round(nmi*100,2))
print("Índice Kappa de Cohen:", round(kappa*100,2))

# Crear un DataFrame para cada métrica
df_precision = pd.DataFrame({'MÉTRICA': ['Precisión'], 'VALOR': [round(precision*100, 2)]})
df_recall = pd.DataFrame({'MÉTRICA': ['Exhaustividad'], 'VALOR': [round(recall*100, 2)]})
df_f1 = pd.DataFrame({'MÉTRICA': ['Puntuación F1'], 'VALOR': [round(f1*100, 2)]})
df_accuracy = pd.DataFrame({'MÉTRICA': ['Exactitud'], 'VALOR': [round(accuracy*100, 2)]})
df_nmi=pd.DataFrame({'MÉTRICA': ['Información Mutua Normalizada (NMI)'], 'VALOR': [round(nmi*100, 2)]})
df_kappa=pd.DataFrame({'MÉTRICA': ['Índice Kappa de Cohen'], 'VALOR': [round(kappa*100, 2)]})

# Concatenar los DataFrames
resultados_df_BG_prueba = pd.concat([resultados_df_BG, df_precision, df_recall, df_f1, df_accuracy,df_nmi,df_kappa], ignore_index=True)
resultados_df_BG_prueba["MODELO"]='BaggingClassifier'
resultados_df_BG_prueba["TIPO_DE_DATOS"]='Prueba'
# Imprimir el DataFrame con los resultados
resultados_df_BG_prueba

Precisión:  80.0
Exhaustividad:  82.95
Puntuación F1:  81.33
Exactitud:  82.95
Información Mutua Normalizada (NMI): 38.75
Índice Kappa de Cohen: 66.5


,MÉTRICA,VALOR,MODELO,TIPO_DE_DATOS
0,Precisión,80.00,BaggingClassifier,Prueba
1,Exhaustividad,82.95,BaggingClassifier,Prueba
2,Puntuación F1,81.33,BaggingClassifier,Prueba
3,Exactitud,82.95,BaggingClassifier,Prueba
4,Información Mutua Normalizada (NMI),38.75,BaggingClassifier,Prueba
5,Índice Kappa de Cohen,66.50,BaggingClassifier,Prueba


In [40]:
resultados_df_BG = pd.concat([resultados_df_BG_prueba,resultados_df_BG_entrenamiento], ignore_index=True)
resultados_df_BG

,MÉTRICA,VALOR,MODELO,TIPO_DE_DATOS
0,Precisión,80.00,BaggingClassifier,Prueba
1,Exhaustividad,82.95,BaggingClassifier,Prueba
2,Puntuación F1,81.33,BaggingClassifier,Prueba
3,Exactitud,82.95,BaggingClassifier,Prueba
4,Información Mutua Normalizada (NMI),38.75,BaggingClassifier,Prueba
5,Índice Kappa de Cohen,66.50,BaggingClassifier,Prueba
6,Precisión,96.26,BaggingClassifier,Entrenamiento
7,Exhaustividad,96.10,BaggingClassifier,Entrenamiento
8,Puntuación F1,95.76,BaggingClassifier,Entrenamiento
9,Exactitud,96.10,BaggingClassifier,Entrenamiento


### RANDOM FOREST

In [41]:
def entrenar_modelo_random_con_transformacion(X_trn, Y_trn):
    X_trn_transformado = X_trn
    parameters = { 
                'min_samples_split' : [ 2 , 4 , 6 , 8 , 10 , 15 ],  
                'min_samples_leaf' : [ 1 , 3 , 5 , 7 , 9 ] 
                  
              }
    modelo = RandomForestClassifier()
    semilla=7
    num_folds=10
    kfold = KFold(n_splits=num_folds, random_state=semilla, shuffle=True)
    metrica = 'accuracy'
    grid = GridSearchCV(estimator=modelo, param_grid=parameters, scoring=metrica, cv=kfold, n_jobs=-1)
    grid_resultado = grid.fit(X_trn, Y_trn)
    print("Resultados de GridSearchCV para el modelo")
    print("Mejor valor EXACTITUD usando k-fold:", grid_resultado.best_score_*100)
    print("Mejor valor PARAMETRO usando k-fold:", grid_resultado.best_params_)
    mejor_modelo = RandomForestClassifier(**grid_resultado.best_params_)
    mejor_modelo.fit(X_trn_transformado, Y_trn)
    return mejor_modelo

X_trn = X_trn
Y_trn = Y_trn 

modelo_random = entrenar_modelo_random_con_transformacion(X_trn, Y_trn)

Resultados de GridSearchCV para el modelo
Mejor valor EXACTITUD usando k-fold: 84.24618096357227
Mejor valor PARAMETRO usando k-fold: {'min_samples_leaf': 9, 'min_samples_split': 6}


In [42]:
# Predecir las etiquetas para los datos de prueba
resultados_df_random = pd.DataFrame(columns=['MÉTRICA', 'VALOR'])

Y_pred_entrenamiento= modelo_random.predict(X_trn)

precision = precision_score(Y_trn, Y_pred_entrenamiento, average='weighted')
recall = recall_score(Y_trn, Y_pred_entrenamiento, average='weighted')
f1 = f1_score(Y_trn, Y_pred_entrenamiento, average='weighted')
accuracy = accuracy_score(Y_trn, Y_pred_entrenamiento)
nmi = normalized_mutual_info_score(Y_trn, Y_pred_entrenamiento)
kappa = cohen_kappa_score(Y_trn, Y_pred_entrenamiento)
print("Precisión: ", round(precision*100,2))
print("Exhaustividad: ", round(recall*100,2))
print("Puntuación F1: ", round(f1*100,2))
print("Exactitud: ", round(accuracy*100,2))
print("Información Mutua Normalizada (NMI):", round(nmi*100,2))
print("Índice Kappa de Cohen:", round(kappa*100,2))

# Crear un DataFrame para cada métrica
df_precision = pd.DataFrame({'MÉTRICA': ['Precisión'], 'VALOR': [round(precision*100, 2)]})
df_recall = pd.DataFrame({'MÉTRICA': ['Exhaustividad'], 'VALOR': [round(recall*100, 2)]})
df_f1 = pd.DataFrame({'MÉTRICA': ['Puntuación F1'], 'VALOR': [round(f1*100, 2)]})
df_accuracy = pd.DataFrame({'MÉTRICA': ['Exactitud'], 'VALOR': [round(accuracy*100, 2)]})
df_nmi=pd.DataFrame({'MÉTRICA': ['Información Mutua Normalizada (NMI)'], 'VALOR': [round(nmi*100, 2)]})
df_kappa=pd.DataFrame({'MÉTRICA': ['Índice Kappa de Cohen'], 'VALOR': [round(kappa*100, 2)]})

# Concatenar los DataFrames
resultados_df_random_entrenamiento = pd.concat([resultados_df_random, df_precision, df_recall, df_f1, df_accuracy,df_nmi,df_kappa], ignore_index=True)
resultados_df_random_entrenamiento["MODELO"]='RandomForestClassifier'
resultados_df_random_entrenamiento["TIPO_DE_DATOS"]='Entrenamiento'
# Imprimir el DataFrame con los resultados
resultados_df_random_entrenamiento

Precisión:  85.12
Exhaustividad:  88.63
Puntuación F1:  86.79
Exactitud:  88.63
Información Mutua Normalizada (NMI): 55.95
Índice Kappa de Cohen: 77.77


,MÉTRICA,VALOR,MODELO,TIPO_DE_DATOS
0,Precisión,85.12,RandomForestClassifier,Entrenamiento
1,Exhaustividad,88.63,RandomForestClassifier,Entrenamiento
2,Puntuación F1,86.79,RandomForestClassifier,Entrenamiento
3,Exactitud,88.63,RandomForestClassifier,Entrenamiento
4,Información Mutua Normalizada (NMI),55.95,RandomForestClassifier,Entrenamiento
5,Índice Kappa de Cohen,77.77,RandomForestClassifier,Entrenamiento


In [43]:
# Predecir las etiquetas para los datos de prueba
resultados_df_random = pd.DataFrame(columns=['MÉTRICA', 'VALOR'])

Y_pred_prueba = modelo_random.predict(X_tst)

precision = precision_score(Y_tst, Y_pred_prueba, average='weighted')
recall = recall_score(Y_tst, Y_pred_prueba, average='weighted')
f1 = f1_score(Y_tst, Y_pred_prueba, average='weighted')
accuracy = accuracy_score(Y_tst, Y_pred_prueba)
nmi = normalized_mutual_info_score(Y_tst, Y_pred_prueba)
kappa = cohen_kappa_score(Y_tst, Y_pred_prueba)
print("Precisión: ", round(precision*100,2))
print("Exhaustividad: ", round(recall*100,2))
print("Puntuación F1: ", round(f1*100,2))
print("Exactitud: ", round(accuracy*100,2))
print("Información Mutua Normalizada (NMI):", round(nmi*100,2))
print("Índice Kappa de Cohen:", round(kappa*100,2))

# Crear un DataFrame para cada métrica
df_precision = pd.DataFrame({'MÉTRICA': ['Precisión'], 'VALOR': [round(precision*100, 2)]})
df_recall = pd.DataFrame({'MÉTRICA': ['Exhaustividad'], 'VALOR': [round(recall*100, 2)]})
df_f1 = pd.DataFrame({'MÉTRICA': ['Puntuación F1'], 'VALOR': [round(f1*100, 2)]})
df_accuracy = pd.DataFrame({'MÉTRICA': ['Exactitud'], 'VALOR': [round(accuracy*100, 2)]})
df_nmi=pd.DataFrame({'MÉTRICA': ['Información Mutua Normalizada (NMI)'], 'VALOR': [round(nmi*100, 2)]})
df_kappa=pd.DataFrame({'MÉTRICA': ['Índice Kappa de Cohen'], 'VALOR': [round(kappa*100, 2)]})

# Concatenar los DataFrames
resultados_df_random_prueba = pd.concat([resultados_df_random, df_precision, df_recall, df_f1, df_accuracy,df_nmi,df_kappa], ignore_index=True)
resultados_df_random_prueba["MODELO"]='RandomForestClassifier'
resultados_df_random_prueba["TIPO_DE_DATOS"]='Prueba'
# Imprimir el DataFrame con los resultados
resultados_df_random_prueba

Precisión:  79.78
Exhaustividad:  82.83
Puntuación F1:  81.22
Exactitud:  82.83
Información Mutua Normalizada (NMI): 38.47
Índice Kappa de Cohen: 66.33


,MÉTRICA,VALOR,MODELO,TIPO_DE_DATOS
0,Precisión,79.78,RandomForestClassifier,Prueba
1,Exhaustividad,82.83,RandomForestClassifier,Prueba
2,Puntuación F1,81.22,RandomForestClassifier,Prueba
3,Exactitud,82.83,RandomForestClassifier,Prueba
4,Información Mutua Normalizada (NMI),38.47,RandomForestClassifier,Prueba
5,Índice Kappa de Cohen,66.33,RandomForestClassifier,Prueba


In [44]:
resultados_df_random = pd.concat([resultados_df_random_prueba,resultados_df_random_entrenamiento], ignore_index=True)
resultados_df_random

,MÉTRICA,VALOR,MODELO,TIPO_DE_DATOS
0,Precisión,79.78,RandomForestClassifier,Prueba
1,Exhaustividad,82.83,RandomForestClassifier,Prueba
2,Puntuación F1,81.22,RandomForestClassifier,Prueba
3,Exactitud,82.83,RandomForestClassifier,Prueba
4,Información Mutua Normalizada (NMI),38.47,RandomForestClassifier,Prueba
5,Índice Kappa de Cohen,66.33,RandomForestClassifier,Prueba
6,Precisión,85.12,RandomForestClassifier,Entrenamiento
7,Exhaustividad,88.63,RandomForestClassifier,Entrenamiento
8,Puntuación F1,86.79,RandomForestClassifier,Entrenamiento
9,Exactitud,88.63,RandomForestClassifier,Entrenamiento


### EXTRA TREES CLASSIFIER

In [45]:
def entrenar_modelo_extra_con_transformacion(X_trn, Y_trn):
    # Aplicar la transformación Yeo-Johnson
    X_trn_transformado = X_trn
    parameters = {'min_samples_split' : [i for i in range(1,3,1)], 
                'min_samples_leaf' : [i for i in range(0,2,1)] }
    
    semilla=7            
    modelo = ExtraTreesClassifier(random_state=semilla, 
                                n_estimators=40, max_features=1,max_depth= 10,
                                min_samples_leaf=1,  min_samples_split = 2,
                                bootstrap=True,criterion='gini') 
    num_folds=10
    kfold = KFold(n_splits=num_folds, random_state=semilla, shuffle=True)
    metrica = 'accuracy'
    grid = GridSearchCV(estimator=modelo, param_grid=parameters, scoring=metrica, cv=kfold, n_jobs=-1)
    grid_resultado = grid.fit(X_trn, Y_trn)
    print("Resultados de GridSearchCV para el modelo")
    print("Mejor valor EXACTITUD usando k-fold:", grid_resultado.best_score_*100)
    print("Mejor valor PARAMETRO usando k-fold:", grid_resultado.best_params_)
    mejor_modelo = ExtraTreesClassifier(**grid_resultado.best_params_)
    mejor_modelo.fit(X_trn_transformado, Y_trn)
    return mejor_modelo

X_trn = X_trn
Y_trn = Y_trn 

modelo_extra = entrenar_modelo_extra_con_transformacion(X_trn, Y_trn)

Resultados de GridSearchCV para el modelo
Mejor valor EXACTITUD usando k-fold: 83.21797884841364
Mejor valor PARAMETRO usando k-fold: {'min_samples_leaf': 1, 'min_samples_split': 2}


In [46]:
# Predecir las etiquetas para los datos de prueba
resultados_df_extra = pd.DataFrame(columns=['MÉTRICA', 'VALOR'])

Y_pred_entrenamiento= modelo_extra.predict(X_trn)

precision = precision_score(Y_trn, Y_pred_entrenamiento, average='weighted')
recall = recall_score(Y_trn, Y_pred_entrenamiento, average='weighted')
f1 = f1_score(Y_trn, Y_pred_entrenamiento, average='weighted')
accuracy = accuracy_score(Y_trn, Y_pred_entrenamiento)
nmi = normalized_mutual_info_score(Y_trn, Y_pred_entrenamiento)
kappa = cohen_kappa_score(Y_trn, Y_pred_entrenamiento)
print("Precisión: ", round(precision*100,2))
print("Exhaustividad: ", round(recall*100,2))
print("Puntuación F1: ", round(f1*100,2))
print("Exactitud: ", round(accuracy*100,2))
print("Información Mutua Normalizada (NMI):", round(nmi*100,2))
print("Índice Kappa de Cohen:", round(kappa*100,2))

# Crear un DataFrame para cada métrica
df_precision = pd.DataFrame({'MÉTRICA': ['Precisión'], 'VALOR': [round(precision*100, 2)]})
df_recall = pd.DataFrame({'MÉTRICA': ['Exhaustividad'], 'VALOR': [round(recall*100, 2)]})
df_f1 = pd.DataFrame({'MÉTRICA': ['Puntuación F1'], 'VALOR': [round(f1*100, 2)]})
df_accuracy = pd.DataFrame({'MÉTRICA': ['Exactitud'], 'VALOR': [round(accuracy*100, 2)]})
df_nmi=pd.DataFrame({'MÉTRICA': ['Información Mutua Normalizada (NMI)'], 'VALOR': [round(nmi*100, 2)]})
df_kappa=pd.DataFrame({'MÉTRICA': ['Índice Kappa de Cohen'], 'VALOR': [round(kappa*100, 2)]})

# Concatenar los DataFrames
resultados_df_extra_entrenamiento = pd.concat([resultados_df_extra, df_precision, df_recall, df_f1, df_accuracy,df_nmi,df_kappa], ignore_index=True)
resultados_df_extra_entrenamiento["MODELO"]='ExtraTreesClassifier'
resultados_df_extra_entrenamiento["TIPO_DE_DATOS"]='Entrenamiento'
# Imprimir el DataFrame con los resultados
resultados_df_extra_entrenamiento


Precisión:  100.0
Exhaustividad:  100.0
Puntuación F1:  100.0
Exactitud:  100.0
Información Mutua Normalizada (NMI): 100.0
Índice Kappa de Cohen: 100.0


,MÉTRICA,VALOR,MODELO,TIPO_DE_DATOS
0,Precisión,100.0,ExtraTreesClassifier,Entrenamiento
1,Exhaustividad,100.0,ExtraTreesClassifier,Entrenamiento
2,Puntuación F1,100.0,ExtraTreesClassifier,Entrenamiento
3,Exactitud,100.0,ExtraTreesClassifier,Entrenamiento
4,Información Mutua Normalizada (NMI),100.0,ExtraTreesClassifier,Entrenamiento
5,Índice Kappa de Cohen,100.0,ExtraTreesClassifier,Entrenamiento


In [47]:
# Predecir las etiquetas para los datos de prueba
resultados_df_extra = pd.DataFrame(columns=['MÉTRICA', 'VALOR'])

Y_pred_prueba = modelo_extra.predict(X_tst)

precision = precision_score(Y_tst, Y_pred_prueba, average='weighted')
recall = recall_score(Y_tst, Y_pred_prueba, average='weighted')
f1 = f1_score(Y_tst, Y_pred_prueba, average='weighted')
accuracy = accuracy_score(Y_tst, Y_pred_prueba)
nmi = normalized_mutual_info_score(Y_tst, Y_pred_prueba)
kappa = cohen_kappa_score(Y_tst, Y_pred_prueba)
print("Precisión: ", round(precision*100,2))
print("Exhaustividad: ", round(recall*100,2))
print("Puntuación F1: ", round(f1*100,2))
print("Exactitud: ", round(accuracy*100,2))
print("Información Mutua Normalizada (NMI):", round(nmi*100,2))
print("Índice Kappa de Cohen:", round(kappa*100,2))

# Crear un DataFrame para cada métrica
df_precision = pd.DataFrame({'MÉTRICA': ['Precisión'], 'VALOR': [round(precision*100, 2)]})
df_recall = pd.DataFrame({'MÉTRICA': ['Exhaustividad'], 'VALOR': [round(recall*100, 2)]})
df_f1 = pd.DataFrame({'MÉTRICA': ['Puntuación F1'], 'VALOR': [round(f1*100, 2)]})
df_accuracy = pd.DataFrame({'MÉTRICA': ['Exactitud'], 'VALOR': [round(accuracy*100, 2)]})
df_nmi=pd.DataFrame({'MÉTRICA': ['Información Mutua Normalizada (NMI)'], 'VALOR': [round(nmi*100, 2)]})
df_kappa=pd.DataFrame({'MÉTRICA': ['Índice Kappa de Cohen'], 'VALOR': [round(kappa*100, 2)]})

# Concatenar los DataFrames
resultados_df_extra_prueba = pd.concat([resultados_df_extra, df_precision, df_recall, df_f1, df_accuracy,df_nmi,df_kappa], ignore_index=True)
resultados_df_extra_prueba["MODELO"]='ExtraTreesClassifier'
resultados_df_extra_prueba["TIPO_DE_DATOS"]='Prueba'
# Imprimir el DataFrame con los resultados
resultados_df_extra_prueba

Precisión:  80.85
Exhaustividad:  83.84
Puntuación F1:  82.27
Exactitud:  83.84
Información Mutua Normalizada (NMI): 40.62
Índice Kappa de Cohen: 68.35


,MÉTRICA,VALOR,MODELO,TIPO_DE_DATOS
0,Precisión,80.85,ExtraTreesClassifier,Prueba
1,Exhaustividad,83.84,ExtraTreesClassifier,Prueba
2,Puntuación F1,82.27,ExtraTreesClassifier,Prueba
3,Exactitud,83.84,ExtraTreesClassifier,Prueba
4,Información Mutua Normalizada (NMI),40.62,ExtraTreesClassifier,Prueba
5,Índice Kappa de Cohen,68.35,ExtraTreesClassifier,Prueba


In [48]:
resultados_df_extra = pd.concat([resultados_df_extra_prueba,resultados_df_extra_entrenamiento], ignore_index=True)
resultados_df_extra

,MÉTRICA,VALOR,MODELO,TIPO_DE_DATOS
0,Precisión,80.85,ExtraTreesClassifier,Prueba
1,Exhaustividad,83.84,ExtraTreesClassifier,Prueba
2,Puntuación F1,82.27,ExtraTreesClassifier,Prueba
3,Exactitud,83.84,ExtraTreesClassifier,Prueba
4,Información Mutua Normalizada (NMI),40.62,ExtraTreesClassifier,Prueba
5,Índice Kappa de Cohen,68.35,ExtraTreesClassifier,Prueba
6,Precisión,100.00,ExtraTreesClassifier,Entrenamiento
7,Exhaustividad,100.00,ExtraTreesClassifier,Entrenamiento
8,Puntuación F1,100.00,ExtraTreesClassifier,Entrenamiento
9,Exactitud,100.00,ExtraTreesClassifier,Entrenamiento


### ADABOOST

In [49]:
from sklearn.ensemble import AdaBoostClassifier            
estimator  = AdaBoostClassifier ()
estimator.get_params()

{'algorithm': 'SAMME.R',
 'estimator': None,
 'learning_rate': 1.0,
 'n_estimators': 50,
 'random_state': None}

In [50]:
def entrenar_modelo_ADA_con_transformacion(X_trn, Y_trn):
    # Aplicar la transformación Yeo-Johnson
    X_trn_transformado = X_trn
    parameters = {'learning_rate' : [i/10000.0 for i in range(5,20,5)]}
    semilla=7            
    modelo = AdaBoostClassifier(estimator = None,  algorithm = 'SAMME.R', 
                                random_state= None, n_estimators =50) 
    num_folds=10
    kfold = KFold(n_splits=num_folds, random_state=semilla, shuffle=True)
    metrica = 'accuracy'
    grid = GridSearchCV(estimator=modelo, param_grid=parameters, scoring=metrica, cv=kfold, n_jobs=-1)
    grid_resultado = grid.fit(X_trn, Y_trn)
    print("Resultados de GridSearchCV para el modelo")
    print("Mejor valor EXACTITUD usando k-fold:", grid_resultado.best_score_*100)
    print("Mejor valor PARAMETRO usando k-fold:", grid_resultado.best_params_)
    mejor_modelo = AdaBoostClassifier(**grid_resultado.best_params_)
    mejor_modelo.fit(X_trn_transformado, Y_trn)
    return mejor_modelo

X_trn = X_trn
Y_trn = Y_trn 

modelo_ADA = entrenar_modelo_ADA_con_transformacion(X_trn, Y_trn)

Resultados de GridSearchCV para el modelo
Mejor valor EXACTITUD usando k-fold: 76.17596944770858
Mejor valor PARAMETRO usando k-fold: {'learning_rate': 0.0005}


In [51]:
# Predecir las etiquetas para los datos de prueba
resultados_df_ADA = pd.DataFrame(columns=['MÉTRICA', 'VALOR'])

Y_pred_entrenamiento= modelo_ADA.predict(X_trn)

precision = precision_score(Y_trn, Y_pred_entrenamiento, average='weighted')
recall = recall_score(Y_trn, Y_pred_entrenamiento, average='weighted')
f1 = f1_score(Y_trn, Y_pred_entrenamiento, average='weighted')
accuracy = accuracy_score(Y_trn, Y_pred_entrenamiento)
nmi = normalized_mutual_info_score(Y_trn, Y_pred_entrenamiento)
kappa = cohen_kappa_score(Y_trn, Y_pred_entrenamiento)
print("Precisión: ", round(precision*100,2))
print("Exhaustividad: ", round(recall*100,2))
print("Puntuación F1: ", round(f1*100,2))
print("Exactitud: ", round(accuracy*100,2))
print("Información Mutua Normalizada (NMI):", round(nmi*100,2))
print("Índice Kappa de Cohen:", round(kappa*100,2))

# Crear un DataFrame para cada métrica
df_precision = pd.DataFrame({'MÉTRICA': ['Precisión'], 'VALOR': [round(precision*100, 2)]})
df_recall = pd.DataFrame({'MÉTRICA': ['Exhaustividad'], 'VALOR': [round(recall*100, 2)]})
df_f1 = pd.DataFrame({'MÉTRICA': ['Puntuación F1'], 'VALOR': [round(f1*100, 2)]})
df_accuracy = pd.DataFrame({'MÉTRICA': ['Exactitud'], 'VALOR': [round(accuracy*100, 2)]})
df_nmi=pd.DataFrame({'MÉTRICA': ['Información Mutua Normalizada (NMI)'], 'VALOR': [round(nmi*100, 2)]})
df_kappa=pd.DataFrame({'MÉTRICA': ['Índice Kappa de Cohen'], 'VALOR': [round(kappa*100, 2)]})

# Concatenar los DataFrames
resultados_df_ADA_entrenamiento = pd.concat([resultados_df_ADA, df_precision, df_recall, df_f1, df_accuracy,df_nmi,df_kappa], ignore_index=True)
resultados_df_ADA_entrenamiento["MODELO"]='AdaBoostClassifier'
resultados_df_ADA_entrenamiento["TIPO_DE_DATOS"]='Entrenamiento'
# Imprimir el DataFrame con los resultados
resultados_df_ADA_entrenamiento

Precisión:  75.68
Exhaustividad:  78.13
Puntuación F1:  76.32
Exactitud:  78.13
Información Mutua Normalizada (NMI): 29.41
Índice Kappa de Cohen: 56.68


,MÉTRICA,VALOR,MODELO,TIPO_DE_DATOS
0,Precisión,75.68,AdaBoostClassifier,Entrenamiento
1,Exhaustividad,78.13,AdaBoostClassifier,Entrenamiento
2,Puntuación F1,76.32,AdaBoostClassifier,Entrenamiento
3,Exactitud,78.13,AdaBoostClassifier,Entrenamiento
4,Información Mutua Normalizada (NMI),29.41,AdaBoostClassifier,Entrenamiento
5,Índice Kappa de Cohen,56.68,AdaBoostClassifier,Entrenamiento


In [52]:
# Predecir las etiquetas para los datos de prueba
resultados_df_ADA = pd.DataFrame(columns=['MÉTRICA', 'VALOR'])

Y_pred_prueba = modelo_ADA.predict(X_tst)

precision = precision_score(Y_tst, Y_pred_prueba, average='weighted')
recall = recall_score(Y_tst, Y_pred_prueba, average='weighted')
f1 = f1_score(Y_tst, Y_pred_prueba, average='weighted')
accuracy = accuracy_score(Y_tst, Y_pred_prueba)
nmi = normalized_mutual_info_score(Y_tst, Y_pred_prueba)
kappa = cohen_kappa_score(Y_tst, Y_pred_prueba)
print("Precisión: ", round(precision*100,2))
print("Exhaustividad: ", round(recall*100,2))
print("Puntuación F1: ", round(f1*100,2))
print("Exactitud: ", round(accuracy*100,2))
print("Información Mutua Normalizada (NMI):", round(nmi*100,2))
print("Índice Kappa de Cohen:", round(kappa*100,2))

# Crear un DataFrame para cada métrica
df_precision = pd.DataFrame({'MÉTRICA': ['Precisión'], 'VALOR': [round(precision*100, 2)]})
df_recall = pd.DataFrame({'MÉTRICA': ['Exhaustividad'], 'VALOR': [round(recall*100, 2)]})
df_f1 = pd.DataFrame({'MÉTRICA': ['Puntuación F1'], 'VALOR': [round(f1*100, 2)]})
df_accuracy = pd.DataFrame({'MÉTRICA': ['Exactitud'], 'VALOR': [round(accuracy*100, 2)]})
df_nmi=pd.DataFrame({'MÉTRICA': ['Información Mutua Normalizada (NMI)'], 'VALOR': [round(nmi*100, 2)]})
df_kappa=pd.DataFrame({'MÉTRICA': ['Índice Kappa de Cohen'], 'VALOR': [round(kappa*100, 2)]})

# Concatenar los DataFrames
resultados_df_ADA_prueba = pd.concat([resultados_df_ADA, df_precision, df_recall, df_f1, df_accuracy,df_nmi,df_kappa], ignore_index=True)
resultados_df_ADA_prueba["MODELO"]='AdaBoostClassifier'
resultados_df_ADA_prueba["TIPO_DE_DATOS"]='Prueba'
# Imprimir el DataFrame con los resultados
resultados_df_ADA_prueba

Precisión:  76.39
Exhaustividad:  78.28
Puntuación F1:  76.53
Exactitud:  78.28
Información Mutua Normalizada (NMI): 29.74
Índice Kappa de Cohen: 56.76


,MÉTRICA,VALOR,MODELO,TIPO_DE_DATOS
0,Precisión,76.39,AdaBoostClassifier,Prueba
1,Exhaustividad,78.28,AdaBoostClassifier,Prueba
2,Puntuación F1,76.53,AdaBoostClassifier,Prueba
3,Exactitud,78.28,AdaBoostClassifier,Prueba
4,Información Mutua Normalizada (NMI),29.74,AdaBoostClassifier,Prueba
5,Índice Kappa de Cohen,56.76,AdaBoostClassifier,Prueba


In [53]:
resultados_df_ADA = pd.concat([resultados_df_ADA_prueba,resultados_df_ADA_entrenamiento], ignore_index=True)
resultados_df_ADA

,MÉTRICA,VALOR,MODELO,TIPO_DE_DATOS
0,Precisión,76.39,AdaBoostClassifier,Prueba
1,Exhaustividad,78.28,AdaBoostClassifier,Prueba
2,Puntuación F1,76.53,AdaBoostClassifier,Prueba
3,Exactitud,78.28,AdaBoostClassifier,Prueba
4,Información Mutua Normalizada (NMI),29.74,AdaBoostClassifier,Prueba
5,Índice Kappa de Cohen,56.76,AdaBoostClassifier,Prueba
6,Precisión,75.68,AdaBoostClassifier,Entrenamiento
7,Exhaustividad,78.13,AdaBoostClassifier,Entrenamiento
8,Puntuación F1,76.32,AdaBoostClassifier,Entrenamiento
9,Exactitud,78.13,AdaBoostClassifier,Entrenamiento


### GRADIENT BOOSTING MACHINE

In [54]:
def entrenar_modelo_GD_con_transformacion(X_trn, Y_trn):
    X_trn_transformado = X_trn
    parameters = { 
                'subsample' : [ 0.5,0.6,0.7 , 0.75 , 0.8 , 0.85 , 0.9 , 0.95 , 1 ]        
              }
    semilla=7
    modelo = GradientBoostingClassifier(random_state=semilla,
                                    n_estimators= 100,learning_rate= 0.1,max_depth= 2,
                                    min_samples_split= 2, min_samples_leaf= 3,max_features= 2)
    semilla=7
    num_folds=10
    kfold = KFold(n_splits=num_folds, random_state=semilla, shuffle=True)
    metrica = 'accuracy'
    grid = GridSearchCV(estimator=modelo, param_grid=parameters, scoring=metrica, cv=kfold, n_jobs=-1)
    grid_resultado = grid.fit(X_trn, Y_trn)
    print("Resultados de GridSearchCV para el modelo")
    print("Mejor valor EXACTITUD usando k-fold:", grid_resultado.best_score_*100)
    print("Mejor valor PARAMETRO usando k-fold:", grid_resultado.best_params_)
    mejor_modelo = GradientBoostingClassifier(**grid_resultado.best_params_)
    mejor_modelo.fit(X_trn_transformado, Y_trn)
    return mejor_modelo

X_trn = X_trn
Y_trn = Y_trn 

modelo_GD = entrenar_modelo_GD_con_transformacion(X_trn, Y_trn)


Resultados de GridSearchCV para el modelo
Mejor valor EXACTITUD usando k-fold: 83.75851938895418
Mejor valor PARAMETRO usando k-fold: {'subsample': 0.7}


In [55]:
# Predecir las etiquetas para los datos de prueba
resultados_df_GD = pd.DataFrame(columns=['MÉTRICA', 'VALOR'])

Y_pred_entrenamiento= modelo_GD.predict(X_trn)

precision = precision_score(Y_trn, Y_pred_entrenamiento, average='weighted')
recall = recall_score(Y_trn, Y_pred_entrenamiento, average='weighted')
f1 = f1_score(Y_trn, Y_pred_entrenamiento, average='weighted')
accuracy = accuracy_score(Y_trn, Y_pred_entrenamiento)
nmi = normalized_mutual_info_score(Y_trn, Y_pred_entrenamiento)
kappa = cohen_kappa_score(Y_trn, Y_pred_entrenamiento)
print("Precisión: ", round(precision*100,2))
print("Exhaustividad: ", round(recall*100,2))
print("Puntuación F1: ", round(f1*100,2))
print("Exactitud: ", round(accuracy*100,2))
print("Información Mutua Normalizada (NMI):", round(nmi*100,2))
print("Índice Kappa de Cohen:", round(kappa*100,2))

# Crear un DataFrame para cada métrica
df_precision = pd.DataFrame({'MÉTRICA': ['Precisión'], 'VALOR': [round(precision*100, 2)]})
df_recall = pd.DataFrame({'MÉTRICA': ['Exhaustividad'], 'VALOR': [round(recall*100, 2)]})
df_f1 = pd.DataFrame({'MÉTRICA': ['Puntuación F1'], 'VALOR': [round(f1*100, 2)]})
df_accuracy = pd.DataFrame({'MÉTRICA': ['Exactitud'], 'VALOR': [round(accuracy*100, 2)]})
df_nmi=pd.DataFrame({'MÉTRICA': ['Información Mutua Normalizada (NMI)'], 'VALOR': [round(nmi*100, 2)]})
df_kappa=pd.DataFrame({'MÉTRICA': ['Índice Kappa de Cohen'], 'VALOR': [round(kappa*100, 2)]})

# Concatenar los DataFrames
resultados_df_GD_entrenamiento = pd.concat([resultados_df_GD, df_precision, df_recall, df_f1, df_accuracy,df_nmi,df_kappa], ignore_index=True)
resultados_df_GD_entrenamiento["MODELO"]='GradientBoostingClassifier'
resultados_df_GD_entrenamiento["TIPO_DE_DATOS"]='Entrenamiento'
# Imprimir el DataFrame con los resultados
resultados_df_GD_entrenamiento

Precisión:  93.84
Exhaustividad:  93.72
Puntuación F1:  93.61
Exactitud:  93.72
Información Mutua Normalizada (NMI): 70.63
Índice Kappa de Cohen: 88.03


,MÉTRICA,VALOR,MODELO,TIPO_DE_DATOS
0,Precisión,93.84,GradientBoostingClassifier,Entrenamiento
1,Exhaustividad,93.72,GradientBoostingClassifier,Entrenamiento
2,Puntuación F1,93.61,GradientBoostingClassifier,Entrenamiento
3,Exactitud,93.72,GradientBoostingClassifier,Entrenamiento
4,Información Mutua Normalizada (NMI),70.63,GradientBoostingClassifier,Entrenamiento
5,Índice Kappa de Cohen,88.03,GradientBoostingClassifier,Entrenamiento


In [56]:
# Predecir las etiquetas para los datos de prueba
resultados_df_GD = pd.DataFrame(columns=['MÉTRICA', 'VALOR'])

Y_pred_prueba = modelo_GD.predict(X_tst)

precision = precision_score(Y_tst, Y_pred_prueba, average='weighted')
recall = recall_score(Y_tst, Y_pred_prueba, average='weighted')
f1 = f1_score(Y_tst, Y_pred_prueba, average='weighted')
accuracy = accuracy_score(Y_tst, Y_pred_prueba)
nmi = normalized_mutual_info_score(Y_tst, Y_pred_prueba)
kappa = cohen_kappa_score(Y_tst, Y_pred_prueba)
print("Precisión: ", round(precision*100,2))
print("Exhaustividad: ", round(recall*100,2))
print("Puntuación F1: ", round(f1*100,2))
print("Exactitud: ", round(accuracy*100,2))
print("Información Mutua Normalizada (NMI):", round(nmi*100,2))
print("Índice Kappa de Cohen:", round(kappa*100,2))

# Crear un DataFrame para cada métrica
df_precision = pd.DataFrame({'MÉTRICA': ['Precisión'], 'VALOR': [round(precision*100, 2)]})
df_recall = pd.DataFrame({'MÉTRICA': ['Exhaustividad'], 'VALOR': [round(recall*100, 2)]})
df_f1 = pd.DataFrame({'MÉTRICA': ['Puntuación F1'], 'VALOR': [round(f1*100, 2)]})
df_accuracy = pd.DataFrame({'MÉTRICA': ['Exactitud'], 'VALOR': [round(accuracy*100, 2)]})
df_nmi=pd.DataFrame({'MÉTRICA': ['Información Mutua Normalizada (NMI)'], 'VALOR': [round(nmi*100, 2)]})
df_kappa=pd.DataFrame({'MÉTRICA': ['Índice Kappa de Cohen'], 'VALOR': [round(kappa*100, 2)]})

# Concatenar los DataFrames
resultados_df_GD_prueba = pd.concat([resultados_df_GD, df_precision, df_recall, df_f1, df_accuracy,df_nmi,df_kappa], ignore_index=True)
resultados_df_GD_prueba["MODELO"]='GradientBoostingClassifier'
resultados_df_GD_prueba["TIPO_DE_DATOS"]='Prueba'
# Imprimir el DataFrame con los resultados
resultados_df_GD_prueba

Precisión:  80.17
Exhaustividad:  81.44
Puntuación F1:  80.54
Exactitud:  81.44
Información Mutua Normalizada (NMI): 35.05
Índice Kappa de Cohen: 64.06


,MÉTRICA,VALOR,MODELO,TIPO_DE_DATOS
0,Precisión,80.17,GradientBoostingClassifier,Prueba
1,Exhaustividad,81.44,GradientBoostingClassifier,Prueba
2,Puntuación F1,80.54,GradientBoostingClassifier,Prueba
3,Exactitud,81.44,GradientBoostingClassifier,Prueba
4,Información Mutua Normalizada (NMI),35.05,GradientBoostingClassifier,Prueba
5,Índice Kappa de Cohen,64.06,GradientBoostingClassifier,Prueba


In [57]:
resultados_df_GD = pd.concat([resultados_df_GD_prueba,resultados_df_GD_entrenamiento], ignore_index=True)
resultados_df_GD

,MÉTRICA,VALOR,MODELO,TIPO_DE_DATOS
0,Precisión,80.17,GradientBoostingClassifier,Prueba
1,Exhaustividad,81.44,GradientBoostingClassifier,Prueba
2,Puntuación F1,80.54,GradientBoostingClassifier,Prueba
3,Exactitud,81.44,GradientBoostingClassifier,Prueba
4,Información Mutua Normalizada (NMI),35.05,GradientBoostingClassifier,Prueba
5,Índice Kappa de Cohen,64.06,GradientBoostingClassifier,Prueba
6,Precisión,93.84,GradientBoostingClassifier,Entrenamiento
7,Exhaustividad,93.72,GradientBoostingClassifier,Entrenamiento
8,Puntuación F1,93.61,GradientBoostingClassifier,Entrenamiento
9,Exactitud,93.72,GradientBoostingClassifier,Entrenamiento


### XGBOOST

In [58]:
def entrenar_modelo_XB_con_transformacion(X_trn, Y_trn):
    # Aplicar la transformación Yeo-Johnson
    X_trn_transformado = X_trn
    parameters = {'reg_alpha': [0,0.1,0.2,0.3,0.4,0.5],
                'reg_lambda':  [i/100.0 for i in range(130,150,5)]}
    semilla=7
    modelo = XGBClassifier(random_state=semilla,                       
                        n_estimators=40,colsample_bytree = 1, subsample =1,max_depth =2,
                        min_child_weight =6,gamma = 0.05,learning_rate = 0.3)
    semilla=7
    num_folds=10
    kfold = KFold(n_splits=num_folds, random_state=semilla, shuffle=True)
    metrica = 'accuracy'
    grid = GridSearchCV(estimator=modelo, param_grid=parameters, scoring=metrica, cv=kfold, n_jobs=-1)
    grid_resultado = grid.fit(X_trn, Y_trn)
    print("Resultados de GridSearchCV para el modelo")
    print("Mejor valor EXACTITUD usando k-fold:", grid_resultado.best_score_*100)
    print("Mejor valor PARAMETRO usando k-fold:", grid_resultado.best_params_)
    mejor_modelo = XGBClassifier(**grid_resultado.best_params_)
    mejor_modelo.fit(X_trn_transformado, Y_trn)
    return mejor_modelo

X_trn = X_trn
Y_trn = Y_trn 

modelo_XB = entrenar_modelo_XB_con_transformacion(X_trn, Y_trn)


Resultados de GridSearchCV para el modelo
Mejor valor EXACTITUD usando k-fold: 83.81257344300825
Mejor valor PARAMETRO usando k-fold: {'reg_alpha': 0.4, 'reg_lambda': 1.4}


In [59]:
# Predecir las etiquetas para los datos de prueba
resultados_df_XB = pd.DataFrame(columns=['MÉTRICA', 'VALOR'])

Y_pred_entrenamiento= modelo_XB.predict(X_trn)

precision = precision_score(Y_trn, Y_pred_entrenamiento, average='weighted')
recall = recall_score(Y_trn, Y_pred_entrenamiento, average='weighted')
f1 = f1_score(Y_trn, Y_pred_entrenamiento, average='weighted')
accuracy = accuracy_score(Y_trn, Y_pred_entrenamiento)
nmi = normalized_mutual_info_score(Y_trn, Y_pred_entrenamiento)
kappa = cohen_kappa_score(Y_trn, Y_pred_entrenamiento)
print("Precisión: ", round(precision*100,2))
print("Exhaustividad: ", round(recall*100,2))
print("Puntuación F1: ", round(f1*100,2))
print("Exactitud: ", round(accuracy*100,2))
print("Información Mutua Normalizada (NMI):", round(nmi*100,2))
print("Índice Kappa de Cohen:", round(kappa*100,2))

# Crear un DataFrame para cada métrica
df_precision = pd.DataFrame({'MÉTRICA': ['Precisión'], 'VALOR': [round(precision*100, 2)]})
df_recall = pd.DataFrame({'MÉTRICA': ['Exhaustividad'], 'VALOR': [round(recall*100, 2)]})
df_f1 = pd.DataFrame({'MÉTRICA': ['Puntuación F1'], 'VALOR': [round(f1*100, 2)]})
df_accuracy = pd.DataFrame({'MÉTRICA': ['Exactitud'], 'VALOR': [round(accuracy*100, 2)]})
df_nmi=pd.DataFrame({'MÉTRICA': ['Información Mutua Normalizada (NMI)'], 'VALOR': [round(nmi*100, 2)]})
df_kappa=pd.DataFrame({'MÉTRICA': ['Índice Kappa de Cohen'], 'VALOR': [round(kappa*100, 2)]})

# Concatenar los DataFrames
resultados_df_XB_entrenamiento = pd.concat([resultados_df_XB, df_precision, df_recall, df_f1, df_accuracy,df_nmi,df_kappa], ignore_index=True)
resultados_df_XB_entrenamiento["MODELO"]=' XGBClassifier'
resultados_df_XB_entrenamiento["TIPO_DE_DATOS"]='Entrenamiento'
# Imprimir el DataFrame con los resultados
resultados_df_XB_entrenamiento

Precisión:  100.0
Exhaustividad:  100.0
Puntuación F1:  100.0
Exactitud:  100.0
Información Mutua Normalizada (NMI): 100.0
Índice Kappa de Cohen: 100.0


,MÉTRICA,VALOR,MODELO,TIPO_DE_DATOS
0,Precisión,100.0,XGBClassifier,Entrenamiento
1,Exhaustividad,100.0,XGBClassifier,Entrenamiento
2,Puntuación F1,100.0,XGBClassifier,Entrenamiento
3,Exactitud,100.0,XGBClassifier,Entrenamiento
4,Información Mutua Normalizada (NMI),100.0,XGBClassifier,Entrenamiento
5,Índice Kappa de Cohen,100.0,XGBClassifier,Entrenamiento


In [60]:
# Predecir las etiquetas para los datos de prueba
resultados_df_XB = pd.DataFrame(columns=['MÉTRICA', 'VALOR'])

Y_pred_prueba = modelo_XB.predict(X_tst)

precision = precision_score(Y_tst, Y_pred_prueba, average='weighted')
recall = recall_score(Y_tst, Y_pred_prueba, average='weighted')
f1 = f1_score(Y_tst, Y_pred_prueba, average='weighted')
accuracy = accuracy_score(Y_tst, Y_pred_prueba)
nmi = normalized_mutual_info_score(Y_tst, Y_pred_prueba)
kappa = cohen_kappa_score(Y_tst, Y_pred_prueba)
print("Precisión: ", round(precision*100,2))
print("Exhaustividad: ", round(recall*100,2))
print("Puntuación F1: ", round(f1*100,2))
print("Exactitud: ", round(accuracy*100,2))
print("Información Mutua Normalizada (NMI):", round(nmi*100,2))
print("Índice Kappa de Cohen:", round(kappa*100,2))

# Crear un DataFrame para cada métrica
df_precision = pd.DataFrame({'MÉTRICA': ['Precisión'], 'VALOR': [round(precision*100, 2)]})
df_recall = pd.DataFrame({'MÉTRICA': ['Exhaustividad'], 'VALOR': [round(recall*100, 2)]})
df_f1 = pd.DataFrame({'MÉTRICA': ['Puntuación F1'], 'VALOR': [round(f1*100, 2)]})
df_accuracy = pd.DataFrame({'MÉTRICA': ['Exactitud'], 'VALOR': [round(accuracy*100, 2)]})
df_nmi=pd.DataFrame({'MÉTRICA': ['Información Mutua Normalizada (NMI)'], 'VALOR': [round(nmi*100, 2)]})
df_kappa=pd.DataFrame({'MÉTRICA': ['Índice Kappa de Cohen'], 'VALOR': [round(kappa*100, 2)]})

# Concatenar los DataFrames
resultados_df_XB_prueba = pd.concat([resultados_df_XB, df_precision, df_recall, df_f1, df_accuracy,df_nmi,df_kappa], ignore_index=True)
resultados_df_XB_prueba["MODELO"]='XGBClassifier'
resultados_df_XB_prueba["TIPO_DE_DATOS"]='Prueba'
# Imprimir el DataFrame con los resultados
resultados_df_XB_prueba

Precisión:  82.37
Exhaustividad:  82.83
Puntuación F1:  81.88
Exactitud:  82.83
Información Mutua Normalizada (NMI): 37.42
Índice Kappa de Cohen: 66.6


,MÉTRICA,VALOR,MODELO,TIPO_DE_DATOS
0,Precisión,82.37,XGBClassifier,Prueba
1,Exhaustividad,82.83,XGBClassifier,Prueba
2,Puntuación F1,81.88,XGBClassifier,Prueba
3,Exactitud,82.83,XGBClassifier,Prueba
4,Información Mutua Normalizada (NMI),37.42,XGBClassifier,Prueba
5,Índice Kappa de Cohen,66.60,XGBClassifier,Prueba


In [61]:
resultados_df_XB = pd.concat([resultados_df_XB_prueba,resultados_df_XB_entrenamiento], ignore_index=True)
resultados_df_XB

,MÉTRICA,VALOR,MODELO,TIPO_DE_DATOS
0,Precisión,82.37,XGBClassifier,Prueba
1,Exhaustividad,82.83,XGBClassifier,Prueba
2,Puntuación F1,81.88,XGBClassifier,Prueba
3,Exactitud,82.83,XGBClassifier,Prueba
4,Información Mutua Normalizada (NMI),37.42,XGBClassifier,Prueba
5,Índice Kappa de Cohen,66.60,XGBClassifier,Prueba
6,Precisión,100.00,XGBClassifier,Entrenamiento
7,Exhaustividad,100.00,XGBClassifier,Entrenamiento
8,Puntuación F1,100.00,XGBClassifier,Entrenamiento
9,Exactitud,100.00,XGBClassifier,Entrenamiento


### CATBOOST

In [62]:
def entrenar_modelo_CB_con_transformacion(X_trn, Y_trn):
    # Aplicar la transformación Yeo-Johnson
    X_trn_transformado = X_trn
    parameters = {'border_count':[53],'l2_leaf_reg': [42]} 
    semilla=7
    modelo = CatBoostClassifier(random_state=semilla, verbose =0)
    semilla=7
    num_folds=10
    kfold = KFold(n_splits=num_folds, random_state=semilla, shuffle=True)
    metrica = 'accuracy'
    grid = GridSearchCV(estimator=modelo, param_grid=parameters, scoring=metrica, cv=kfold, n_jobs=-1)
    grid_resultado = grid.fit(X_trn, Y_trn)
    print("Resultados de GridSearchCV para el modelo")
    print("Mejor valor EXACTITUD usando k-fold:", grid_resultado.best_score_*100)
    print("Mejor valor PARAMETRO usando k-fold:", grid_resultado.best_params_)
    mejor_modelo = CatBoostClassifier(verbose=0,**grid_resultado.best_params_)
    mejor_modelo.fit(X_trn_transformado, Y_trn)
    return mejor_modelo

X_trn = X_trn
Y_trn = Y_trn 

modelo_CB = entrenar_modelo_CB_con_transformacion(X_trn, Y_trn)


Resultados de GridSearchCV para el modelo
Mejor valor EXACTITUD usando k-fold: 84.24706227967098
Mejor valor PARAMETRO usando k-fold: {'border_count': 53, 'l2_leaf_reg': 42}


In [63]:
# Predecir las etiquetas para los datos de prueba
resultados_df_CB = pd.DataFrame(columns=['MÉTRICA', 'VALOR'])

Y_pred_entrenamiento= modelo_CB.predict(X_trn)

precision = precision_score(Y_trn, Y_pred_entrenamiento, average='weighted')
recall = recall_score(Y_trn, Y_pred_entrenamiento, average='weighted')
f1 = f1_score(Y_trn, Y_pred_entrenamiento, average='weighted')
accuracy = accuracy_score(Y_trn, Y_pred_entrenamiento)
nmi = normalized_mutual_info_score(Y_trn, Y_pred_entrenamiento.ravel())
kappa = cohen_kappa_score(Y_trn, Y_pred_entrenamiento)
print("Precisión: ", round(precision*100,2))
print("Exhaustividad: ", round(recall*100,2))
print("Puntuación F1: ", round(f1*100,2))
print("Exactitud: ", round(accuracy*100,2))
print("Información Mutua Normalizada (NMI):", round(nmi*100,2))
print("Índice Kappa de Cohen:", round(kappa*100,2))

# Crear un DataFrame para cada métrica
df_precision = pd.DataFrame({'MÉTRICA': ['Precisión'], 'VALOR': [round(precision*100, 2)]})
df_recall = pd.DataFrame({'MÉTRICA': ['Exhaustividad'], 'VALOR': [round(recall*100, 2)]})
df_f1 = pd.DataFrame({'MÉTRICA': ['Puntuación F1'], 'VALOR': [round(f1*100, 2)]})
df_accuracy = pd.DataFrame({'MÉTRICA': ['Exactitud'], 'VALOR': [round(accuracy*100, 2)]})
df_nmi=pd.DataFrame({'MÉTRICA': ['Información Mutua Normalizada (NMI)'], 'VALOR': [round(nmi*100, 2)]})
df_kappa=pd.DataFrame({'MÉTRICA': ['Índice Kappa de Cohen'], 'VALOR': [round(kappa*100, 2)]})

# Concatenar los DataFrames
resultados_df_CB_entrenamiento = pd.concat([resultados_df_CB, df_precision, df_recall, df_f1, df_accuracy,df_nmi,df_kappa], ignore_index=True)
resultados_df_CB_entrenamiento["MODELO"]='CatBoostClassifier'
resultados_df_CB_entrenamiento["TIPO_DE_DATOS"]='Entrenamiento'
# Imprimir el DataFrame con los resultados
resultados_df_CB_entrenamiento

Precisión:  88.84
Exhaustividad:  88.74
Puntuación F1:  87.71
Exactitud:  88.74
Información Mutua Normalizada (NMI): 55.3
Índice Kappa de Cohen: 78.13


,MÉTRICA,VALOR,MODELO,TIPO_DE_DATOS
0,Precisión,88.84,CatBoostClassifier,Entrenamiento
1,Exhaustividad,88.74,CatBoostClassifier,Entrenamiento
2,Puntuación F1,87.71,CatBoostClassifier,Entrenamiento
3,Exactitud,88.74,CatBoostClassifier,Entrenamiento
4,Información Mutua Normalizada (NMI),55.30,CatBoostClassifier,Entrenamiento
5,Índice Kappa de Cohen,78.13,CatBoostClassifier,Entrenamiento


In [64]:
# Predecir las etiquetas para los datos de prueba
resultados_df_CB = pd.DataFrame(columns=['MÉTRICA', 'VALOR'])

Y_pred_prueba = modelo_CB.predict(X_tst)

precision = precision_score(Y_tst, Y_pred_prueba, average='weighted')
recall = recall_score(Y_tst, Y_pred_prueba, average='weighted')
f1 = f1_score(Y_tst, Y_pred_prueba, average='weighted')
accuracy = accuracy_score(Y_tst, Y_pred_prueba)
nmi = normalized_mutual_info_score(Y_tst, Y_pred_prueba.ravel())
kappa = cohen_kappa_score(Y_tst, Y_pred_prueba)
print("Precisión: ", round(precision*100,2))
print("Exhaustividad: ", round(recall*100,2))
print("Puntuación F1: ", round(f1*100,2))
print("Exactitud: ", round(accuracy*100,2))
print("Información Mutua Normalizada (NMI):", round(nmi*100,2))
print("Índice Kappa de Cohen:", round(kappa*100,2))

# Crear un DataFrame para cada métrica
df_precision = pd.DataFrame({'MÉTRICA': ['Precisión'], 'VALOR': [round(precision*100, 2)]})
df_recall = pd.DataFrame({'MÉTRICA': ['Exhaustividad'], 'VALOR': [round(recall*100, 2)]})
df_f1 = pd.DataFrame({'MÉTRICA': ['Puntuación F1'], 'VALOR': [round(f1*100, 2)]})
df_accuracy = pd.DataFrame({'MÉTRICA': ['Exactitud'], 'VALOR': [round(accuracy*100, 2)]})
df_nmi=pd.DataFrame({'MÉTRICA': ['Información Mutua Normalizada (NMI)'], 'VALOR': [round(nmi*100, 2)]})
df_kappa=pd.DataFrame({'MÉTRICA': ['Índice Kappa de Cohen'], 'VALOR': [round(kappa*100, 2)]})

# Concatenar los DataFrames
resultados_df_CB_prueba = pd.concat([resultados_df_CB, df_precision, df_recall, df_f1, df_accuracy,df_nmi,df_kappa], ignore_index=True)
resultados_df_CB_prueba["MODELO"]='CatBoostClassifier'
resultados_df_CB_prueba["TIPO_DE_DATOS"]='Prueba'
# Imprimir el DataFrame con los resultados
resultados_df_CB_prueba

Precisión:  83.65
Exhaustividad:  82.95
Puntuación F1:  81.51
Exactitud:  82.95
Información Mutua Normalizada (NMI): 38.26
Índice Kappa de Cohen: 66.63


,MÉTRICA,VALOR,MODELO,TIPO_DE_DATOS
0,Precisión,83.65,CatBoostClassifier,Prueba
1,Exhaustividad,82.95,CatBoostClassifier,Prueba
2,Puntuación F1,81.51,CatBoostClassifier,Prueba
3,Exactitud,82.95,CatBoostClassifier,Prueba
4,Información Mutua Normalizada (NMI),38.26,CatBoostClassifier,Prueba
5,Índice Kappa de Cohen,66.63,CatBoostClassifier,Prueba


In [65]:
resultados_df_CB = pd.concat([resultados_df_CB_prueba,resultados_df_CB_entrenamiento], ignore_index=True)
resultados_df_CB

,MÉTRICA,VALOR,MODELO,TIPO_DE_DATOS
0,Precisión,83.65,CatBoostClassifier,Prueba
1,Exhaustividad,82.95,CatBoostClassifier,Prueba
2,Puntuación F1,81.51,CatBoostClassifier,Prueba
3,Exactitud,82.95,CatBoostClassifier,Prueba
4,Información Mutua Normalizada (NMI),38.26,CatBoostClassifier,Prueba
5,Índice Kappa de Cohen,66.63,CatBoostClassifier,Prueba
6,Precisión,88.84,CatBoostClassifier,Entrenamiento
7,Exhaustividad,88.74,CatBoostClassifier,Entrenamiento
8,Puntuación F1,87.71,CatBoostClassifier,Entrenamiento
9,Exactitud,88.74,CatBoostClassifier,Entrenamiento


### LIGHT GBM

In [66]:
def entrenar_modelo_LIGHT_con_transformacion(X_trn, Y_trn):
    # Aplicar la transformación Yeo-Johnson
    X_trn_transformado = X_trn
    parameters = { 'min_child_samples' : [i for i in range(200, 10000, 100)]}
    semilla=7
    modelo = LGBMClassifier (random_state=semilla,                           
                        num_leaves =  10,max_depth = 1, n_estimators = 100,    
                        learning_rate = 0.1 ,class_weight=  None, subsample = 1,
                        colsample_bytree= 1, reg_alpha=  0, reg_lambda = 0,
                        min_split_gain = 0, boosting_type = 'gbdt')
    semilla=7
    num_folds=10
    kfold = KFold(n_splits=num_folds, random_state=semilla, shuffle=True)
    metrica = 'accuracy'
    grid = GridSearchCV(estimator=modelo, param_grid=parameters, scoring=metrica, cv=kfold, n_jobs=-1)
    grid_resultado = grid.fit(X_trn, Y_trn)
    print("Resultados de GridSearchCV para el modelo")
    print("Mejor valor EXACTITUD usando k-fold:", grid_resultado.best_score_*100)
    print("Mejor valor PARAMETRO usando k-fold:", grid_resultado.best_params_)
    mejor_modelo = LGBMClassifier(verbose=-1,**grid_resultado.best_params_)
    mejor_modelo.fit(X_trn_transformado, Y_trn)
    return mejor_modelo

X_trn = X_trn
Y_trn = Y_trn 

modelo_LIGHT = entrenar_modelo_LIGHT_con_transformacion(X_trn, Y_trn)

[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.001403 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 859
[LightGBM] [Info] Number of data points in the train set: 1847, number of used features: 18
[LightGBM] [Info] Start training from score -0.858185
[LightGBM] [Info] Start training from score -0.625635
[LightGBM] [Info] Start training from score -3.230859
[LightGBM] [Info] Start training from score -6.422706
Resultados de GridSearchCV para el modelo
Mejor valor EXACTITUD usando k-fold: 82.83754406580493
Mejor valor PARAMETRO usando k-fold: {'min_child_samples': 300}


In [67]:
resultados_df_LIGHT = pd.DataFrame(columns=['MÉTRICA', 'VALOR'])

Y_pred_entrenamiento= modelo_LIGHT.predict(X_trn)

precision = precision_score(Y_trn, Y_pred_entrenamiento, average='weighted')
recall = recall_score(Y_trn, Y_pred_entrenamiento, average='weighted')
f1 = f1_score(Y_trn, Y_pred_entrenamiento, average='weighted')
accuracy = accuracy_score(Y_trn, Y_pred_entrenamiento)
nmi = normalized_mutual_info_score(Y_trn, Y_pred_entrenamiento)
kappa = cohen_kappa_score(Y_trn, Y_pred_entrenamiento)
print("Precisión: ", round(precision*100,2))
print("Exhaustividad: ", round(recall*100,2))
print("Puntuación F1: ", round(f1*100,2))
print("Exactitud: ", round(accuracy*100,2))
print("Información Mutua Normalizada (NMI):", round(nmi*100,2))
print("Índice Kappa de Cohen:", round(kappa*100,2))

# Crear un DataFrame para cada métrica
df_precision = pd.DataFrame({'MÉTRICA': ['Precisión'], 'VALOR': [round(precision*100, 2)]})
df_recall = pd.DataFrame({'MÉTRICA': ['Exhaustividad'], 'VALOR': [round(recall*100, 2)]})
df_f1 = pd.DataFrame({'MÉTRICA': ['Puntuación F1'], 'VALOR': [round(f1*100, 2)]})
df_accuracy = pd.DataFrame({'MÉTRICA': ['Exactitud'], 'VALOR': [round(accuracy*100, 2)]})
df_nmi=pd.DataFrame({'MÉTRICA': ['Información Mutua Normalizada (NMI)'], 'VALOR': [round(nmi*100, 2)]})
df_kappa=pd.DataFrame({'MÉTRICA': ['Índice Kappa de Cohen'], 'VALOR': [round(kappa*100, 2)]})

# Concatenar los DataFrames
resultados_df_LIGHT_entrenamiento = pd.concat([resultados_df_LIGHT, df_precision, df_recall, df_f1, df_accuracy,df_nmi,df_kappa], ignore_index=True)
resultados_df_LIGHT_entrenamiento["MODELO"]='LGBMClassifier'
resultados_df_LIGHT_entrenamiento["TIPO_DE_DATOS"]='Entrenamiento'
# Imprimir el DataFrame con los resultados
resultados_df_LIGHT_entrenamiento

Precisión:  88.92
Exhaustividad:  88.47
Puntuación F1:  87.6
Exactitud:  88.47
Información Mutua Normalizada (NMI): 54.84
Índice Kappa de Cohen: 77.66


,MÉTRICA,VALOR,MODELO,TIPO_DE_DATOS
0,Precisión,88.92,LGBMClassifier,Entrenamiento
1,Exhaustividad,88.47,LGBMClassifier,Entrenamiento
2,Puntuación F1,87.60,LGBMClassifier,Entrenamiento
3,Exactitud,88.47,LGBMClassifier,Entrenamiento
4,Información Mutua Normalizada (NMI),54.84,LGBMClassifier,Entrenamiento
5,Índice Kappa de Cohen,77.66,LGBMClassifier,Entrenamiento


In [68]:
# Predecir las etiquetas para los datos de prueba
resultados_df_LIGHT = pd.DataFrame(columns=['MÉTRICA', 'VALOR'])

Y_pred_prueba = modelo_LIGHT.predict(X_tst)

precision = precision_score(Y_tst, Y_pred_prueba, average='weighted')
recall = recall_score(Y_tst, Y_pred_prueba, average='weighted')
f1 = f1_score(Y_tst, Y_pred_prueba, average='weighted')
accuracy = accuracy_score(Y_tst, Y_pred_prueba)
nmi = normalized_mutual_info_score(Y_tst, Y_pred_prueba)
kappa = cohen_kappa_score(Y_tst, Y_pred_prueba)
print("Precisión: ", round(precision*100,2))
print("Exhaustividad: ", round(recall*100,2))
print("Puntuación F1: ", round(f1*100,2))
print("Exactitud: ", round(accuracy*100,2))
print("Información Mutua Normalizada (NMI):", round(nmi*100,2))
print("Índice Kappa de Cohen:", round(kappa*100,2))

# Crear un DataFrame para cada métrica
df_precision = pd.DataFrame({'MÉTRICA': ['Precisión'], 'VALOR': [round(precision*100, 2)]})
df_recall = pd.DataFrame({'MÉTRICA': ['Exhaustividad'], 'VALOR': [round(recall*100, 2)]})
df_f1 = pd.DataFrame({'MÉTRICA': ['Puntuación F1'], 'VALOR': [round(f1*100, 2)]})
df_accuracy = pd.DataFrame({'MÉTRICA': ['Exactitud'], 'VALOR': [round(accuracy*100, 2)]})
df_nmi=pd.DataFrame({'MÉTRICA': ['Información Mutua Normalizada (NMI)'], 'VALOR': [round(nmi*100, 2)]})
df_kappa=pd.DataFrame({'MÉTRICA': ['Índice Kappa de Cohen'], 'VALOR': [round(kappa*100, 2)]})

# Concatenar los DataFrames
resultados_df_LIGHT_prueba = pd.concat([resultados_df_LIGHT, df_precision, df_recall, df_f1, df_accuracy,df_nmi,df_kappa], ignore_index=True)
resultados_df_LIGHT_prueba["MODELO"]='LGBMClassifier'
resultados_df_LIGHT_prueba["TIPO_DE_DATOS"]='Prueba'
# Imprimir el DataFrame con los resultados
resultados_df_LIGHT_prueba

Precisión:  82.11
Exhaustividad:  82.83
Puntuación F1:  81.77
Exactitud:  82.83
Información Mutua Normalizada (NMI): 37.95
Índice Kappa de Cohen: 66.66


,MÉTRICA,VALOR,MODELO,TIPO_DE_DATOS
0,Precisión,82.11,LGBMClassifier,Prueba
1,Exhaustividad,82.83,LGBMClassifier,Prueba
2,Puntuación F1,81.77,LGBMClassifier,Prueba
3,Exactitud,82.83,LGBMClassifier,Prueba
4,Información Mutua Normalizada (NMI),37.95,LGBMClassifier,Prueba
5,Índice Kappa de Cohen,66.66,LGBMClassifier,Prueba


In [69]:
resultados_df_LIGHT = pd.concat([resultados_df_LIGHT_prueba,resultados_df_LIGHT_entrenamiento], ignore_index=True)
resultados_df_LIGHT

,MÉTRICA,VALOR,MODELO,TIPO_DE_DATOS
0,Precisión,82.11,LGBMClassifier,Prueba
1,Exhaustividad,82.83,LGBMClassifier,Prueba
2,Puntuación F1,81.77,LGBMClassifier,Prueba
3,Exactitud,82.83,LGBMClassifier,Prueba
4,Información Mutua Normalizada (NMI),37.95,LGBMClassifier,Prueba
5,Índice Kappa de Cohen,66.66,LGBMClassifier,Prueba
6,Precisión,88.92,LGBMClassifier,Entrenamiento
7,Exhaustividad,88.47,LGBMClassifier,Entrenamiento
8,Puntuación F1,87.60,LGBMClassifier,Entrenamiento
9,Exactitud,88.47,LGBMClassifier,Entrenamiento


### VOTING - VOTACIÓN DURA

In [70]:
def entrenar_modelo_voting_hard_con_transformacion(X_trn, Y_trn):
    X_trn_transformado = X_trn
    semilla= 7 
    kfold = KFold(n_splits=10, random_state=semilla, shuffle=True)
    
    modelo1 = GradientBoostingClassifier(random_state=semilla, n_estimators=800, 
                                    learning_rate = 0.01, max_depth = 2,
                                    max_features =2, min_samples_leaf = 9, 
                                    min_samples_split = 2, subsample = 1 )
    
    base_estimator=DecisionTreeClassifier(ccp_alpha=0.0, class_weight=None, 
                                    criterion='gini',
                                    max_depth=8, max_features=1, max_leaf_nodes=None,
                                    min_impurity_decrease=0.0,
                                    min_samples_leaf=1, min_samples_split=5,
                                    min_weight_fraction_leaf=0.0,
                                    random_state=5, splitter='random')

    modelo2 = AdaBoostClassifier(estimator=base_estimator,n_estimators=1200, 
                                    algorithm ='SAMME.R', learning_rate = 0.001, 
                                    random_state=semilla)

    modelo3 = ExtraTreesClassifier(n_estimators=300, max_features=None,
                                    bootstrap = False, max_depth = 11,min_samples_split = 4, 
                                    min_samples_leaf = 1)

    modelo4 = RandomForestClassifier (n_estimators=1100, max_features=None,
                                    min_samples_split = 5, max_depth= 3, 
                                    min_samples_leaf= 1)

    model = DecisionTreeClassifier(criterion= 'gini', 
                                    max_depth=6, 
                                    max_features= None, 
                                    min_samples_leaf= 9, 
                                    min_samples_split = 2, 
                                    random_state= 10, 
                                    splitter= 'random')

    modelo5 = BaggingClassifier(estimator=model, n_estimators=50, 
                                    random_state=semilla,
                                    bootstrap= True, bootstrap_features = False, 
                                    max_features = 0.9,
                                    max_samples= 0.95)

    modelo6 = DecisionTreeClassifier(criterion= 'gini', max_depth=6,max_features= None, 
                                    min_samples_leaf= 9, 
                                    min_samples_split = 2, random_state= 10, splitter= 'random')

    modelo7 = XGBClassifier(n_estimators= 40, random_state=semilla,colsample_bytree = 1, 
                                    subsample =1, reg_alpha = 0.2, reg_lambda= 1.35,
                                    learning_rate= 0.3,max_depth =2, min_child_weight =6,
                                    gamma = 0.05)
    metrica = 'accuracy'
    mejor_modelo = VotingClassifier(
    estimators=[('Gradient', modelo1), ('Adaboost', modelo2), 
                                    ('Extratrees', modelo3),('Random Forest',modelo4),
                                    ('Bagging',modelo5),('Decision tree',modelo6),
                                    ('XGB',modelo7)],voting='hard') 
    mejor_modelo.fit(X_trn_transformado, Y_trn)
    resultados = cross_val_score(mejor_modelo, X_trn_transformado, Y_trn, 
                                    cv=kfold,scoring = metrica)
    print("Rendimiento del modelo:") 
    print ("Promedio de Exactitud (DATOS ENTRENAMIENTO) usando k-fold:", resultados.mean()*100," % ") 
    return mejor_modelo

X_trn = X_trn
Y_trn = Y_trn 

modelo_voting_hard = entrenar_modelo_voting_hard_con_transformacion(X_trn, Y_trn)

Rendimiento del modelo:
Promedio de Exactitud (DATOS ENTRENAMIENTO) usando k-fold: 83.16392479435957  % 


In [71]:
resultados_df_voting_hard = pd.DataFrame(columns=['MÉTRICA', 'VALOR'])

Y_pred_entrenamiento= modelo_voting_hard.predict(X_trn)

precision = precision_score(Y_trn, Y_pred_entrenamiento, average='weighted')
recall = recall_score(Y_trn, Y_pred_entrenamiento, average='weighted')
f1 = f1_score(Y_trn, Y_pred_entrenamiento, average='weighted')
accuracy = accuracy_score(Y_trn, Y_pred_entrenamiento)
nmi = normalized_mutual_info_score(Y_trn, Y_pred_entrenamiento)
kappa = cohen_kappa_score(Y_trn, Y_pred_entrenamiento)
print("Precisión: ", round(precision*100,2))
print("Exhaustividad: ", round(recall*100,2))
print("Puntuación F1: ", round(f1*100,2))
print("Exactitud: ", round(accuracy*100,2))
print("Información Mutua Normalizada (NMI):", round(nmi*100,2))
print("Índice Kappa de Cohen:", round(kappa*100,2))

# Crear un DataFrame para cada métrica
df_precision = pd.DataFrame({'MÉTRICA': ['Precisión'], 'VALOR': [round(precision*100, 2)]})
df_recall = pd.DataFrame({'MÉTRICA': ['Exhaustividad'], 'VALOR': [round(recall*100, 2)]})
df_f1 = pd.DataFrame({'MÉTRICA': ['Puntuación F1'], 'VALOR': [round(f1*100, 2)]})
df_accuracy = pd.DataFrame({'MÉTRICA': ['Exactitud'], 'VALOR': [round(accuracy*100, 2)]})
df_nmi=pd.DataFrame({'MÉTRICA': ['Información Mutua Normalizada (NMI)'], 'VALOR': [round(nmi*100, 2)]})
df_kappa=pd.DataFrame({'MÉTRICA': ['Índice Kappa de Cohen'], 'VALOR': [round(kappa*100, 2)]})

# Concatenar los DataFrames
resultados_df_voting_hard_entrenamiento = pd.concat([resultados_df_voting_hard, df_precision, df_recall, df_f1, df_accuracy,df_nmi,df_kappa], ignore_index=True)
resultados_df_voting_hard_entrenamiento["MODELO"]='VotingClassifier_hard'
resultados_df_voting_hard_entrenamiento["TIPO_DE_DATOS"]='Entrenamiento'
# Imprimir el DataFrame con los resultados
resultados_df_voting_hard_entrenamiento

Precisión:  88.02
Exhaustividad:  87.28
Puntuación F1:  85.53
Exactitud:  87.28
Información Mutua Normalizada (NMI): 51.75
Índice Kappa de Cohen: 75.02


,MÉTRICA,VALOR,MODELO,TIPO_DE_DATOS
0,Precisión,88.02,VotingClassifier_hard,Entrenamiento
1,Exhaustividad,87.28,VotingClassifier_hard,Entrenamiento
2,Puntuación F1,85.53,VotingClassifier_hard,Entrenamiento
3,Exactitud,87.28,VotingClassifier_hard,Entrenamiento
4,Información Mutua Normalizada (NMI),51.75,VotingClassifier_hard,Entrenamiento
5,Índice Kappa de Cohen,75.02,VotingClassifier_hard,Entrenamiento


In [72]:
# Predecir las etiquetas para los datos de prueba
resultados_df_voting_hard = pd.DataFrame(columns=['MÉTRICA', 'VALOR'])

Y_pred_prueba = modelo_voting_hard.predict(X_tst)

precision = precision_score(Y_tst, Y_pred_prueba, average='weighted')
recall = recall_score(Y_tst, Y_pred_prueba, average='weighted')
f1 = f1_score(Y_tst, Y_pred_prueba, average='weighted')
accuracy = accuracy_score(Y_tst, Y_pred_prueba)
nmi = normalized_mutual_info_score(Y_tst, Y_pred_prueba)
kappa = cohen_kappa_score(Y_tst, Y_pred_prueba)
print("Precisión: ", round(precision*100,2))
print("Exhaustividad: ", round(recall*100,2))
print("Puntuación F1: ", round(f1*100,2))
print("Exactitud: ", round(accuracy*100,2))
print("Información Mutua Normalizada (NMI):", round(nmi*100,2))
print("Índice Kappa de Cohen:", round(kappa*100,2))

# Crear un DataFrame para cada métrica
df_precision = pd.DataFrame({'MÉTRICA': ['Precisión'], 'VALOR': [round(precision*100, 2)]})
df_recall = pd.DataFrame({'MÉTRICA': ['Exhaustividad'], 'VALOR': [round(recall*100, 2)]})
df_f1 = pd.DataFrame({'MÉTRICA': ['Puntuación F1'], 'VALOR': [round(f1*100, 2)]})
df_accuracy = pd.DataFrame({'MÉTRICA': ['Exactitud'], 'VALOR': [round(accuracy*100, 2)]})
df_nmi=pd.DataFrame({'MÉTRICA': ['Información Mutua Normalizada (NMI)'], 'VALOR': [round(nmi*100, 2)]})
df_kappa=pd.DataFrame({'MÉTRICA': ['Índice Kappa de Cohen'], 'VALOR': [round(kappa*100, 2)]})

# Concatenar los DataFrames
resultados_df_voting_hard_prueba = pd.concat([resultados_df_voting_hard, df_precision, df_recall, df_f1, df_accuracy,df_nmi,df_kappa], ignore_index=True)
resultados_df_voting_hard_prueba["MODELO"]='VotingClassifier_hard'
resultados_df_voting_hard_prueba["TIPO_DE_DATOS"]='Prueba'
# Imprimir el DataFrame con los resultados
resultados_df_voting_hard_prueba

Precisión:  80.05
Exhaustividad:  82.95
Puntuación F1:  81.32
Exactitud:  82.95
Información Mutua Normalizada (NMI): 38.34
Índice Kappa de Cohen: 66.45


,MÉTRICA,VALOR,MODELO,TIPO_DE_DATOS
0,Precisión,80.05,VotingClassifier_hard,Prueba
1,Exhaustividad,82.95,VotingClassifier_hard,Prueba
2,Puntuación F1,81.32,VotingClassifier_hard,Prueba
3,Exactitud,82.95,VotingClassifier_hard,Prueba
4,Información Mutua Normalizada (NMI),38.34,VotingClassifier_hard,Prueba
5,Índice Kappa de Cohen,66.45,VotingClassifier_hard,Prueba


In [73]:
resultados_df_voting_hard = pd.concat([resultados_df_voting_hard_prueba,resultados_df_voting_hard_entrenamiento], ignore_index=True)
resultados_df_voting_hard

,MÉTRICA,VALOR,MODELO,TIPO_DE_DATOS
0,Precisión,80.05,VotingClassifier_hard,Prueba
1,Exhaustividad,82.95,VotingClassifier_hard,Prueba
2,Puntuación F1,81.32,VotingClassifier_hard,Prueba
3,Exactitud,82.95,VotingClassifier_hard,Prueba
4,Información Mutua Normalizada (NMI),38.34,VotingClassifier_hard,Prueba
5,Índice Kappa de Cohen,66.45,VotingClassifier_hard,Prueba
6,Precisión,88.02,VotingClassifier_hard,Entrenamiento
7,Exhaustividad,87.28,VotingClassifier_hard,Entrenamiento
8,Puntuación F1,85.53,VotingClassifier_hard,Entrenamiento
9,Exactitud,87.28,VotingClassifier_hard,Entrenamiento


### VOTING - VOTACIÓN SUAVE

In [74]:
def entrenar_modelo_voting_soft_con_transformacion(X_trn, Y_trn):
    X_trn_transformado = X_trn
    semilla= 7 
    kfold = KFold(n_splits=10, random_state=semilla, shuffle=True)
    modelo1 = GradientBoostingClassifier(random_state=semilla, n_estimators=800, 
                                    learning_rate = 0.01, max_depth = 2,
                                    max_features =2, min_samples_leaf = 9, 
                                    min_samples_split = 2, subsample = 1 )
    
    base_estimator=DecisionTreeClassifier(ccp_alpha=0.0, class_weight=None, criterion='gini',
                                    max_depth=8, max_features=1, max_leaf_nodes=None,
                                    min_impurity_decrease=0.0,
                                    min_samples_leaf=1, min_samples_split=5,
                                    min_weight_fraction_leaf=0.0,
                                    random_state=5, splitter='random')

    modelo2 = AdaBoostClassifier(estimator=base_estimator,n_estimators=1200, 
                                    algorithm ='SAMME.R', learning_rate = 0.001, 
                                    random_state=semilla)

    modelo3 = ExtraTreesClassifier(n_estimators=300, max_features=None,bootstrap = False, 
                                    max_depth = 11,min_samples_split = 4, 
                                    min_samples_leaf = 1)

    modelo4 = RandomForestClassifier (n_estimators=1100, max_features=None,
                                    min_samples_split = 5, max_depth= 3, min_samples_leaf= 1)

    model = DecisionTreeClassifier(criterion= 'gini', 
                                    max_depth=6, 
                                    max_features= None, 
                                    min_samples_leaf= 9, 
                                    min_samples_split = 2, 
                                    random_state= 10, 
                                    splitter= 'random')

    modelo5 = BaggingClassifier(estimator=model, n_estimators=50, random_state=semilla,
                                    bootstrap= True, bootstrap_features = False, 
                                    max_features = 0.9, max_samples= 0.95)

    modelo6 = DecisionTreeClassifier(criterion= 'gini', max_depth=6,max_features= None, 
                                    min_samples_leaf= 9, min_samples_split = 2, 
                                    random_state= 10, splitter= 'random')
    metrica = 'accuracy'
    mejor_modelo = VotingClassifier(
    estimators=[('Gradient', modelo1), ('Adaboost', modelo2), ('Extratrees', modelo3),
                                    ('Random Forest',modelo4),
                                    ('Bagging',modelo5),('Decision tree',modelo6)],
    voting='soft',weights=[0.9,0.7,0.9,0.9,0.9,0.9]) 
    mejor_modelo.fit(X_trn_transformado, Y_trn)
    resultados = cross_val_score(mejor_modelo, X_trn_transformado, Y_trn, cv=kfold,scoring = metrica)
    print("Rendimiento del modelo:") 
    print ("Promedio de Exactitud (DATOS ENTRENAMIENTO) usando k-fold:", resultados.mean()*100," % ") 
    return mejor_modelo

# Cargar los datos
#datos = cargar_datos(carrera, semestre)
X_trn = X_trn
Y_trn = Y_trn 

# Entrenar el modelo KNN con transformación Yeo-Johnson
modelo_voting_soft = entrenar_modelo_voting_soft_con_transformacion(X_trn, Y_trn)

Rendimiento del modelo:
Promedio de Exactitud (DATOS ENTRENAMIENTO) usando k-fold: 83.11045828437132  % 


In [75]:
resultados_df_voting_soft = pd.DataFrame(columns=['MÉTRICA', 'VALOR'])

Y_pred_entrenamiento= modelo_voting_soft.predict(X_trn)

precision = precision_score(Y_trn, Y_pred_entrenamiento, average='weighted')
recall = recall_score(Y_trn, Y_pred_entrenamiento, average='weighted')
f1 = f1_score(Y_trn, Y_pred_entrenamiento, average='weighted')
accuracy = accuracy_score(Y_trn, Y_pred_entrenamiento)
nmi = normalized_mutual_info_score(Y_trn, Y_pred_entrenamiento)
kappa = cohen_kappa_score(Y_trn, Y_pred_entrenamiento)
print("Precisión: ", round(precision*100,2))
print("Exhaustividad: ", round(recall*100,2))
print("Puntuación F1: ", round(f1*100,2))
print("Exactitud: ", round(accuracy*100,2))
print("Información Mutua Normalizada (NMI):", round(nmi*100,2))
print("Índice Kappa de Cohen:", round(kappa*100,2))

# Crear un DataFrame para cada métrica
df_precision = pd.DataFrame({'MÉTRICA': ['Precisión'], 'VALOR': [round(precision*100, 2)]})
df_recall = pd.DataFrame({'MÉTRICA': ['Exhaustividad'], 'VALOR': [round(recall*100, 2)]})
df_f1 = pd.DataFrame({'MÉTRICA': ['Puntuación F1'], 'VALOR': [round(f1*100, 2)]})
df_accuracy = pd.DataFrame({'MÉTRICA': ['Exactitud'], 'VALOR': [round(accuracy*100, 2)]})
df_nmi=pd.DataFrame({'MÉTRICA': ['Información Mutua Normalizada (NMI)'], 'VALOR': [round(nmi*100, 2)]})
df_kappa=pd.DataFrame({'MÉTRICA': ['Índice Kappa de Cohen'], 'VALOR': [round(kappa*100, 2)]})

# Concatenar los DataFrames
resultados_df_voting_soft_entrenamiento = pd.concat([resultados_df_voting_soft, df_precision, df_recall, df_f1, df_accuracy,df_nmi,df_kappa], ignore_index=True)
resultados_df_voting_soft_entrenamiento["MODELO"]='VotingClassifier_soft'
resultados_df_voting_soft_entrenamiento["TIPO_DE_DATOS"]='Entrenamiento'
# Imprimir el DataFrame con los resultados
resultados_df_voting_soft_entrenamiento

Precisión:  90.42
Exhaustividad:  89.66
Puntuación F1:  88.01
Exactitud:  89.66
Información Mutua Normalizada (NMI): 59.51
Índice Kappa de Cohen: 79.74


,MÉTRICA,VALOR,MODELO,TIPO_DE_DATOS
0,Precisión,90.42,VotingClassifier_soft,Entrenamiento
1,Exhaustividad,89.66,VotingClassifier_soft,Entrenamiento
2,Puntuación F1,88.01,VotingClassifier_soft,Entrenamiento
3,Exactitud,89.66,VotingClassifier_soft,Entrenamiento
4,Información Mutua Normalizada (NMI),59.51,VotingClassifier_soft,Entrenamiento
5,Índice Kappa de Cohen,79.74,VotingClassifier_soft,Entrenamiento


In [76]:
# Predecir las etiquetas para los datos de prueba
resultados_df_voting_soft = pd.DataFrame(columns=['MÉTRICA', 'VALOR'])

Y_pred_prueba = modelo_voting_soft.predict(X_tst)

precision = precision_score(Y_tst, Y_pred_prueba, average='weighted')
recall = recall_score(Y_tst, Y_pred_prueba, average='weighted')
f1 = f1_score(Y_tst, Y_pred_prueba, average='weighted')
accuracy = accuracy_score(Y_tst, Y_pred_prueba)
nmi = normalized_mutual_info_score(Y_tst, Y_pred_prueba)
kappa = cohen_kappa_score(Y_tst, Y_pred_prueba)
print("Precisión: ", round(precision*100,2))
print("Exhaustividad: ", round(recall*100,2))
print("Puntuación F1: ", round(f1*100,2))
print("Exactitud: ", round(accuracy*100,2))
print("Información Mutua Normalizada (NMI):", round(nmi*100,2))
print("Índice Kappa de Cohen:", round(kappa*100,2))

# Crear un DataFrame para cada métrica
df_precision = pd.DataFrame({'MÉTRICA': ['Precisión'], 'VALOR': [round(precision*100, 2)]})
df_recall = pd.DataFrame({'MÉTRICA': ['Exhaustividad'], 'VALOR': [round(recall*100, 2)]})
df_f1 = pd.DataFrame({'MÉTRICA': ['Puntuación F1'], 'VALOR': [round(f1*100, 2)]})
df_accuracy = pd.DataFrame({'MÉTRICA': ['Exactitud'], 'VALOR': [round(accuracy*100, 2)]})
df_nmi=pd.DataFrame({'MÉTRICA': ['Información Mutua Normalizada (NMI)'], 'VALOR': [round(nmi*100, 2)]})
df_kappa=pd.DataFrame({'MÉTRICA': ['Índice Kappa de Cohen'], 'VALOR': [round(kappa*100, 2)]})

# Concatenar los DataFrames
resultados_df_voting_soft_prueba = pd.concat([resultados_df_voting_soft, df_precision, df_recall, df_f1, df_accuracy,df_nmi,df_kappa], ignore_index=True)
resultados_df_voting_soft_prueba["MODELO"]='VotingClassifier_soft'
resultados_df_voting_soft_prueba["TIPO_DE_DATOS"]='Prueba'
# Imprimir el DataFrame con los resultados
resultados_df_voting_soft_prueba

Precisión:  79.86
Exhaustividad:  82.83
Puntuación F1:  81.2
Exactitud:  82.83
Información Mutua Normalizada (NMI): 38.0
Índice Kappa de Cohen: 66.25


,MÉTRICA,VALOR,MODELO,TIPO_DE_DATOS
0,Precisión,79.86,VotingClassifier_soft,Prueba
1,Exhaustividad,82.83,VotingClassifier_soft,Prueba
2,Puntuación F1,81.20,VotingClassifier_soft,Prueba
3,Exactitud,82.83,VotingClassifier_soft,Prueba
4,Información Mutua Normalizada (NMI),38.00,VotingClassifier_soft,Prueba
5,Índice Kappa de Cohen,66.25,VotingClassifier_soft,Prueba


In [77]:
resultados_df_voting_soft = pd.concat([resultados_df_voting_soft_prueba,resultados_df_voting_soft_entrenamiento], ignore_index=True)
resultados_df_voting_soft

,MÉTRICA,VALOR,MODELO,TIPO_DE_DATOS
0,Precisión,79.86,VotingClassifier_soft,Prueba
1,Exhaustividad,82.83,VotingClassifier_soft,Prueba
2,Puntuación F1,81.20,VotingClassifier_soft,Prueba
3,Exactitud,82.83,VotingClassifier_soft,Prueba
4,Información Mutua Normalizada (NMI),38.00,VotingClassifier_soft,Prueba
5,Índice Kappa de Cohen,66.25,VotingClassifier_soft,Prueba
6,Precisión,90.42,VotingClassifier_soft,Entrenamiento
7,Exhaustividad,89.66,VotingClassifier_soft,Entrenamiento
8,Puntuación F1,88.01,VotingClassifier_soft,Entrenamiento
9,Exactitud,89.66,VotingClassifier_soft,Entrenamiento


### STACKING ( METAMODELO LINEAL)

In [78]:
def entrenar_modelo_stacking_lineal_con_transformacion(X_trn, Y_trn):
    X_trn_transformado = X_trn
    semilla= 7 
    kfold = KFold(n_splits=10, random_state=semilla, shuffle=True)
    base_estimator=DecisionTreeClassifier(ccp_alpha=0.0, class_weight=None, criterion='gini',
                                    max_depth=8, max_features=1, max_leaf_nodes=None,
                                    min_impurity_decrease=0.0,
                                    min_samples_leaf=1, min_samples_split=5,
                                    min_weight_fraction_leaf=0.0,
                                    random_state=5, splitter='random')
    modelo2 = AdaBoostClassifier(estimator=base_estimator,n_estimators=500, 
                                    algorithm ='SAMME.R', learning_rate = 0.001, 
                                    random_state=semilla)

    modelo3 = ExtraTreesClassifier(n_estimators=300, max_features=None,bootstrap = False, 
                                    max_depth = 11,min_samples_split = 4, 
                                    min_samples_leaf = 1)

    modelo4 = RandomForestClassifier (n_estimators=300, max_features=None,
                                    min_samples_split = 5, max_depth= 3, min_samples_leaf= 1)
    model = DecisionTreeClassifier(criterion= 'gini', 
                                    max_depth=6, 
                                    max_features= None, 
                                    min_samples_leaf= 9, 
                                    min_samples_split = 2, 
                                    random_state= 10, 
                                    splitter= 'random')

    modelo5 = BaggingClassifier(estimator=model, n_estimators=50, random_state=semilla,
                                    bootstrap= True, bootstrap_features = False, 
                                    max_features = 0.9,
                                    max_samples= 0.95)

    modelo6 = DecisionTreeClassifier(criterion= 'gini', max_depth=6,max_features= None, 
                                    min_samples_leaf= 9, 
                                    min_samples_split = 2, random_state= 10, splitter= 'random')
    estimador_final = LogisticRegression()
    metrica = 'accuracy'
    mejor_modelo = StackingClassifier(
    estimators=[ ('Adaboost', modelo2), ('Extratrees', modelo3),('Random Forest',modelo4),
                                    ('Bagging',modelo5),('Decision tree',modelo6)], 
                                    final_estimator=estimador_final) 
    mejor_modelo.fit(X_trn_transformado, Y_trn)
    resultados = cross_val_score(mejor_modelo, X_trn_transformado, Y_trn, cv=kfold,scoring = metrica)
    print("Rendimiento del modelo:") 
    print ("Promedio de Exactitud (DATOS ENTRENAMIENTO) usando k-fold:", resultados.mean()*100," % ") 
    return mejor_modelo

X_trn = X_trn
Y_trn = Y_trn 

modelo_stacking_lineal = entrenar_modelo_stacking_lineal_con_transformacion(X_trn, Y_trn)

Rendimiento del modelo:
Promedio de Exactitud (DATOS ENTRENAMIENTO) usando k-fold: 83.3804347826087  % 


In [79]:
resultados_df_stacking_lineal = pd.DataFrame(columns=['MÉTRICA', 'VALOR'])

Y_pred_entrenamiento= modelo_stacking_lineal.predict(X_trn)

precision = precision_score(Y_trn, Y_pred_entrenamiento, average='weighted')
recall = recall_score(Y_trn, Y_pred_entrenamiento, average='weighted')
f1 = f1_score(Y_trn, Y_pred_entrenamiento, average='weighted')
accuracy = accuracy_score(Y_trn, Y_pred_entrenamiento)
nmi = normalized_mutual_info_score(Y_trn, Y_pred_entrenamiento)
kappa = cohen_kappa_score(Y_trn, Y_pred_entrenamiento)
print("Precisión: ", round(precision*100,2))
print("Exhaustividad: ", round(recall*100,2))
print("Puntuación F1: ", round(f1*100,2))
print("Exactitud: ", round(accuracy*100,2))
print("Información Mutua Normalizada (NMI):", round(nmi*100,2))
print("Índice Kappa de Cohen:", round(kappa*100,2))

# Crear un DataFrame para cada métrica
df_precision = pd.DataFrame({'MÉTRICA': ['Precisión'], 'VALOR': [round(precision*100, 2)]})
df_recall = pd.DataFrame({'MÉTRICA': ['Exhaustividad'], 'VALOR': [round(recall*100, 2)]})
df_f1 = pd.DataFrame({'MÉTRICA': ['Puntuación F1'], 'VALOR': [round(f1*100, 2)]})
df_accuracy = pd.DataFrame({'MÉTRICA': ['Exactitud'], 'VALOR': [round(accuracy*100, 2)]})
df_nmi=pd.DataFrame({'MÉTRICA': ['Información Mutua Normalizada (NMI)'], 'VALOR': [round(nmi*100, 2)]})
df_kappa=pd.DataFrame({'MÉTRICA': ['Índice Kappa de Cohen'], 'VALOR': [round(kappa*100, 2)]})

# Concatenar los DataFrames
resultados_df_stacking_lineal_entrenamiento = pd.concat([resultados_df_stacking_lineal, df_precision, df_recall, df_f1, df_accuracy,df_nmi,df_kappa], ignore_index=True)
resultados_df_stacking_lineal_entrenamiento["MODELO"]='StackingClassifier_lineal'
resultados_df_stacking_lineal_entrenamiento["TIPO_DE_DATOS"]='Entrenamiento'
# Imprimir el DataFrame con los resultados
resultados_df_stacking_lineal_entrenamiento

Precisión:  92.82
Exhaustividad:  92.69
Puntuación F1:  92.08
Exactitud:  92.69
Información Mutua Normalizada (NMI): 68.24
Índice Kappa de Cohen: 85.87


,MÉTRICA,VALOR,MODELO,TIPO_DE_DATOS
0,Precisión,92.82,StackingClassifier_lineal,Entrenamiento
1,Exhaustividad,92.69,StackingClassifier_lineal,Entrenamiento
2,Puntuación F1,92.08,StackingClassifier_lineal,Entrenamiento
3,Exactitud,92.69,StackingClassifier_lineal,Entrenamiento
4,Información Mutua Normalizada (NMI),68.24,StackingClassifier_lineal,Entrenamiento
5,Índice Kappa de Cohen,85.87,StackingClassifier_lineal,Entrenamiento


In [80]:
# Predecir las etiquetas para los datos de prueba
resultados_df_stacking_lineal = pd.DataFrame(columns=['MÉTRICA', 'VALOR'])

Y_pred_prueba = modelo_stacking_lineal.predict(X_tst)

precision = precision_score(Y_tst, Y_pred_prueba, average='weighted')
recall = recall_score(Y_tst, Y_pred_prueba, average='weighted')
f1 = f1_score(Y_tst, Y_pred_prueba, average='weighted')
accuracy = accuracy_score(Y_tst, Y_pred_prueba)
nmi = normalized_mutual_info_score(Y_tst, Y_pred_prueba)
kappa = cohen_kappa_score(Y_tst, Y_pred_prueba)
print("Precisión: ", round(precision*100,2))
print("Exhaustividad: ", round(recall*100,2))
print("Puntuación F1: ", round(f1*100,2))
print("Exactitud: ", round(accuracy*100,2))
print("Información Mutua Normalizada (NMI):", round(nmi*100,2))
print("Índice Kappa de Cohen:", round(kappa*100,2))

# Crear un DataFrame para cada métrica
df_precision = pd.DataFrame({'MÉTRICA': ['Precisión'], 'VALOR': [round(precision*100, 2)]})
df_recall = pd.DataFrame({'MÉTRICA': ['Exhaustividad'], 'VALOR': [round(recall*100, 2)]})
df_f1 = pd.DataFrame({'MÉTRICA': ['Puntuación F1'], 'VALOR': [round(f1*100, 2)]})
df_accuracy = pd.DataFrame({'MÉTRICA': ['Exactitud'], 'VALOR': [round(accuracy*100, 2)]})
df_nmi=pd.DataFrame({'MÉTRICA': ['Información Mutua Normalizada (NMI)'], 'VALOR': [round(nmi*100, 2)]})
df_kappa=pd.DataFrame({'MÉTRICA': ['Índice Kappa de Cohen'], 'VALOR': [round(kappa*100, 2)]})

# Concatenar los DataFrames
resultados_df_stacking_lineal_prueba = pd.concat([resultados_df_stacking_lineal, df_precision, df_recall, df_f1, df_accuracy,df_nmi,df_kappa], ignore_index=True)
resultados_df_stacking_lineal_prueba["MODELO"]='StackingClassifier_lineal'
resultados_df_stacking_lineal_prueba["TIPO_DE_DATOS"]='Prueba'
# Imprimir el DataFrame con los resultados
resultados_df_stacking_lineal_prueba

Precisión:  80.42
Exhaustividad:  83.21
Puntuación F1:  81.7
Exactitud:  83.21
Información Mutua Normalizada (NMI): 39.64
Índice Kappa de Cohen: 67.09


,MÉTRICA,VALOR,MODELO,TIPO_DE_DATOS
0,Precisión,80.42,StackingClassifier_lineal,Prueba
1,Exhaustividad,83.21,StackingClassifier_lineal,Prueba
2,Puntuación F1,81.70,StackingClassifier_lineal,Prueba
3,Exactitud,83.21,StackingClassifier_lineal,Prueba
4,Información Mutua Normalizada (NMI),39.64,StackingClassifier_lineal,Prueba
5,Índice Kappa de Cohen,67.09,StackingClassifier_lineal,Prueba


In [81]:
resultados_df_stacking_lineal = pd.concat([resultados_df_stacking_lineal_prueba,resultados_df_stacking_lineal_entrenamiento], ignore_index=True)
resultados_df_stacking_lineal

,MÉTRICA,VALOR,MODELO,TIPO_DE_DATOS
0,Precisión,80.42,StackingClassifier_lineal,Prueba
1,Exhaustividad,83.21,StackingClassifier_lineal,Prueba
2,Puntuación F1,81.70,StackingClassifier_lineal,Prueba
3,Exactitud,83.21,StackingClassifier_lineal,Prueba
4,Información Mutua Normalizada (NMI),39.64,StackingClassifier_lineal,Prueba
5,Índice Kappa de Cohen,67.09,StackingClassifier_lineal,Prueba
6,Precisión,92.82,StackingClassifier_lineal,Entrenamiento
7,Exhaustividad,92.69,StackingClassifier_lineal,Entrenamiento
8,Puntuación F1,92.08,StackingClassifier_lineal,Entrenamiento
9,Exactitud,92.69,StackingClassifier_lineal,Entrenamiento


### STACKING (METAMODELO NO LINEAL)

In [82]:
def entrenar_modelo_stacking_nolineal_con_transformacion(X_trn, Y_trn):
    X_trn_transformado = X_trn
    semilla= 7 
    kfold = KFold(n_splits=10, random_state=semilla, shuffle=True)
    modelo3 = ExtraTreesClassifier(n_estimators=400, max_features=None,bootstrap = False, 
                                    max_depth = 11,min_samples_split = 4, 
                                    min_samples_leaf = 1)

    modelo4 = RandomForestClassifier (n_estimators=500, max_features=None,
                                    min_samples_split = 5, max_depth= 3, min_samples_leaf= 1)
    model = DecisionTreeClassifier(criterion= 'gini', 
                                    max_depth=6, max_features= None,  min_samples_leaf= 9, 
                                    min_samples_split = 2, 
                                    random_state= 10, splitter= 'random')

    modelo5 = BaggingClassifier(estimator=model, n_estimators=50, random_state=semilla,
                                    bootstrap= True, bootstrap_features = False, 
                                    max_features = 0.9, max_samples= 0.95)

    modelo6 = DecisionTreeClassifier(criterion= 'gini', max_depth=6,max_features= None, 
                                    min_samples_leaf= 9, min_samples_split = 2, 
                                    random_state= 10, splitter= 'random')
    estimador_final = ExtraTreesClassifier(n_estimators=100, max_features=None,
                                    bootstrap = False, max_depth = 11,min_samples_split = 4, 
                                    min_samples_leaf = 1)
    metrica = 'accuracy'
    mejor_modelo = StackingClassifier(
    estimators=[  ('Extratrees', modelo3),('Random Forest',modelo4),('Bagging',modelo5),
                                    ('Decision tree',modelo6)], 
                                    final_estimator=estimador_final) 
    mejor_modelo.fit(X_trn_transformado, Y_trn)
    resultados = cross_val_score(mejor_modelo, X_trn_transformado, Y_trn, cv=kfold,scoring = metrica)
    print("Rendimiento del modelo:") 
    print ("Promedio de Exactitud (DATOS ENTRENAMIENTO) usando k-fold:", resultados.mean()*100," % ") 
    return mejor_modelo

X_trn = X_trn
Y_trn = Y_trn 

modelo_stacking_nolineal = entrenar_modelo_stacking_nolineal_con_transformacion(X_trn, Y_trn)

Rendimiento del modelo:
Promedio de Exactitud (DATOS ENTRENAMIENTO) usando k-fold: 82.45857814336077  % 


In [83]:
resultados_df_stacking_nolineal = pd.DataFrame(columns=['MÉTRICA', 'VALOR'])

Y_pred_entrenamiento= modelo_stacking_nolineal.predict(X_trn)

precision = precision_score(Y_trn, Y_pred_entrenamiento, average='weighted')
recall = recall_score(Y_trn, Y_pred_entrenamiento, average='weighted')
f1 = f1_score(Y_trn, Y_pred_entrenamiento, average='weighted')
accuracy = accuracy_score(Y_trn, Y_pred_entrenamiento)
nmi = normalized_mutual_info_score(Y_trn, Y_pred_entrenamiento)
kappa = cohen_kappa_score(Y_trn, Y_pred_entrenamiento)
print("Precisión: ", round(precision*100,2))
print("Exhaustividad: ", round(recall*100,2))
print("Puntuación F1: ", round(f1*100,2))
print("Exactitud: ", round(accuracy*100,2))
print("Información Mutua Normalizada (NMI):", round(nmi*100,2))
print("Índice Kappa de Cohen:", round(kappa*100,2))

# Crear un DataFrame para cada métrica
df_precision = pd.DataFrame({'MÉTRICA': ['Precisión'], 'VALOR': [round(precision*100, 2)]})
df_recall = pd.DataFrame({'MÉTRICA': ['Exhaustividad'], 'VALOR': [round(recall*100, 2)]})
df_f1 = pd.DataFrame({'MÉTRICA': ['Puntuación F1'], 'VALOR': [round(f1*100, 2)]})
df_accuracy = pd.DataFrame({'MÉTRICA': ['Exactitud'], 'VALOR': [round(accuracy*100, 2)]})
df_nmi=pd.DataFrame({'MÉTRICA': ['Información Mutua Normalizada (NMI)'], 'VALOR': [round(nmi*100, 2)]})
df_kappa=pd.DataFrame({'MÉTRICA': ['Índice Kappa de Cohen'], 'VALOR': [round(kappa*100, 2)]})

# Concatenar los DataFrames
resultados_df_stacking_nolineal_entrenamiento = pd.concat([resultados_df_stacking_nolineal, df_precision, df_recall, df_f1, df_accuracy,df_nmi,df_kappa], ignore_index=True)
resultados_df_stacking_nolineal_entrenamiento["MODELO"]='StackingClassifier_nolineal'
resultados_df_stacking_nolineal_entrenamiento["TIPO_DE_DATOS"]='Entrenamiento'
# Imprimir el DataFrame con los resultados
resultados_df_stacking_nolineal_entrenamiento

Precisión:  85.7
Exhaustividad:  85.81
Puntuación F1:  84.63
Exactitud:  85.81
Información Mutua Normalizada (NMI): 46.53
Índice Kappa de Cohen: 72.36


,MÉTRICA,VALOR,MODELO,TIPO_DE_DATOS
0,Precisión,85.70,StackingClassifier_nolineal,Entrenamiento
1,Exhaustividad,85.81,StackingClassifier_nolineal,Entrenamiento
2,Puntuación F1,84.63,StackingClassifier_nolineal,Entrenamiento
3,Exactitud,85.81,StackingClassifier_nolineal,Entrenamiento
4,Información Mutua Normalizada (NMI),46.53,StackingClassifier_nolineal,Entrenamiento
5,Índice Kappa de Cohen,72.36,StackingClassifier_nolineal,Entrenamiento


In [84]:
# Predecir las etiquetas para los datos de prueba
resultados_df_stacking_nolineal = pd.DataFrame(columns=['MÉTRICA', 'VALOR'])

Y_pred_prueba = modelo_stacking_nolineal.predict(X_tst)

precision = precision_score(Y_tst, Y_pred_prueba, average='weighted')
recall = recall_score(Y_tst, Y_pred_prueba, average='weighted')
f1 = f1_score(Y_tst, Y_pred_prueba, average='weighted')
accuracy = accuracy_score(Y_tst, Y_pred_prueba)
nmi = normalized_mutual_info_score(Y_tst, Y_pred_prueba)
kappa = cohen_kappa_score(Y_tst, Y_pred_prueba)
print("Precisión: ", round(precision*100,2))
print("Exhaustividad: ", round(recall*100,2))
print("Puntuación F1: ", round(f1*100,2))
print("Exactitud: ", round(accuracy*100,2))
print("Información Mutua Normalizada (NMI):", round(nmi*100,2))
print("Índice Kappa de Cohen:", round(kappa*100,2))

# Crear un DataFrame para cada métrica
df_precision = pd.DataFrame({'MÉTRICA': ['Precisión'], 'VALOR': [round(precision*100, 2)]})
df_recall = pd.DataFrame({'MÉTRICA': ['Exhaustividad'], 'VALOR': [round(recall*100, 2)]})
df_f1 = pd.DataFrame({'MÉTRICA': ['Puntuación F1'], 'VALOR': [round(f1*100, 2)]})
df_accuracy = pd.DataFrame({'MÉTRICA': ['Exactitud'], 'VALOR': [round(accuracy*100, 2)]})
df_nmi=pd.DataFrame({'MÉTRICA': ['Información Mutua Normalizada (NMI)'], 'VALOR': [round(nmi*100, 2)]})
df_kappa=pd.DataFrame({'MÉTRICA': ['Índice Kappa de Cohen'], 'VALOR': [round(kappa*100, 2)]})

# Concatenar los DataFrames
resultados_df_stacking_nolineal_prueba = pd.concat([resultados_df_stacking_nolineal, df_precision, df_recall, df_f1, df_accuracy,df_nmi,df_kappa], ignore_index=True)
resultados_df_stacking_nolineal_prueba["MODELO"]='StackingClassifier_nolineal'
resultados_df_stacking_nolineal_prueba["TIPO_DE_DATOS"]='Prueba'
# Imprimir el DataFrame con los resultados
resultados_df_stacking_nolineal_prueba

Precisión:  79.64
Exhaustividad:  82.2
Puntuación F1:  80.81
Exactitud:  82.2
Información Mutua Normalizada (NMI): 37.55
Índice Kappa de Cohen: 65.2


,MÉTRICA,VALOR,MODELO,TIPO_DE_DATOS
0,Precisión,79.64,StackingClassifier_nolineal,Prueba
1,Exhaustividad,82.20,StackingClassifier_nolineal,Prueba
2,Puntuación F1,80.81,StackingClassifier_nolineal,Prueba
3,Exactitud,82.20,StackingClassifier_nolineal,Prueba
4,Información Mutua Normalizada (NMI),37.55,StackingClassifier_nolineal,Prueba
5,Índice Kappa de Cohen,65.20,StackingClassifier_nolineal,Prueba


In [85]:
resultados_df_stacking_nolineal = pd.concat([resultados_df_stacking_nolineal_prueba,resultados_df_stacking_nolineal_entrenamiento], ignore_index=True)
resultados_df_stacking_nolineal

,MÉTRICA,VALOR,MODELO,TIPO_DE_DATOS
0,Precisión,79.64,StackingClassifier_nolineal,Prueba
1,Exhaustividad,82.20,StackingClassifier_nolineal,Prueba
2,Puntuación F1,80.81,StackingClassifier_nolineal,Prueba
3,Exactitud,82.20,StackingClassifier_nolineal,Prueba
4,Información Mutua Normalizada (NMI),37.55,StackingClassifier_nolineal,Prueba
5,Índice Kappa de Cohen,65.20,StackingClassifier_nolineal,Prueba
6,Precisión,85.70,StackingClassifier_nolineal,Entrenamiento
7,Exhaustividad,85.81,StackingClassifier_nolineal,Entrenamiento
8,Puntuación F1,84.63,StackingClassifier_nolineal,Entrenamiento
9,Exactitud,85.81,StackingClassifier_nolineal,Entrenamiento


### VOTING WEIGHTED VOTING 

In [86]:
import numpy as np
from sklearn.model_selection import KFold, cross_val_score
from sklearn.metrics import accuracy_score
from sklearn.ensemble import GradientBoostingClassifier, AdaBoostClassifier, ExtraTreesClassifier, RandomForestClassifier, BaggingClassifier
from sklearn.tree import DecisionTreeClassifier
from xgboost import XGBClassifier

def weighted_voting_ensemble(X_trn, Y_trn):
    X_trn_transformado = X_trn
    modelos = [
        GradientBoostingClassifier(random_state=7, n_estimators=800, learning_rate=0.01, max_depth=2, max_features=2, min_samples_leaf=9, min_samples_split=2, subsample=1),
        AdaBoostClassifier(estimator=DecisionTreeClassifier(ccp_alpha=0.0, class_weight=None, criterion='gini', max_depth=8, max_features=1, max_leaf_nodes=None, min_impurity_decrease=0.0, min_samples_leaf=1, min_samples_split=5, min_weight_fraction_leaf=0.0, random_state=5, splitter='random'), n_estimators=1200, algorithm='SAMME.R', learning_rate=0.001, random_state=7),
        ExtraTreesClassifier(n_estimators=300, max_features=None, bootstrap=False, max_depth=11, min_samples_split=4, min_samples_leaf=1),
        RandomForestClassifier(n_estimators=300, max_features=None, min_samples_split=5, max_depth=3, min_samples_leaf=1),
        BaggingClassifier(estimator=DecisionTreeClassifier(criterion='gini', max_depth=6, max_features=None, min_samples_leaf=9, min_samples_split=2, random_state=10, splitter='random'), n_estimators=50, random_state=7, bootstrap=True, bootstrap_features=False, max_features=0.9, max_samples=0.95),
        DecisionTreeClassifier(criterion='gini', max_depth=6, max_features=None, min_samples_leaf=9, min_samples_split=2, random_state=10, splitter='random'),
        XGBClassifier(n_estimators=40, random_state=7, colsample_bytree=1, subsample=1, reg_alpha=0.2, reg_lambda=1.35, learning_rate=0.3, max_depth=2, min_child_weight=6, gamma=0.05)
    ]
    
    # Inicializar pesos para el voto ponderado
    pesos = np.ones(len(modelos))
    
    # Entrenar cada modelo y obtener su precisión
    precisión_modelos = []
    for modelo in modelos:
        modelo.fit(X_trn_transformado, Y_trn)  # <-- Ajustar el modelo
        precisión = np.mean(cross_val_score(modelo, X_trn_transformado, Y_trn, cv=KFold(n_splits=10, random_state=7, shuffle=True), scoring='accuracy'))
        precisión_modelos.append(precisión)
    
    # Calcular los pesos como inversa de la precisión de cada modelo
    suma_precisión = sum(precisión_modelos)
    for i in range(len(precisión_modelos)):
        pesos[i] = suma_precisión / (precisión_modelos[i] * len(modelos))
    
    # Devolver modelos y pesos
    return modelos, pesos

def weighted_voting_predict(modelos, pesos, X_test):
    # Obtener las predicciones de cada modelo
    predicciones_modelos = [modelo.predict(X_test) for modelo in modelos]
    
    # Convertir los pesos a float64
    pesos_float64 = np.array(pesos, dtype=np.float64)
    
    # Calcular las predicciones ponderadas
    predicciones_ponderadas = np.zeros_like(predicciones_modelos[0], dtype=np.float64)
    for i, predicciones_modelo in enumerate(predicciones_modelos):
        predicciones_ponderadas += predicciones_modelo * pesos_float64[i]
    
    # Normalizar las predicciones ponderadas
    predicciones_ponderadas /= len(modelos)
    predicciones_ponderadas = np.round(predicciones_ponderadas).astype(int)
    
    return predicciones_ponderadas

# Utilizar la función
modelos, pesos = weighted_voting_ensemble(X_trn, Y_trn)

# Predecir las etiquetas para los datos de prueba
Y_pred = weighted_voting_predict(modelos, pesos, X_tst)

# Calcular la precisión de las predicciones
precision = accuracy_score(Y_tst, Y_pred)
recall = recall_score(Y_tst, Y_pred, average='weighted')
f1 = f1_score(Y_tst, Y_pred, average='weighted')
accuracy = accuracy_score(Y_tst, Y_pred)
print("Precisión: ", precision)
print("Exhaustividad (Recall): ", recall)
print("Puntuación F1: ", f1)
print("Exactitud: ", accuracy)
print("Precisión del modelo (Ensamblaje con voto ponderado): {:.2f}%".format(precision * 100))


Precisión:  0.8282828282828283
Exhaustividad (Recall):  0.8282828282828283
Puntuación F1:  0.8119376229162162
Exactitud:  0.8282828282828283
Precisión del modelo (Ensamblaje con voto ponderado): 82.83%


In [87]:
resultados_df_weighted_voting = pd.DataFrame(columns=['MÉTRICA', 'VALOR'])

Y_pred_entrenamiento= weighted_voting_predict(modelos, pesos, X_trn)

precision = precision_score(Y_trn, Y_pred_entrenamiento, average='weighted')
recall = recall_score(Y_trn, Y_pred_entrenamiento, average='weighted')
f1 = f1_score(Y_trn, Y_pred_entrenamiento, average='weighted')
accuracy = accuracy_score(Y_trn, Y_pred_entrenamiento)
nmi = normalized_mutual_info_score(Y_trn, Y_pred_entrenamiento)
kappa = cohen_kappa_score(Y_trn, Y_pred_entrenamiento)
print("Precisión: ", round(precision*100,2))
print("Exhaustividad: ", round(recall*100,2))
print("Puntuación F1: ", round(f1*100,2))
print("Exactitud: ", round(accuracy*100,2))
print("Información Mutua Normalizada (NMI):", round(nmi*100,2))
print("Índice Kappa de Cohen:", round(kappa*100,2))

# Crear un DataFrame para cada métrica
df_precision = pd.DataFrame({'MÉTRICA': ['Precisión'], 'VALOR': [round(precision*100, 2)]})
df_recall = pd.DataFrame({'MÉTRICA': ['Exhaustividad'], 'VALOR': [round(recall*100, 2)]})
df_f1 = pd.DataFrame({'MÉTRICA': ['Puntuación F1'], 'VALOR': [round(f1*100, 2)]})
df_accuracy = pd.DataFrame({'MÉTRICA': ['Exactitud'], 'VALOR': [round(accuracy*100, 2)]})
df_nmi=pd.DataFrame({'MÉTRICA': ['Información Mutua Normalizada (NMI)'], 'VALOR': [round(nmi*100, 2)]})
df_kappa=pd.DataFrame({'MÉTRICA': ['Índice Kappa de Cohen'], 'VALOR': [round(kappa*100, 2)]})

# Concatenar los DataFrames
resultados_df_weighted_voting_entrenamiento = pd.concat([resultados_df_weighted_voting, df_precision, df_recall, df_f1, df_accuracy,df_nmi,df_kappa], ignore_index=True)
resultados_df_weighted_voting_entrenamiento["MODELO"]='VotingClassifier_weighted'
resultados_df_weighted_voting_entrenamiento["TIPO_DE_DATOS"]='Entrenamiento'
# Imprimir el DataFrame con los resultados
resultados_df_weighted_voting_entrenamiento

Precisión:  85.18
Exhaustividad:  87.28
Puntuación F1:  85.6
Exactitud:  87.28
Información Mutua Normalizada (NMI): 52.72
Índice Kappa de Cohen: 75.06


,MÉTRICA,VALOR,MODELO,TIPO_DE_DATOS
0,Precisión,85.18,VotingClassifier_weighted,Entrenamiento
1,Exhaustividad,87.28,VotingClassifier_weighted,Entrenamiento
2,Puntuación F1,85.60,VotingClassifier_weighted,Entrenamiento
3,Exactitud,87.28,VotingClassifier_weighted,Entrenamiento
4,Información Mutua Normalizada (NMI),52.72,VotingClassifier_weighted,Entrenamiento
5,Índice Kappa de Cohen,75.06,VotingClassifier_weighted,Entrenamiento


In [88]:
# Predecir las etiquetas para los datos de prueba
resultados_df_weighted_voting = pd.DataFrame(columns=['MÉTRICA', 'VALOR'])

Y_pred_prueba= weighted_voting_predict(modelos, pesos, X_tst)

precision = precision_score(Y_tst, Y_pred_prueba, average='weighted')
recall = recall_score(Y_tst, Y_pred_prueba, average='weighted')
f1 = f1_score(Y_tst, Y_pred_prueba, average='weighted')
accuracy = accuracy_score(Y_tst, Y_pred_prueba)
nmi = normalized_mutual_info_score(Y_tst, Y_pred_prueba)
kappa = cohen_kappa_score(Y_tst, Y_pred_prueba)
print("Precisión: ", round(precision*100,2))
print("Exhaustividad: ", round(recall*100,2))
print("Puntuación F1: ", round(f1*100,2))
print("Exactitud: ", round(accuracy*100,2))
print("Información Mutua Normalizada (NMI):", round(nmi*100,2))
print("Índice Kappa de Cohen:", round(kappa*100,2))

# Crear un DataFrame para cada métrica
df_precision = pd.DataFrame({'MÉTRICA': ['Precisión'], 'VALOR': [round(precision*100, 2)]})
df_recall = pd.DataFrame({'MÉTRICA': ['Exhaustividad'], 'VALOR': [round(recall*100, 2)]})
df_f1 = pd.DataFrame({'MÉTRICA': ['Puntuación F1'], 'VALOR': [round(f1*100, 2)]})
df_accuracy = pd.DataFrame({'MÉTRICA': ['Exactitud'], 'VALOR': [round(accuracy*100, 2)]})
df_nmi=pd.DataFrame({'MÉTRICA': ['Información Mutua Normalizada (NMI)'], 'VALOR': [round(nmi*100, 2)]})
df_kappa=pd.DataFrame({'MÉTRICA': ['Índice Kappa de Cohen'], 'VALOR': [round(kappa*100, 2)]})

# Concatenar los DataFrames
resultados_df_weighted_voting_prueba = pd.concat([resultados_df_weighted_voting, df_precision, df_recall, df_f1, df_accuracy,df_nmi,df_kappa], ignore_index=True)
resultados_df_weighted_voting_prueba["MODELO"]='VotingClassifier_weighted'
resultados_df_weighted_voting_prueba["TIPO_DE_DATOS"]='Prueba'
# Imprimir el DataFrame con los resultados
resultados_df_weighted_voting_prueba

Precisión:  79.95
Exhaustividad:  82.83
Puntuación F1:  81.19
Exactitud:  82.83
Información Mutua Normalizada (NMI): 38.04
Índice Kappa de Cohen: 66.2


,MÉTRICA,VALOR,MODELO,TIPO_DE_DATOS
0,Precisión,79.95,VotingClassifier_weighted,Prueba
1,Exhaustividad,82.83,VotingClassifier_weighted,Prueba
2,Puntuación F1,81.19,VotingClassifier_weighted,Prueba
3,Exactitud,82.83,VotingClassifier_weighted,Prueba
4,Información Mutua Normalizada (NMI),38.04,VotingClassifier_weighted,Prueba
5,Índice Kappa de Cohen,66.20,VotingClassifier_weighted,Prueba


In [89]:
resultados_df_weighted_voting = pd.concat([resultados_df_weighted_voting_prueba,resultados_df_weighted_voting_entrenamiento], ignore_index=True)
resultados_df_weighted_voting

,MÉTRICA,VALOR,MODELO,TIPO_DE_DATOS
0,Precisión,79.95,VotingClassifier_weighted,Prueba
1,Exhaustividad,82.83,VotingClassifier_weighted,Prueba
2,Puntuación F1,81.19,VotingClassifier_weighted,Prueba
3,Exactitud,82.83,VotingClassifier_weighted,Prueba
4,Información Mutua Normalizada (NMI),38.04,VotingClassifier_weighted,Prueba
5,Índice Kappa de Cohen,66.20,VotingClassifier_weighted,Prueba
6,Precisión,85.18,VotingClassifier_weighted,Entrenamiento
7,Exhaustividad,87.28,VotingClassifier_weighted,Entrenamiento
8,Puntuación F1,85.60,VotingClassifier_weighted,Entrenamiento
9,Exactitud,87.28,VotingClassifier_weighted,Entrenamiento


### SUPER APRENDIZ MLENS

In [90]:
Y_trn = Y_trn.astype(int)
def entrenar_modelo_super_aprendiz(X_trn, Y_trn):
    X_trn_transformado = X_trn
    semilla = 7 
    kfold = KFold(n_splits=10, random_state=semilla, shuffle=True)
    modelo3 = ExtraTreesClassifier(n_estimators=400, max_features=None, bootstrap=False, 
                                   max_depth=11, min_samples_split=4, min_samples_leaf=1)

    modelo4 = RandomForestClassifier(n_estimators=500, max_features=None,
                                     min_samples_split=5, max_depth=3, min_samples_leaf=1)
    model = DecisionTreeClassifier(criterion='gini', 
                                   max_depth=6, max_features=None,  min_samples_leaf=9, 
                                   min_samples_split=2, 
                                   random_state=10, splitter='random')

    modelo5 = BaggingClassifier(estimator=model, n_estimators=50, random_state=semilla,
                                bootstrap=True, bootstrap_features=False, 
                                max_features=0.9, max_samples=0.95)

    modelo6 = DecisionTreeClassifier(criterion='gini', max_depth=6, max_features=None, 
                                     min_samples_leaf=9, min_samples_split=2, 
                                     random_state=10, splitter='random')
    
    modelo7 = GradientBoostingClassifier(n_estimators=50, random_state=semilla,
                                     max_depth= 2,
                                    min_samples_split= 13,
                                    min_samples_leaf= 1,
                                    subsample =1,
                                    learning_rate= 0.06                                    
                                    ) 

    estimadores = [('Extratrees', modelo3), ('Random Forest', modelo4), 
                   ('Bagging', modelo5), ('Decision tree', modelo6),('Gradient',modelo7)]
    
    super_learner = SuperLearner(folds=10, random_state=semilla, verbose=2)
    super_learner.add(estimadores)
    
    estimador_final = ExtraTreesClassifier(n_estimators=100, max_features=None,
                                           bootstrap=False, max_depth=11, min_samples_split=4, 
                                           min_samples_leaf=1)
    
    super_learner.add_meta(estimador_final)
    
    super_learner.fit(X_trn_transformado, Y_trn)
    resultados = cross_val_score(super_learner, X_trn_transformado, Y_trn, cv=kfold, scoring='accuracy')
    
    print("Rendimiento del modelo:") 
    print("Promedio de Exactitud (DATOS ENTRENAMIENTO) usando k-fold:", resultados.mean() * 100, "%") 
    return super_learner


X_trn = X_trn
Y_trn = Y_trn 

modelo_superaprendiz = entrenar_modelo_super_aprendiz(X_trn, Y_trn)



Fitting 2 layers
Processing layer-1             done | 00:00:17
Processing layer-2             done | 00:00:00
Fit complete                        | 00:00:17

Fitting 2 layers
Processing layer-1             done | 00:00:16
Processing layer-2             done | 00:00:00
Fit complete                        | 00:00:17

Predicting 2 layers
Processing layer-1             done | 00:00:00
Processing layer-2             done | 00:00:00
Predict complete                    | 00:00:00

Fitting 2 layers
Processing layer-1             done | 00:00:16
Processing layer-2             done | 00:00:00
Fit complete                        | 00:00:16

Predicting 2 layers
Processing layer-1             done | 00:00:00
Processing layer-2             done | 00:00:00
Predict complete                    | 00:00:00

Fitting 2 layers
Processing layer-1             done | 00:00:18
Processing layer-2             done | 00:00:00
Fit complete                        | 00:00:18

Predicting 2 layers
Processing layer-1 

In [91]:
print(modelo_superaprendiz.data)

                          ft-m  ft-s  pt-m  pt-s
layer-1  bagging          2.13  0.11  0.25  0.10
layer-1  decision-tree    0.08  0.06  0.02  0.03
layer-1  extratrees       8.05  0.12  1.36  0.47
layer-1  gradient         6.23  0.61  0.01  0.00
layer-1  random-forest    7.82  0.34  0.37  0.11



In [92]:
resultados_df_superaprendiz = pd.DataFrame(columns=['MÉTRICA', 'VALOR'])

Y_pred_entrenamiento= modelo_superaprendiz.predict(X_trn)

precision = precision_score(Y_trn, Y_pred_entrenamiento, average='weighted')
recall = recall_score(Y_trn, Y_pred_entrenamiento, average='weighted')
f1 = f1_score(Y_trn, Y_pred_entrenamiento, average='weighted')
accuracy = accuracy_score(Y_trn, Y_pred_entrenamiento)
nmi = normalized_mutual_info_score(Y_trn, Y_pred_entrenamiento)
kappa = cohen_kappa_score(Y_trn, Y_pred_entrenamiento)
print("Precisión: ", round(precision*100,2))
print("Exhaustividad: ", round(recall*100,2))
print("Puntuación F1: ", round(f1*100,2))
print("Exactitud: ", round(accuracy*100,2))
print("Información Mutua Normalizada (NMI):", round(nmi*100,2))
print("Índice Kappa de Cohen:", round(kappa*100,2))

# Crear un DataFrame para cada métrica
df_precision = pd.DataFrame({'MÉTRICA': ['Precisión'], 'VALOR': [round(precision*100, 2)]})
df_recall = pd.DataFrame({'MÉTRICA': ['Exhaustividad'], 'VALOR': [round(recall*100, 2)]})
df_f1 = pd.DataFrame({'MÉTRICA': ['Puntuación F1'], 'VALOR': [round(f1*100, 2)]})
df_accuracy = pd.DataFrame({'MÉTRICA': ['Exactitud'], 'VALOR': [round(accuracy*100, 2)]})
df_nmi=pd.DataFrame({'MÉTRICA': ['Información Mutua Normalizada (NMI)'], 'VALOR': [round(nmi*100, 2)]})
df_kappa=pd.DataFrame({'MÉTRICA': ['Índice Kappa de Cohen'], 'VALOR': [round(kappa*100, 2)]})

# Concatenar los DataFrames
resultados_df_superaprendiz_entrenamiento = pd.concat([resultados_df_superaprendiz, df_precision, df_recall, df_f1, df_accuracy,df_nmi,df_kappa], ignore_index=True)
resultados_df_superaprendiz_entrenamiento["MODELO"]='Super_Aprendiz'
resultados_df_superaprendiz_entrenamiento["TIPO_DE_DATOS"]='Entrenamiento'
# Imprimir el DataFrame con los resultados
resultados_df_superaprendiz_entrenamiento


Predicting 2 layers
Processing layer-1             done | 00:00:00
Processing layer-2             done | 00:00:00
Predict complete                    | 00:00:00
Precisión:  85.79
Exhaustividad:  89.39
Puntuación F1:  87.53
Exactitud:  89.39
Información Mutua Normalizada (NMI): 58.56
Índice Kappa de Cohen: 79.28


,MÉTRICA,VALOR,MODELO,TIPO_DE_DATOS
0,Precisión,85.79,Super_Aprendiz,Entrenamiento
1,Exhaustividad,89.39,Super_Aprendiz,Entrenamiento
2,Puntuación F1,87.53,Super_Aprendiz,Entrenamiento
3,Exactitud,89.39,Super_Aprendiz,Entrenamiento
4,Información Mutua Normalizada (NMI),58.56,Super_Aprendiz,Entrenamiento
5,Índice Kappa de Cohen,79.28,Super_Aprendiz,Entrenamiento


In [93]:
# Predecir las etiquetas para los datos de prueba
resultados_df_superaprendiz = pd.DataFrame(columns=['MÉTRICA', 'VALOR'])

Y_pred_prueba = modelo_superaprendiz.predict(X_tst)

precision = precision_score(Y_tst, Y_pred_prueba, average='weighted')
recall = recall_score(Y_tst, Y_pred_prueba, average='weighted')
f1 = f1_score(Y_tst, Y_pred_prueba, average='weighted')
accuracy = accuracy_score(Y_tst, Y_pred_prueba)
nmi = normalized_mutual_info_score(Y_tst, Y_pred_prueba)
kappa = cohen_kappa_score(Y_tst, Y_pred_prueba)
print("Precisión: ", round(precision*100,2))
print("Exhaustividad: ", round(recall*100,2))
print("Puntuación F1: ", round(f1*100,2))
print("Exactitud: ", round(accuracy*100,2))
print("Información Mutua Normalizada (NMI):", round(nmi*100,2))
print("Índice Kappa de Cohen:", round(kappa*100,2))

# Crear un DataFrame para cada métrica
df_precision = pd.DataFrame({'MÉTRICA': ['Precisión'], 'VALOR': [round(precision*100, 2)]})
df_recall = pd.DataFrame({'MÉTRICA': ['Exhaustividad'], 'VALOR': [round(recall*100, 2)]})
df_f1 = pd.DataFrame({'MÉTRICA': ['Puntuación F1'], 'VALOR': [round(f1*100, 2)]})
df_accuracy = pd.DataFrame({'MÉTRICA': ['Exactitud'], 'VALOR': [round(accuracy*100, 2)]})
df_nmi=pd.DataFrame({'MÉTRICA': ['Información Mutua Normalizada (NMI)'], 'VALOR': [round(nmi*100, 2)]})
df_kappa=pd.DataFrame({'MÉTRICA': ['Índice Kappa de Cohen'], 'VALOR': [round(kappa*100, 2)]})

# Concatenar los DataFrames
resultados_df_superaprendiz_prueba = pd.concat([resultados_df_superaprendiz, df_precision, df_recall, df_f1, df_accuracy,df_nmi,df_kappa], ignore_index=True)
resultados_df_superaprendiz_prueba["MODELO"]='Super_aprendiz'
resultados_df_superaprendiz_prueba["TIPO_DE_DATOS"]='Prueba'
# Imprimir el DataFrame con los resultados
resultados_df_superaprendiz_prueba


Predicting 2 layers
Processing layer-1             done | 00:00:00
Processing layer-2             done | 00:00:00
Predict complete                    | 00:00:00
Precisión:  80.56
Exhaustividad:  83.71
Puntuación F1:  82.09
Exactitud:  83.71
Información Mutua Normalizada (NMI): 40.34
Índice Kappa de Cohen: 68.17


,MÉTRICA,VALOR,MODELO,TIPO_DE_DATOS
0,Precisión,80.56,Super_aprendiz,Prueba
1,Exhaustividad,83.71,Super_aprendiz,Prueba
2,Puntuación F1,82.09,Super_aprendiz,Prueba
3,Exactitud,83.71,Super_aprendiz,Prueba
4,Información Mutua Normalizada (NMI),40.34,Super_aprendiz,Prueba
5,Índice Kappa de Cohen,68.17,Super_aprendiz,Prueba


In [94]:
resultados_df_superaprendiz = pd.concat([resultados_df_superaprendiz_prueba,resultados_df_superaprendiz_entrenamiento], ignore_index=True)
resultados_df_superaprendiz

,MÉTRICA,VALOR,MODELO,TIPO_DE_DATOS
0,Precisión,80.56,Super_aprendiz,Prueba
1,Exhaustividad,83.71,Super_aprendiz,Prueba
2,Puntuación F1,82.09,Super_aprendiz,Prueba
3,Exactitud,83.71,Super_aprendiz,Prueba
4,Información Mutua Normalizada (NMI),40.34,Super_aprendiz,Prueba
5,Índice Kappa de Cohen,68.17,Super_aprendiz,Prueba
6,Precisión,85.79,Super_Aprendiz,Entrenamiento
7,Exhaustividad,89.39,Super_Aprendiz,Entrenamiento
8,Puntuación F1,87.53,Super_Aprendiz,Entrenamiento
9,Exactitud,89.39,Super_Aprendiz,Entrenamiento


### SUPER APRENDIZ MULTICAPA

In [95]:
Y_trn = Y_trn.astype(int)
def entrenar_modelo_super_aprendiz_dos_capas(X_trn, Y_trn):
    X_trn_transformado = X_trn
    semilla = 7 
    kfold = KFold(n_splits=10, random_state=semilla, shuffle=True)
    modelo3 = ExtraTreesClassifier(n_estimators=400, max_features=None, bootstrap=False, 
                                   max_depth=11, min_samples_split=4, min_samples_leaf=1)

    modelo4 = RandomForestClassifier(n_estimators=500, max_features=None,
                                     min_samples_split=5, max_depth=3, min_samples_leaf=1)
    model = DecisionTreeClassifier(criterion='gini', 
                                   max_depth=6, max_features=None,  min_samples_leaf=9, 
                                   min_samples_split=2, 
                                   random_state=10, splitter='random')

    modelo5 = BaggingClassifier(estimator=model, n_estimators=50, random_state=semilla,
                                bootstrap=True, bootstrap_features=False, 
                                max_features=0.9, max_samples=0.95)

    modelo6 = DecisionTreeClassifier(criterion='gini', max_depth=6, max_features=None, 
                                     min_samples_leaf=9, min_samples_split=2, 
                                     random_state=10, splitter='random')
    
    estimadores = [('Extratrees', modelo3), ('Random Forest', modelo4), 
                   ('Bagging', modelo5), ('Decision tree', modelo6)]
    
    super_learner = SuperLearner(folds=10, random_state=semilla, verbose=2)
    super_learner.add(estimadores)
    super_learner.add(estimadores)
    estimador_final = ExtraTreesClassifier(n_estimators=100, max_features=None,
                                           bootstrap=False, max_depth=11, min_samples_split=4, 
                                           min_samples_leaf=1)
    
    super_learner.add_meta(estimador_final)
    
    super_learner.fit(X_trn_transformado, Y_trn)
    resultados = cross_val_score(super_learner, X_trn_transformado, Y_trn, cv=kfold, scoring='accuracy')
    
    print("Rendimiento del modelo:") 
    print("Promedio de Exactitud (DATOS ENTRENAMIENTO) usando k-fold:", resultados.mean() * 100, "%") 
    return super_learner


X_trn = X_trn
Y_trn = Y_trn 

modelo_superaprendiz_dos_capas = entrenar_modelo_super_aprendiz_dos_capas(X_trn, Y_trn)


Fitting 3 layers
Processing layer-1             done | 00:00:14
Processing layer-2             done | 00:00:12
Processing layer-3             done | 00:00:00
Fit complete                        | 00:00:27

Fitting 3 layers
Processing layer-1             done | 00:00:14
Processing layer-2             done | 00:00:10
Processing layer-3             done | 00:00:00
Fit complete                        | 00:00:25

Predicting 3 layers
Processing layer-1             done | 00:00:00
Processing layer-2             done | 00:00:00
Processing layer-3             done | 00:00:00
Predict complete                    | 00:00:00

Fitting 3 layers
Processing layer-1             done | 00:00:14
Processing layer-2             done | 00:00:10
Processing layer-3             done | 00:00:00
Fit complete                        | 00:00:25

Predicting 3 layers
Processing layer-1             done | 00:00:00
Processing layer-2             done | 00:00:00
Processing layer-3             done | 00:00:00
Predict com

In [96]:
print(modelo_superaprendiz_dos_capas.data)

                          ft-m  ft-s  pt-m  pt-s
layer-1  bagging          2.15  0.19  0.37  0.26
layer-1  decision-tree    0.06  0.04  0.01  0.02
layer-1  extratrees       9.30  0.26  0.84  0.18
layer-1  random-forest    9.37  2.54  0.30  0.15
layer-2  bagging          1.79  0.11  0.43  0.21
layer-2  decision-tree    0.09  0.13  0.01  0.02
layer-2  extratrees       7.59  0.27  0.80  0.20
layer-2  random-forest    8.04  2.12  0.29  0.14



In [97]:
resultados_df_superaprendiz_dos_capas = pd.DataFrame(columns=['MÉTRICA', 'VALOR'])

Y_pred_entrenamiento= modelo_superaprendiz_dos_capas.predict(X_trn)

precision = precision_score(Y_trn, Y_pred_entrenamiento, average='weighted')
recall = recall_score(Y_trn, Y_pred_entrenamiento, average='weighted')
f1 = f1_score(Y_trn, Y_pred_entrenamiento, average='weighted')
accuracy = accuracy_score(Y_trn, Y_pred_entrenamiento)
nmi = normalized_mutual_info_score(Y_trn, Y_pred_entrenamiento)
kappa = cohen_kappa_score(Y_trn, Y_pred_entrenamiento)
print("Precisión: ", round(precision*100,2))
print("Exhaustividad: ", round(recall*100,2))
print("Puntuación F1: ", round(f1*100,2))
print("Exactitud: ", round(accuracy*100,2))
print("Información Mutua Normalizada (NMI):", round(nmi*100,2))
print("Índice Kappa de Cohen:", round(kappa*100,2))

# Crear un DataFrame para cada métrica
df_precision = pd.DataFrame({'MÉTRICA': ['Precisión'], 'VALOR': [round(precision*100, 2)]})
df_recall = pd.DataFrame({'MÉTRICA': ['Exhaustividad'], 'VALOR': [round(recall*100, 2)]})
df_f1 = pd.DataFrame({'MÉTRICA': ['Puntuación F1'], 'VALOR': [round(f1*100, 2)]})
df_accuracy = pd.DataFrame({'MÉTRICA': ['Exactitud'], 'VALOR': [round(accuracy*100, 2)]})
df_nmi=pd.DataFrame({'MÉTRICA': ['Información Mutua Normalizada (NMI)'], 'VALOR': [round(nmi*100, 2)]})
df_kappa=pd.DataFrame({'MÉTRICA': ['Índice Kappa de Cohen'], 'VALOR': [round(kappa*100, 2)]})

# Concatenar los DataFrames
resultados_df_superaprendiz_dos_capas_entrenamiento = pd.concat([resultados_df_superaprendiz_dos_capas, df_precision, df_recall, df_f1, df_accuracy,df_nmi,df_kappa], ignore_index=True)
resultados_df_superaprendiz_dos_capas_entrenamiento["MODELO"]='Super_Aprendiz_dos_capas'
resultados_df_superaprendiz_dos_capas_entrenamiento["TIPO_DE_DATOS"]='Entrenamiento'
# Imprimir el DataFrame con los resultados
resultados_df_superaprendiz_dos_capas_entrenamiento


Predicting 3 layers
Processing layer-1             done | 00:00:00
Processing layer-2             done | 00:00:00
Processing layer-3             done | 00:00:00
Predict complete                    | 00:00:00
Precisión:  82.46
Exhaustividad:  85.76
Puntuación F1:  83.97
Exactitud:  85.76
Información Mutua Normalizada (NMI): 47.21
Índice Kappa de Cohen: 72.09


,MÉTRICA,VALOR,MODELO,TIPO_DE_DATOS
0,Precisión,82.46,Super_Aprendiz_dos_capas,Entrenamiento
1,Exhaustividad,85.76,Super_Aprendiz_dos_capas,Entrenamiento
2,Puntuación F1,83.97,Super_Aprendiz_dos_capas,Entrenamiento
3,Exactitud,85.76,Super_Aprendiz_dos_capas,Entrenamiento
4,Información Mutua Normalizada (NMI),47.21,Super_Aprendiz_dos_capas,Entrenamiento
5,Índice Kappa de Cohen,72.09,Super_Aprendiz_dos_capas,Entrenamiento


In [98]:
# Predecir las etiquetas para los datos de prueba
resultados_df_superaprendiz_dos_capas = pd.DataFrame(columns=['MÉTRICA', 'VALOR'])

Y_pred_prueba = modelo_superaprendiz_dos_capas.predict(X_tst)

precision = precision_score(Y_tst, Y_pred_prueba, average='weighted')
recall = recall_score(Y_tst, Y_pred_prueba, average='weighted')
f1 = f1_score(Y_tst, Y_pred_prueba, average='weighted')
accuracy = accuracy_score(Y_tst, Y_pred_prueba)
nmi = normalized_mutual_info_score(Y_tst, Y_pred_prueba)
kappa = cohen_kappa_score(Y_tst, Y_pred_prueba)
print("Precisión: ", round(precision*100,2))
print("Exhaustividad: ", round(recall*100,2))
print("Puntuación F1: ", round(f1*100,2))
print("Exactitud: ", round(accuracy*100,2))
print("Información Mutua Normalizada (NMI):", round(nmi*100,2))
print("Índice Kappa de Cohen:", round(kappa*100,2))

# Crear un DataFrame para cada métrica
df_precision = pd.DataFrame({'MÉTRICA': ['Precisión'], 'VALOR': [round(precision*100, 2)]})
df_recall = pd.DataFrame({'MÉTRICA': ['Exhaustividad'], 'VALOR': [round(recall*100, 2)]})
df_f1 = pd.DataFrame({'MÉTRICA': ['Puntuación F1'], 'VALOR': [round(f1*100, 2)]})
df_accuracy = pd.DataFrame({'MÉTRICA': ['Exactitud'], 'VALOR': [round(accuracy*100, 2)]})
df_nmi=pd.DataFrame({'MÉTRICA': ['Información Mutua Normalizada (NMI)'], 'VALOR': [round(nmi*100, 2)]})
df_kappa=pd.DataFrame({'MÉTRICA': ['Índice Kappa de Cohen'], 'VALOR': [round(kappa*100, 2)]})

# Concatenar los DataFrames
resultados_df_superaprendiz_dos_capas_prueba = pd.concat([resultados_df_superaprendiz_dos_capas, df_precision, df_recall, df_f1, df_accuracy,df_nmi,df_kappa], ignore_index=True)
resultados_df_superaprendiz_dos_capas_prueba["MODELO"]='Super_aprendiz_dos_capas'
resultados_df_superaprendiz_dos_capas_prueba["TIPO_DE_DATOS"]='Prueba'
# Imprimir el DataFrame con los resultados
resultados_df_superaprendiz_dos_capas_prueba


Predicting 3 layers
Processing layer-1             done | 00:00:00
Processing layer-2             done | 00:00:00
Processing layer-3             done | 00:00:00
Predict complete                    | 00:00:00
Precisión:  79.88
Exhaustividad:  82.95
Puntuación F1:  81.34
Exactitud:  82.95
Información Mutua Normalizada (NMI): 38.33
Índice Kappa de Cohen: 66.6


,MÉTRICA,VALOR,MODELO,TIPO_DE_DATOS
0,Precisión,79.88,Super_aprendiz_dos_capas,Prueba
1,Exhaustividad,82.95,Super_aprendiz_dos_capas,Prueba
2,Puntuación F1,81.34,Super_aprendiz_dos_capas,Prueba
3,Exactitud,82.95,Super_aprendiz_dos_capas,Prueba
4,Información Mutua Normalizada (NMI),38.33,Super_aprendiz_dos_capas,Prueba
5,Índice Kappa de Cohen,66.60,Super_aprendiz_dos_capas,Prueba


In [99]:
resultados_df_superaprendiz_dos_capas = pd.concat([resultados_df_superaprendiz_dos_capas_prueba,resultados_df_superaprendiz_dos_capas_entrenamiento], ignore_index=True)
resultados_df_superaprendiz_dos_capas

,MÉTRICA,VALOR,MODELO,TIPO_DE_DATOS
0,Precisión,79.88,Super_aprendiz_dos_capas,Prueba
1,Exhaustividad,82.95,Super_aprendiz_dos_capas,Prueba
2,Puntuación F1,81.34,Super_aprendiz_dos_capas,Prueba
3,Exactitud,82.95,Super_aprendiz_dos_capas,Prueba
4,Información Mutua Normalizada (NMI),38.33,Super_aprendiz_dos_capas,Prueba
5,Índice Kappa de Cohen,66.60,Super_aprendiz_dos_capas,Prueba
6,Precisión,82.46,Super_Aprendiz_dos_capas,Entrenamiento
7,Exhaustividad,85.76,Super_Aprendiz_dos_capas,Entrenamiento
8,Puntuación F1,83.97,Super_Aprendiz_dos_capas,Entrenamiento
9,Exactitud,85.76,Super_Aprendiz_dos_capas,Entrenamiento


### RED NEURONAL

In [100]:
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

class NeuralNetwork(nn.Module):
    def __init__(self, input_size, hidden_size, num_classes):
        super(NeuralNetwork, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_size)
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(hidden_size, num_classes)
    
    def forward(self, x):
        out = self.fc1(x)
        out = self.relu(out)
        out = self.fc2(out)
        return out

def train_neural_network(X_train, y_train, input_size, hidden_size, num_classes, num_epochs=100, learning_rate=0.001):
    # Convertir datos de entrenamiento a tensores de PyTorch
    X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
    y_train_tensor = torch.tensor(y_train, dtype=torch.long)
    
    # Inicializar modelo y función de pérdida
    model = NeuralNetwork(input_size, hidden_size, num_classes)
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=learning_rate)
    
    # Entrenamiento del modelo
    for epoch in range(num_epochs):
        # Forward pass y pérdida
        outputs = model(X_train_tensor)
        loss = criterion(outputs, y_train_tensor)
        
        # Backward pass y optimización
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        if (epoch+1) % 10 == 0:
            print ('Epoch [{}/{}], Loss: {:.4f}'.format(epoch+1, num_epochs, loss.item()))

    return model

def evaluate_model(model, X_test, y_test):
    # Convertir datos de prueba a tensores de PyTorch
    X_test_tensor = torch.tensor(X_test, dtype=torch.float32)
    y_test_tensor = torch.tensor(y_test, dtype=torch.long)

    # Obtener las predicciones del modelo
    outputs = model(X_test_tensor)
    _, predicted = torch.max(outputs, 1)

    # Calcular las métricas del modelo
    accuracy = accuracy_score(y_test_tensor.numpy(), predicted.numpy())
    precision = precision_score(y_test, predicted.numpy(), average='weighted')
    recall = recall_score(y_test, predicted.numpy(), average='weighted')
    f1 = f1_score(y_test, predicted.numpy(), average='weighted')

    print('Accuracy: {:.2f}%'.format(accuracy * 100))
    print("Precisión: ", precision)
    print("Exhaustividad (Recall): ", recall)
    print("Puntuación F1: ", f1)

X_train_array = X_trn.values
X_test_array = X_tst.values
Y_trn = Y_trn 
input_size = X_trn.shape[1]
num_classes = 4
hidden_size = 170

# Supongamos que X_train, y_train, X_test, y_test son tus datos de entrenamiento y prueba
modelo_red_reuronal = train_neural_network(X_train_array , Y_trn, input_size, hidden_size, num_classes)
evaluate_model(modelo_red_reuronal, X_test_array , Y_tst)


Epoch [10/100], Loss: 1.0663
Epoch [20/100], Loss: 0.7840
Epoch [30/100], Loss: 0.6383
Epoch [40/100], Loss: 0.5651
Epoch [50/100], Loss: 0.5209
Epoch [60/100], Loss: 0.4894
Epoch [70/100], Loss: 0.4667
Epoch [80/100], Loss: 0.4501
Epoch [90/100], Loss: 0.4375
Epoch [100/100], Loss: 0.4274
Accuracy: 80.68%
Precisión:  0.7808902641588664
Exhaustividad (Recall):  0.8068181818181818
Puntuación F1:  0.7902897467970968


In [101]:
import pandas as pd
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, normalized_mutual_info_score, cohen_kappa_score

# Predecir las etiquetas para los datos de entrenamiento
resultados_df_red_reuronal = pd.DataFrame(columns=['MÉTRICA', 'VALOR'])

# Convertir datos de entrenamiento a tensores de PyTorch
X_train_tensor = torch.tensor(X_train_array, dtype=torch.float32)

# Obtener las predicciones del modelo
outputs = modelo_red_reuronal(X_train_tensor)
_, predicted = torch.max(outputs, 1)
Y_pred_entrenamiento = predicted.numpy()

precision = precision_score(Y_trn, Y_pred_entrenamiento, average='weighted')
recall = recall_score(Y_trn, Y_pred_entrenamiento, average='weighted')
f1 = f1_score(Y_trn, Y_pred_entrenamiento, average='weighted')
accuracy = accuracy_score(Y_trn, Y_pred_entrenamiento)
nmi = normalized_mutual_info_score(Y_trn, Y_pred_entrenamiento)
kappa = cohen_kappa_score(Y_trn, Y_pred_entrenamiento)

print("Precisión: ", round(precision*100, 2))
print("Exhaustividad: ", round(recall*100, 2))
print("Puntuación F1: ", round(f1*100, 2))
print("Exactitud: ", round(accuracy*100, 2))
print("Información Mutua Normalizada (NMI):", round(nmi*100, 2))
print("Índice Kappa de Cohen:", round(kappa*100, 2))

# Crear un DataFrame para cada métrica
df_precision = pd.DataFrame({'MÉTRICA': ['Precisión'], 'VALOR': [round(precision*100, 2)]})
df_recall = pd.DataFrame({'MÉTRICA': ['Exhaustividad'], 'VALOR': [round(recall*100, 2)]})
df_f1 = pd.DataFrame({'MÉTRICA': ['Puntuación F1'], 'VALOR': [round(f1*100, 2)]})
df_accuracy = pd.DataFrame({'MÉTRICA': ['Exactitud'], 'VALOR': [round(accuracy*100, 2)]})
df_nmi = pd.DataFrame({'MÉTRICA': ['Información Mutua Normalizada (NMI)'], 'VALOR': [round(nmi*100, 2)]})
df_kappa = pd.DataFrame({'MÉTRICA': ['Índice Kappa de Cohen'], 'VALOR': [round(kappa*100, 2)]})

# Concatenar los DataFrames
resultados_df_red_reuronal_entrenamiento = pd.concat([resultados_df_red_reuronal, df_precision, df_recall, df_f1, df_accuracy, df_nmi, df_kappa], ignore_index=True)
resultados_df_red_reuronal_entrenamiento["MODELO"] = 'Red_Neuronal'
resultados_df_red_reuronal_entrenamiento["TIPO_DE_DATOS"] = 'Entrenamiento'

# Imprimir el DataFrame con los resultados
resultados_df_red_reuronal_entrenamiento


Precisión:  80.45
Exhaustividad:  83.6
Puntuación F1:  81.83
Exactitud:  83.6
Información Mutua Normalizada (NMI): 41.44
Índice Kappa de Cohen: 67.79


,MÉTRICA,VALOR,MODELO,TIPO_DE_DATOS
0,Precisión,80.45,Red_Neuronal,Entrenamiento
1,Exhaustividad,83.60,Red_Neuronal,Entrenamiento
2,Puntuación F1,81.83,Red_Neuronal,Entrenamiento
3,Exactitud,83.60,Red_Neuronal,Entrenamiento
4,Información Mutua Normalizada (NMI),41.44,Red_Neuronal,Entrenamiento
5,Índice Kappa de Cohen,67.79,Red_Neuronal,Entrenamiento


In [102]:
# Predecir las etiquetas para los datos de prueba
resultados_df_red_reuronal_prueba = pd.DataFrame(columns=['MÉTRICA', 'VALOR'])

# Convertir datos de prueba a tensores de PyTorch
X_test_tensor = torch.tensor(X_test_array, dtype=torch.float32)

# Obtener las predicciones del modelo
outputs = modelo_red_reuronal(X_test_tensor)
_, predicted = torch.max(outputs, 1)
Y_pred_prueba = predicted.numpy()

precision = precision_score(Y_tst, Y_pred_prueba, average='weighted')
recall = recall_score(Y_tst, Y_pred_prueba, average='weighted')
f1 = f1_score(Y_tst, Y_pred_prueba, average='weighted')
accuracy = accuracy_score(Y_tst, Y_pred_prueba)
nmi = normalized_mutual_info_score(Y_tst, Y_pred_prueba)
kappa = cohen_kappa_score(Y_tst, Y_pred_prueba)

print("Precisión: ", round(precision*100, 2))
print("Exhaustividad: ", round(recall*100, 2))
print("Puntuación F1: ", round(f1*100, 2))
print("Exactitud: ", round(accuracy*100, 2))
print("Información Mutua Normalizada (NMI):", round(nmi*100, 2))
print("Índice Kappa de Cohen:", round(kappa*100, 2))

# Crear un DataFrame para cada métrica
df_precision = pd.DataFrame({'MÉTRICA': ['Precisión'], 'VALOR': [round(precision*100, 2)]})
df_recall = pd.DataFrame({'MÉTRICA': ['Exhaustividad'], 'VALOR': [round(recall*100, 2)]})
df_f1 = pd.DataFrame({'MÉTRICA': ['Puntuación F1'], 'VALOR': [round(f1*100, 2)]})
df_accuracy = pd.DataFrame({'MÉTRICA': ['Exactitud'], 'VALOR': [round(accuracy*100, 2)]})
df_nmi = pd.DataFrame({'MÉTRICA': ['Información Mutua Normalizada (NMI)'], 'VALOR': [round(nmi*100, 2)]})
df_kappa = pd.DataFrame({'MÉTRICA': ['Índice Kappa de Cohen'], 'VALOR': [round(kappa*100, 2)]})

# Concatenar los DataFrames
resultados_df_red_reuronal_prueba = pd.concat([resultados_df_red_reuronal_prueba, df_precision, df_recall, df_f1, df_accuracy, df_nmi, df_kappa], ignore_index=True)
resultados_df_red_reuronal_prueba["MODELO"] = 'Red_Neuronal'
resultados_df_red_reuronal_prueba["TIPO_DE_DATOS"] = 'Prueba'

# Imprimir el DataFrame con los resultados
resultados_df_red_reuronal_prueba


Precisión:  78.09
Exhaustividad:  80.68
Puntuación F1:  79.03
Exactitud:  80.68
Información Mutua Normalizada (NMI): 33.23
Índice Kappa de Cohen: 61.82


,MÉTRICA,VALOR,MODELO,TIPO_DE_DATOS
0,Precisión,78.09,Red_Neuronal,Prueba
1,Exhaustividad,80.68,Red_Neuronal,Prueba
2,Puntuación F1,79.03,Red_Neuronal,Prueba
3,Exactitud,80.68,Red_Neuronal,Prueba
4,Información Mutua Normalizada (NMI),33.23,Red_Neuronal,Prueba
5,Índice Kappa de Cohen,61.82,Red_Neuronal,Prueba


In [103]:
resultados_df_red_reuronal = pd.concat([resultados_df_red_reuronal_prueba,resultados_df_red_reuronal_entrenamiento], ignore_index=True)
resultados_df_red_reuronal

,MÉTRICA,VALOR,MODELO,TIPO_DE_DATOS
0,Precisión,78.09,Red_Neuronal,Prueba
1,Exhaustividad,80.68,Red_Neuronal,Prueba
2,Puntuación F1,79.03,Red_Neuronal,Prueba
3,Exactitud,80.68,Red_Neuronal,Prueba
4,Información Mutua Normalizada (NMI),33.23,Red_Neuronal,Prueba
5,Índice Kappa de Cohen,61.82,Red_Neuronal,Prueba
6,Precisión,80.45,Red_Neuronal,Entrenamiento
7,Exhaustividad,83.60,Red_Neuronal,Entrenamiento
8,Puntuación F1,81.83,Red_Neuronal,Entrenamiento
9,Exactitud,83.60,Red_Neuronal,Entrenamiento
